# Baroque Ceiling Paintings - DuckDB Analysis

This notebook loads the Parquet database created from NFDI4Culture and CbDD data into a **persistent DuckDB database** for collaborative analysis.

## Data Sources
- **CbDD (Primary)**: NFDI4Culture Knowledge Graph (SPARQL endpoint)
  - 6,228 baroque ceiling paintings with metadata
  - Enrichment from CbDD graphData.json (persons, buildings, rooms)
  - Subject Resolution via ICONCLASS & Getty AAT
  
- **Bildindex (Secondary)**: Historical art photographs from Bildindex (n4c:E6161)
  - Cross-referenced via shared GND identifiers (painters & buildings)
  - Enables linking ceiling paintings to historical photographs

## Tech Stack
- **DuckDB**: Persistent analytical database with SQL interface
- **Polars**: High-performance DataFrame operations
- **Altair**: Declarative visualizations

## Table Overview
| Prefix | Source | Description |
|--------|--------|-------------|
| (none) | CbDD | Core tables: paintings, persons, buildings, rooms, subjects, etc. |
| `bi_` | Bildindex | Cross-reference tables: bi_items, bi_buildings, bi_painters, etc. |

In [2]:
# =============================================================================
# CELL 1: Setup & Configuration
# =============================================================================
import duckdb
import polars as pl
import altair as alt
import re
from pathlib import Path
from typing import Tuple, Optional
from IPython.display import display as ipython_display

# Configuration
PARQUET_DIR = Path(r"c:/Users/thano/Documents/_Studium/KIT/DataStories/DataStories")
DUCKDB_PATH = PARQUET_DIR / "baroque.duckdb"
PREFIX = "baroque_"
BILDINDEX_PREFIX = "bildindex_"

# CbDD Table names (from baroque_ parquet files)
CBDD_TABLES = [
    'paintings', 'persons', 'buildings', 'rooms', 'ensembles',
    'subjects', 'painting_persons', 'painting_subjects',
    'building_persons', 'room_persons'
]

# Bildindex Table names (from bildindex_ parquet files)
BILDINDEX_TABLES = [
    'items', 'buildings', 'painters', 'subjects', 'gnd_overlaps'
]

# Combined for backward compatibility
TABLES = CBDD_TABLES

# Enable Altair for larger datasets
alt.data_transformers.disable_max_rows()

print(f"📁 Parquet directory: {PARQUET_DIR}")
print(f"🗄️ DuckDB database: {DUCKDB_PATH}")
print(f"📊 CbDD tables: {len(CBDD_TABLES)}")
print(f"📊 Bildindex tables: {len(BILDINDEX_TABLES)}")

📁 Parquet directory: c:\Users\thano\Documents\_Studium\KIT\DataStories\DataStories
🗄️ DuckDB database: c:\Users\thano\Documents\_Studium\KIT\DataStories\DataStories\baroque.duckdb
📊 CbDD tables: 10
📊 Bildindex tables: 5


In [3]:
# =============================================================================
# CELL 2: German Year Parsing Utility
# =============================================================================
# Handles formats like:
#   - "1732" → (1732, 1732, False)
#   - "1732-1742" → (1732, 1742, False)
#   - "um 1732" / "ca. 1579" → (1732, 1732, True)
#   - "um 1732-1742" → (1732, 1742, True)
#   - "17. Jh" / "17. Jahrhundert" → (1600, 1699, False)
#   - "2. Hälfte 17. Jahrhundert" → (1650, 1699, False)
#   - "1. Hälfte 18. Jh." → (1700, 1749, False)
#   - "Anfang 18. Jh." → (1700, 1730, True)
#   - "Ende 17. Jh." → (1670, 1699, True)
#   - "Mitte 18. Jahrhundert" → (1740, 1760, True)
#   - "vor 1750" → (None, 1750, True)
#   - "nach 1700" → (1700, None, True)
#   - "bis 1716" → (None, 1716, False)

def parse_german_year(year_str: Optional[str]) -> Tuple[Optional[int], Optional[int], bool, bool]:
    """
    Parse German year strings into structured data.
    
    Returns:
        (year_start, year_end, is_approximate, is_parsed)
        - year_start: Earliest possible year (or None)
        - year_end: Latest possible year (or None)
        - is_approximate: True if "um", "ca.", "circa", etc.
        - is_parsed: True if successfully parsed, False if unparseable
    """
    if year_str is None or str(year_str).strip() == '':
        return (None, None, False, False)
    
    text = str(year_str).strip().lower()
    original_text = text
    
    # Track approximation markers
    is_approx = False
    approx_markers = ['um ', 'ca. ', 'ca ', 'circa ', 'etwa ', 'wohl ', 'vermutlich ']
    for marker in approx_markers:
        if marker in text:
            is_approx = True
            text = text.replace(marker, '')
    
    # Remove common noise
    text = re.sub(r'\s+', ' ', text)  # Normalize whitespace
    text = text.replace('jhd', 'jh').replace('jahrhundert', 'jh')
    
    # Pattern 1: Simple year "1732"
    match = re.search(r'^(\d{4})$', text.strip())
    if match:
        year = int(match.group(1))
        return (year, year, is_approx, True)
    
    # Pattern 2: Year range "1732-1742" or "1732–1742" or "1732/1742"
    match = re.search(r'(\d{4})\s*[-–/]\s*(\d{4})', text)
    if match:
        return (int(match.group(1)), int(match.group(2)), is_approx, True)
    
    # Pattern 3: Century "17. jh" or "17.jh."
    match = re.search(r'(\d{1,2})\.?\s*jh', text)
    if match:
        century = int(match.group(1))
        base_year = (century - 1) * 100
        
        # Check for half-century modifiers
        if '2. hälfte' in text or 'zweite hälfte' in text:
            return (base_year + 50, base_year + 99, is_approx, True)
        elif '1. hälfte' in text or 'erste hälfte' in text:
            return (base_year, base_year + 49, is_approx, True)
        elif 'anfang' in text or 'beginn' in text or 'früh' in text:
            return (base_year, base_year + 30, True, True)
        elif 'ende' in text or 'spät' in text:
            return (base_year + 70, base_year + 99, True, True)
        elif 'mitte' in text:
            return (base_year + 40, base_year + 60, True, True)
        else:
            return (base_year, base_year + 99, is_approx, True)
    
    # Pattern 4: "vor 1750" (before)
    match = re.search(r'vor\s+(\d{4})', text)
    if match:
        return (None, int(match.group(1)), True, True)
    
    # Pattern 5: "nach 1700" (after) or "ab 1700"
    match = re.search(r'(?:nach|ab)\s+(\d{4})', text)
    if match:
        return (int(match.group(1)), None, True, True)
    
    # Pattern 6: "bis 1716" (until)
    match = re.search(r'bis\s+(\d{4})', text)
    if match:
        return (None, int(match.group(1)), is_approx, True)
    
    # Pattern 7: Extract first 4-digit year as fallback
    match = re.search(r'(\d{4})', text)
    if match:
        year = int(match.group(1))
        # Check if there's a second year
        all_years = re.findall(r'(\d{4})', text)
        if len(all_years) >= 2:
            years = [int(y) for y in all_years]
            return (min(years), max(years), is_approx, True)
        return (year, year, is_approx, True)
    
    # Unparseable
    return (None, None, False, False)


# Test the parser
test_cases = [
    "1732",
    "1732-1742",
    "um 1732-1742",
    "ca. 1579",
    "17. Jh",
    "2. Hälfte 17. Jahrhundert",
    "Anfang 18. Jh.",
    "vor 1750",
    "nach 1700",
    "bis 1716, Weihe 1736",
    "1697-1749, 1782",
    "wohl 18. Jahrhundert",
    "mittelalterlich, 1730-1734",
    None,
    ""
]

print("🧪 Year Parser Test Results:")
print("=" * 80)
print(f"{'Input':<40} {'Start':>6} {'End':>6} {'Approx':>7} {'Parsed':>7}")
print("-" * 80)
for tc in test_cases:
    start, end, approx, parsed = parse_german_year(tc)
    display = str(tc)[:38] if tc else "(None)"
    start_s = str(start) if start else "-"
    end_s = str(end) if end else "-"
    print(f"{display:<40} {start_s:>6} {end_s:>6} {str(approx):>7} {str(parsed):>7}")

🧪 Year Parser Test Results:
Input                                     Start    End  Approx  Parsed
--------------------------------------------------------------------------------
1732                                       1732   1732   False    True
1732-1742                                  1732   1742   False    True
um 1732-1742                               1732   1742    True    True
ca. 1579                                   1579   1579    True    True
17. Jh                                     1600   1699   False    True
2. Hälfte 17. Jahrhundert                  1650   1699   False    True
Anfang 18. Jh.                             1700   1730    True    True
vor 1750                                      -   1750    True    True
nach 1700                                  1700      -    True    True
bis 1716, Weihe 1736                          -   1716   False    True
1697-1749, 1782                            1697   1749   False    True
wohl 18. Jahrhundert                   

In [4]:
# =============================================================================
# CELL 1b: Reconnect to Existing Database (skip if creating fresh)
# =============================================================================
# Use this cell to reconnect to an existing database without rebuilding

if DUCKDB_PATH.exists():
    con = duckdb.connect(str(DUCKDB_PATH), read_only=True)
    print(f"✅ Reconnected to existing database: {DUCKDB_PATH.name} (read-only)")
    
    # Show table counts grouped by source
    tables_df = con.execute("SHOW TABLES").pl()
    print(f"📋 Found {len(tables_df)} tables:")
    
    cbdd_tables = []
    bi_tables = []
    for row in tables_df.iter_rows(named=True):
        count = con.execute(f"SELECT COUNT(*) FROM {row['name']}").fetchone()[0]
        if row['name'].startswith('bi_'):
            bi_tables.append((row['name'], count))
        else:
            cbdd_tables.append((row['name'], count))
    
    print("\n   📘 CbDD Tables:")
    for name, count in cbdd_tables:
        print(f"      {name:<25}: {count:>6,} rows")
    
    if bi_tables:
        print("\n   📗 Bildindex Tables:")
        for name, count in bi_tables:
            print(f"      {name:<25}: {count:>6,} rows")
else:
    print(f"⚠️ Database not found. Run cells 2-5 to create it.")
    con = None

✅ Reconnected to existing database: baroque.duckdb (read-only)
📋 Found 15 tables:

   📘 CbDD Tables:
      building_persons         :  1,983 rows
      buildings                :  1,260 rows
      ensembles                :     32 rows
      painting_persons         :  5,848 rows
      painting_subjects        : 17,474 rows
      paintings                :  4,594 rows
      persons                  :  2,831 rows
      room_persons             :  4,676 rows
      rooms                    :  2,376 rows
      subjects                 :  4,082 rows

   📗 Bildindex Tables:
      bi_buildings             :    215 rows
      bi_gnd_overlaps          :    433 rows
      bi_items                 :    968 rows
      bi_painters              :    753 rows
      bi_subjects              :    964 rows


In [5]:
# =============================================================================
# CELL 3: Create Persistent DuckDB Database
# =============================================================================
# This creates/opens the baroque.duckdb file for persistent storage

# Close any existing connection first
try:
    con.close()
    print("🔌 Closed existing connection")
except:
    pass

# Remove existing database to recreate fresh (comment out to append)
if DUCKDB_PATH.exists():
    try:
        DUCKDB_PATH.unlink()
        print(f"🗑️ Removed existing database: {DUCKDB_PATH.name}")
    except PermissionError:
        print(f"⚠️ Could not remove existing database (in use). Will overwrite tables.")

# Connect to persistent database
con = duckdb.connect(str(DUCKDB_PATH))
print(f"✅ Connected to DuckDB: {DUCKDB_PATH.name}")

# Register the year parser as Python UDFs with special null handling
# null_handling='special' allows the function to receive and return NULL values
con.create_function(
    'parse_year_start', 
    lambda x: parse_german_year(x)[0], 
    ['VARCHAR'], 
    'INTEGER',
    null_handling='special'
)
con.create_function(
    'parse_year_end', 
    lambda x: parse_german_year(x)[1], 
    ['VARCHAR'], 
    'INTEGER',
    null_handling='special'
)
con.create_function(
    'parse_year_approx', 
    lambda x: parse_german_year(x)[2], 
    ['VARCHAR'], 
    'BOOLEAN',
    null_handling='special'
)
con.create_function(
    'parse_year_success', 
    lambda x: parse_german_year(x)[3], 
    ['VARCHAR'], 
    'BOOLEAN',
    null_handling='special'
)

print("✅ Registered year parsing UDFs (with null handling)")

🔌 Closed existing connection
🗑️ Removed existing database: baroque.duckdb
✅ Connected to DuckDB: baroque.duckdb
✅ Registered year parsing UDFs (with null handling)


In [6]:
# =============================================================================
# CELL 4: Load Parquet Tables into DuckDB with Year Parsing
# =============================================================================

def load_parquet_table(con: duckdb.DuckDBPyConnection, table_name: str, prefix: str = PREFIX) -> int:
    """Load a Parquet file into DuckDB as a table."""
    parquet_path = PARQUET_DIR / f"{prefix}{table_name}.parquet"
    
    if not parquet_path.exists():
        print(f"   ⚠️ {table_name}: File not found at {parquet_path}")
        return 0
    
    # Determine the target table name in DuckDB
    # For bildindex tables, prefix with "bi_" to avoid name collisions
    if prefix == BILDINDEX_PREFIX:
        db_table_name = f"bi_{table_name}"
    else:
        db_table_name = table_name
    
    # For paintings table, add parsed year columns
    if table_name == 'paintings' and prefix == PREFIX:
        con.execute(f"""
            CREATE TABLE {db_table_name} AS
            SELECT 
                *,
                parse_year_start(year) AS year_start,
                parse_year_end(year) AS year_end,
                parse_year_approx(year) AS year_is_approximate,
                parse_year_success(year) AS year_parsed
            FROM read_parquet('{parquet_path}')
        """)
    else:
        con.execute(f"""
            CREATE TABLE {db_table_name} AS
            SELECT * FROM read_parquet('{parquet_path}')
        """)
    
    row_count = con.execute(f"SELECT COUNT(*) FROM {db_table_name}").fetchone()[0]
    return row_count


print("📥 Loading CbDD Parquet tables into DuckDB...")
print("=" * 60)

table_stats = {}
for table in CBDD_TABLES:
    row_count = load_parquet_table(con, table, PREFIX)
    table_stats[table] = row_count
    status = "✅" if row_count > 0 else "⚠️"
    print(f"   {status} {table:<25}: {row_count:>8,} rows")

print("=" * 60)
total_rows = sum(table_stats.values())
print(f"📊 Total: {total_rows:,} rows across {len([t for t in table_stats if table_stats[t] > 0])} tables")

# =============================================================================
# Load Bildindex Tables
# =============================================================================
print("\n📥 Loading Bildindex Parquet tables into DuckDB...")
print("=" * 60)

bildindex_stats = {}
for table in BILDINDEX_TABLES:
    row_count = load_parquet_table(con, table, BILDINDEX_PREFIX)
    bildindex_stats[table] = row_count
    db_table_name = f"bi_{table}"
    status = "✅" if row_count > 0 else "⚠️"
    print(f"   {status} {db_table_name:<25}: {row_count:>8,} rows")

print("=" * 60)
bi_total = sum(bildindex_stats.values())
print(f"📊 Bildindex Total: {bi_total:,} rows across {len([t for t in bildindex_stats if bildindex_stats[t] > 0])} tables")

# Combine stats
all_table_stats = {**table_stats, **{f"bi_{k}": v for k, v in bildindex_stats.items()}}

📥 Loading CbDD Parquet tables into DuckDB...
   ✅ paintings                :    4,594 rows
   ✅ persons                  :    2,831 rows
   ✅ buildings                :    1,260 rows
   ✅ rooms                    :    2,376 rows
   ✅ ensembles                :       32 rows
   ✅ subjects                 :    4,082 rows
   ✅ painting_persons         :    4,696 rows
   ✅ painting_subjects        :   17,474 rows
   ✅ building_persons         :    1,983 rows
   ✅ room_persons             :    4,676 rows
📊 Total: 44,004 rows across 10 tables

📥 Loading Bildindex Parquet tables into DuckDB...
   ✅ bi_items                 :      948 rows
   ✅ bi_buildings             :      211 rows
   ✅ bi_painters              :      737 rows
   ✅ bi_subjects              :      946 rows
   ✅ bi_gnd_overlaps          :      433 rows
📊 Bildindex Total: 3,275 rows across 5 tables


In [7]:
# =============================================================================
# CELL 5: Create Indexes for Performance
# =============================================================================

print("🔧 Creating indexes for CbDD tables...")

indexes = [
    # Paintings table
    ("idx_paintings_nfdi", "paintings", "nfdi_uri"),
    ("idx_paintings_cbdd", "paintings", "cbdd_painting_id"),
    ("idx_paintings_building", "paintings", "building_id"),
    ("idx_paintings_room", "paintings", "room_id"),
    ("idx_paintings_year_start", "paintings", "year_start"),
    ("idx_paintings_state", "paintings", "location_state"),
    
    # Junction tables
    ("idx_pp_nfdi", "painting_persons", "nfdi_uri"),
    ("idx_pp_person", "painting_persons", "person_id"),
    ("idx_ps_nfdi", "painting_subjects", "nfdi_uri"),
    ("idx_ps_subject", "painting_subjects", "subject_uri"),
    ("idx_bp_building", "building_persons", "building_id"),
    ("idx_bp_person", "building_persons", "person_id"),
    ("idx_rp_room", "room_persons", "room_id"),
    ("idx_rp_person", "room_persons", "person_id"),
    
    # Entity tables
    ("idx_persons_id", "persons", "person_id"),
    ("idx_buildings_id", "buildings", "building_id"),
    ("idx_rooms_id", "rooms", "room_id"),
    ("idx_subjects_uri", "subjects", "subject_uri"),
]

for idx_name, table, column in indexes:
    try:
        con.execute(f"CREATE INDEX {idx_name} ON {table}({column})")
        print(f"   ✅ {idx_name}")
    except Exception as e:
        if "already exists" in str(e).lower():
            print(f"   ⏭️ {idx_name} (exists)")
        elif "does not exist" in str(e).lower() or "not found" in str(e).lower():
            print(f"   ⚠️ {idx_name} - table/column not found")
        else:
            print(f"   ❌ {idx_name}: {e}")

# =============================================================================
# Bildindex Indexes
# =============================================================================
print("\n🔧 Creating indexes for Bildindex tables...")

bi_indexes = [
    # bi_items - main Bildindex items table
    ("idx_bi_items_uri", "bi_items", "bildindex_uri"),
    ("idx_bi_items_url", "bi_items", "bildindex_url"),
    ("idx_bi_items_tier", "bi_items", "tier"),
    
    # bi_buildings - Bildindex items linked to buildings via GND
    ("idx_bi_buildings_uri", "bi_buildings", "bildindex_uri"),
    ("idx_bi_buildings_gnd", "bi_buildings", "building_gnd"),
    
    # bi_painters - Bildindex items linked to painters via GND
    ("idx_bi_painters_uri", "bi_painters", "bildindex_uri"),
    ("idx_bi_painters_gnd", "bi_painters", "painter_gnd"),
    
    # bi_subjects - Bildindex items linked to ICONCLASS subjects
    ("idx_bi_subjects_uri", "bi_subjects", "bildindex_uri"),
    ("idx_bi_subjects_code", "bi_subjects", "iconclass_code"),
    
    # bi_gnd_overlaps - GND overlap summary
    ("idx_bi_gnd_overlaps_uri", "bi_gnd_overlaps", "gnd_uri"),
    ("idx_bi_gnd_overlaps_type", "bi_gnd_overlaps", "gnd_type"),
]

for idx_name, table, column in bi_indexes:
    try:
        con.execute(f"CREATE INDEX {idx_name} ON {table}({column})")
        print(f"   ✅ {idx_name}")
    except Exception as e:
        if "already exists" in str(e).lower():
            print(f"   ⏭️ {idx_name} (exists)")
        elif "does not exist" in str(e).lower() or "not found" in str(e).lower():
            print(f"   ⚠️ {idx_name} - table/column not found")
        else:
            print(f"   ❌ {idx_name}: {e}")

print("\n✅ Indexing complete")

🔧 Creating indexes for CbDD tables...
   ✅ idx_paintings_nfdi
   ✅ idx_paintings_cbdd
   ✅ idx_paintings_building
   ✅ idx_paintings_room
   ✅ idx_paintings_year_start
   ✅ idx_paintings_state
   ✅ idx_pp_nfdi
   ✅ idx_pp_person
   ✅ idx_ps_nfdi
   ✅ idx_ps_subject
   ✅ idx_bp_building
   ✅ idx_bp_person
   ✅ idx_rp_room
   ✅ idx_rp_person
   ✅ idx_persons_id
   ✅ idx_buildings_id
   ✅ idx_rooms_id
   ✅ idx_subjects_uri

🔧 Creating indexes for Bildindex tables...
   ✅ idx_bi_items_uri
   ✅ idx_bi_items_url
   ✅ idx_bi_items_tier
   ✅ idx_bi_buildings_uri
   ✅ idx_bi_buildings_gnd
   ✅ idx_bi_painters_uri
   ✅ idx_bi_painters_gnd
   ✅ idx_bi_subjects_uri
   ✅ idx_bi_subjects_code
   ✅ idx_bi_gnd_overlaps_uri
   ✅ idx_bi_gnd_overlaps_type

✅ Indexing complete


In [8]:
# =============================================================================
# CELL 6: Year Parsing Validation
# =============================================================================

print("📅 Year Parsing Statistics:")
print("=" * 60)

year_stats = con.execute("""
    SELECT
        COUNT(*) as total,
        SUM(CASE WHEN year IS NOT NULL AND year != '' THEN 1 ELSE 0 END) as has_year_string,
        SUM(CASE WHEN year_parsed THEN 1 ELSE 0 END) as parsed_successfully,
        SUM(CASE WHEN year IS NOT NULL AND year != '' AND NOT year_parsed THEN 1 ELSE 0 END) as parse_failed,
        SUM(CASE WHEN year_is_approximate THEN 1 ELSE 0 END) as approximate_dates,
        MIN(year_start) as earliest_year,
        MAX(year_end) as latest_year
    FROM paintings
""").fetchone()

total, has_year, parsed, failed, approx, earliest, latest = year_stats

print(f"   Total paintings:        {total:>6}")
print(f"   Has year string:        {has_year:>6} ({100*has_year/total:.1f}%)")
print(f"   ✅ Parsed successfully: {parsed:>6} ({100*parsed/total:.1f}%)")
print(f"   ❌ Parse failed:        {failed:>6} ({100*failed/total:.1f}%)")
print(f"   ~ Approximate dates:    {approx:>6}")
print(f"   📆 Date range:          {earliest} - {latest}")

# Show unparseable years for review
if failed > 0:
    print("\n" + "=" * 60)
    print("⚠️ UNPARSEABLE YEAR STRINGS (for manual review):")
    print("=" * 60)
    
    unparsed = con.execute("""
        SELECT year, COUNT(*) as count
        FROM paintings
        WHERE year IS NOT NULL AND year != '' AND NOT year_parsed
        GROUP BY year
        ORDER BY count DESC
        LIMIT 20
    """).pl()
    
    for row in unparsed.iter_rows(named=True):
        print(f"   [{row['count']:>2}x] {row['year']}")

📅 Year Parsing Statistics:
   Total paintings:          4594
   Has year string:          4150 (90.3%)
   ✅ Parsed successfully:   4149 (90.3%)
   ❌ Parse failed:             1 (0.0%)
   ~ Approximate dates:       724
   📆 Date range:          1000 - 2020

⚠️ UNPARSEABLE YEAR STRINGS (for manual review):
   [ 1x] spätgotisch


In [9]:
# =============================================================================
# CELL 7: Data Quality Summary
# =============================================================================

print("📊 DATA QUALITY SUMMARY")
print("=" * 70)

# Row counts per table
print("\n📋 Table Row Counts:")
for table in TABLES:
    try:
        count = con.execute(f"SELECT COUNT(*) FROM {table}").fetchone()[0]
        print(f"   {table:<25}: {count:>8,}")
    except:
        print(f"   {table:<25}: (not loaded)")

# Paintings table coverage
print("\n📋 Paintings Column Coverage:")
coverage = con.execute("""
    SELECT
        COUNT(*) as total,
        SUM(CASE WHEN label IS NOT NULL THEN 1 ELSE 0 END) as has_label,
        SUM(CASE WHEN year_parsed THEN 1 ELSE 0 END) as has_year,
        SUM(CASE WHEN lat IS NOT NULL THEN 1 ELSE 0 END) as has_coords,
        SUM(CASE WHEN imageUrl IS NOT NULL THEN 1 ELSE 0 END) as has_image,
        SUM(CASE WHEN cbdd_painting_id IS NOT NULL THEN 1 ELSE 0 END) as has_cbdd,
        SUM(CASE WHEN building_name IS NOT NULL THEN 1 ELSE 0 END) as has_building,
        SUM(CASE WHEN painters IS NOT NULL AND painters != '' THEN 1 ELSE 0 END) as has_painter,
        SUM(CASE WHEN location_state IS NOT NULL THEN 1 ELSE 0 END) as has_state
    FROM paintings
""").fetchone()

total = coverage[0]
cols = ['label', 'year (parsed)', 'coordinates', 'imageUrl', 'CbDD match', 
        'building_name', 'painters', 'location_state']
for i, col in enumerate(cols):
    val = coverage[i + 1]
    pct = 100 * val / total if total > 0 else 0
    bar = "█" * int(pct / 5) + "░" * (20 - int(pct / 5))
    print(f"   {col:<20}: {bar} {val:>5}/{total} ({pct:>5.1f}%)")

# Foreign key integrity
print("\n🔗 Foreign Key Integrity:")

fk_checks = [
    ("painting_persons → paintings", 
     "SELECT COUNT(*) FROM painting_persons pp LEFT JOIN paintings p ON pp.nfdi_uri = p.nfdi_uri WHERE p.nfdi_uri IS NULL"),
    ("painting_subjects → paintings",
     "SELECT COUNT(*) FROM painting_subjects ps LEFT JOIN paintings p ON ps.nfdi_uri = p.nfdi_uri WHERE p.nfdi_uri IS NULL"),
    ("paintings → buildings",
     "SELECT COUNT(*) FROM paintings p LEFT JOIN buildings b ON p.building_id = b.building_id WHERE p.building_id IS NOT NULL AND b.building_id IS NULL"),
    ("paintings → rooms",
     "SELECT COUNT(*) FROM paintings p LEFT JOIN rooms r ON p.room_id = r.room_id WHERE p.room_id IS NOT NULL AND r.room_id IS NULL"),
]

for desc, query in fk_checks:
    try:
        orphans = con.execute(query).fetchone()[0]
        status = "✅" if orphans == 0 else "⚠️"
        print(f"   {status} {desc}: {orphans} orphan(s)")
    except Exception as e:
        print(f"   ⚠️ {desc}: check failed")

📊 DATA QUALITY SUMMARY

📋 Table Row Counts:
   paintings                :    4,594
   persons                  :    2,831
   buildings                :    1,260
   rooms                    :    2,376
   ensembles                :       32
   subjects                 :    4,082
   painting_persons         :    4,696
   painting_subjects        :   17,474
   building_persons         :    1,983
   room_persons             :    4,676

📋 Paintings Column Coverage:
   label               : ████████████████████  4594/4594 (100.0%)
   year (parsed)       : ██████████████████░░  4149/4594 ( 90.3%)
   coordinates         : █████████████████░░░  3927/4594 ( 85.5%)
   imageUrl            : ████████████████████  4594/4594 (100.0%)
   CbDD match          : ████████████░░░░░░░░  2845/4594 ( 61.9%)
   building_name       : ████████████░░░░░░░░  2845/4594 ( 61.9%)
   painters            : ██████░░░░░░░░░░░░░░  1575/4594 ( 34.3%)
   location_state      : ████████████░░░░░░░░  2845/4594 ( 61.9%)

🔗 Forei

In [10]:
# =============================================================================
# CELL 8: Save and Verify Database
# =============================================================================

# Checkpoint to ensure data is written
con.execute("CHECKPOINT")

# Verify database size
db_size = DUCKDB_PATH.stat().st_size / (1024 * 1024)

print("💾 DATABASE SAVED")
print("=" * 60)
print(f"   📁 Path: {DUCKDB_PATH}")
print(f"   📊 Size: {db_size:.2f} MB")

# Count CbDD tables
cbdd_count = len([t for t in CBDD_TABLES if table_stats.get(t, 0) > 0])
cbdd_rows = sum(table_stats.values())
print(f"\n   📘 CbDD Tables: {cbdd_count}")
print(f"      Total rows: {cbdd_rows:,}")

# Count Bildindex tables
bi_count = len([t for t in BILDINDEX_TABLES if bildindex_stats.get(t, 0) > 0])
bi_rows = sum(bildindex_stats.values())
print(f"\n   📗 Bildindex Tables: {bi_count}")
print(f"      Total rows: {bi_rows:,}")

# List all tables in the database
print("\n📋 All Tables in Database:")
tables_in_db = con.execute("SHOW TABLES").pl()
cbdd_list = []
bi_list = []
for row in tables_in_db.iter_rows(named=True):
    if row['name'].startswith('bi_'):
        bi_list.append(row['name'])
    else:
        cbdd_list.append(row['name'])

print("   CbDD: " + ", ".join(cbdd_list))
print("   Bildindex: " + ", ".join(bi_list))

print("\n✅ Database ready for collaborative use!")
print(f"   Share the file: {DUCKDB_PATH.name}")

💾 DATABASE SAVED
   📁 Path: c:\Users\thano\Documents\_Studium\KIT\DataStories\DataStories\baroque.duckdb
   📊 Size: 16.26 MB

   📘 CbDD Tables: 10
      Total rows: 44,004

   📗 Bildindex Tables: 5
      Total rows: 3,275

📋 All Tables in Database:
   CbDD: building_persons, buildings, ensembles, painting_persons, painting_subjects, paintings, persons, room_persons, rooms, subjects
   Bildindex: bi_buildings, bi_gnd_overlaps, bi_items, bi_painters, bi_subjects

✅ Database ready for collaborative use!
   Share the file: baroque.duckdb


---
# Analysis Queries

Now that the database is loaded, we can run analytical queries using DuckDB SQL with Polars DataFrames as output.

## Data Sources
- **CbDD Tables**: `paintings`, `persons`, `buildings`, `rooms`, `ensembles`, `subjects`, and junction tables
- **Bildindex Tables**: `bi_items`, `bi_buildings`, `bi_painters`, `bi_subjects`, `bi_gnd_overlaps`

## Cross-Reference Capability
Join CbDD and Bildindex data via shared GNDs (German National Library identifiers):
- `paintings.creatorGnds` ↔ `bi_painters.painter_gnd`
- `paintings.locationGnds` ↔ `bi_buildings.building_gnd`

In [11]:
# =============================================================================
# CELL 9: Geographic Analysis - Paintings by German State
# =============================================================================

print("🗺️ GEOGRAPHIC DISTRIBUTION")
print("=" * 60)

geo_df = con.execute("""
    SELECT 
        location_state,
        COUNT(*) as painting_count,
        COUNT(DISTINCT building_id) as building_count,
        MIN(year_start) as earliest,
        MAX(year_end) as latest,
        ROUND(AVG(year_start), 0) as avg_year
    FROM paintings
    WHERE location_state IS NOT NULL
    GROUP BY location_state
    ORDER BY painting_count DESC
""").pl()

print(geo_df)

# Visualization
if len(geo_df) > 0:
    chart = alt.Chart(geo_df.to_pandas()).mark_bar().encode(
        x=alt.X('painting_count:Q', title='Number of Paintings'),
        y=alt.Y('location_state:N', sort='-x', title='German State'),
        color=alt.Color('painting_count:Q', scale=alt.Scale(scheme='blues'), legend=None),
        tooltip=['location_state', 'painting_count', 'building_count', 'earliest', 'latest']
    ).properties(
        title='Baroque Ceiling Paintings by German State',
        width=500,
        height=400
    )
    ipython_display(chart)

🗺️ GEOGRAPHIC DISTRIBUTION
shape: (15, 6)
┌────────────────────────┬────────────────┬────────────────┬──────────┬────────┬──────────┐
│ location_state         ┆ painting_count ┆ building_count ┆ earliest ┆ latest ┆ avg_year │
│ ---                    ┆ ---            ┆ ---            ┆ ---      ┆ ---    ┆ ---      │
│ str                    ┆ i64            ┆ i64            ┆ i32      ┆ i32    ┆ f64      │
╞════════════════════════╪════════════════╪════════════════╪══════════╪════════╪══════════╡
│ Bayern                 ┆ 1150           ┆ 101            ┆ 1500     ┆ 2004   ┆ 1656.0   │
│ Thüringen              ┆ 424            ┆ 52             ┆ 1459     ┆ 1921   ┆ 1701.0   │
│ Baden-Württemberg      ┆ 362            ┆ 80             ┆ 1534     ┆ 1972   ┆ 1735.0   │
│ Niedersachsen          ┆ 172            ┆ 35             ┆ 1506     ┆ 1973   ┆ 1649.0   │
│ Nordrhein-Westfalen    ┆ 127            ┆ 31             ┆ 1590     ┆ 1866   ┆ 1708.0   │
│ …                      ┆ …          

alt.Chart(...)

In [12]:
# =============================================================================
# CELL 10: Temporal Analysis - Paintings by Decade
# =============================================================================
# Filter by Getty AAT "ceiling paintings" to exclude buildings

print("📅 TEMPORAL DISTRIBUTION (Ceiling Paintings Only)")
print("=" * 60)

temporal_df = con.execute("""
    SELECT 
        CAST(FLOOR(p.year_start / 10) * 10 AS INTEGER) as decade,
        COUNT(*) as count,
        CAST(SUM(CASE WHEN p.year_is_approximate THEN 1 ELSE 0 END) AS INTEGER) as approximate_count
    FROM paintings p
    JOIN painting_subjects ps ON p.nfdi_uri = ps.nfdi_uri
    JOIN subjects s ON ps.subject_uri = s.subject_uri
    WHERE p.year_start IS NOT NULL
      AND p.year_start >= 1500 AND p.year_start <= 1900
      AND s.subject_source = 'GETTY_AAT'
      AND s.subject_label = 'ceiling paintings'
    GROUP BY FLOOR(p.year_start / 10) * 10
    ORDER BY decade
""").pl()

print(temporal_df)

# Visualization
if len(temporal_df) > 0:
    chart = alt.Chart(temporal_df.to_pandas()).mark_bar().encode(
        x=alt.X('decade:O', title='Decade'),
        y=alt.Y('count:Q', title='Number of Paintings'),
        color=alt.Color('approximate_count:Q', 
                       scale=alt.Scale(scheme='oranges'),
                       title='Approximate Dates'),
        tooltip=['decade', 'count', 'approximate_count']
    ).properties(
        title='Baroque Ceiling Paintings by Decade',
        width=600,
        height=300
    )
    ipython_display(chart)

📅 TEMPORAL DISTRIBUTION (Ceiling Paintings Only)
shape: (34, 3)
┌────────┬───────┬───────────────────┐
│ decade ┆ count ┆ approximate_count │
│ ---    ┆ ---   ┆ ---               │
│ i32    ┆ i64   ┆ i32               │
╞════════╪═══════╪═══════════════════╡
│ 1500   ┆ 16    ┆ 4                 │
│ 1520   ┆ 2     ┆ 0                 │
│ 1530   ┆ 10    ┆ 2                 │
│ 1540   ┆ 375   ┆ 86                │
│ 1550   ┆ 5     ┆ 2                 │
│ …      ┆ …     ┆ …                 │
│ 1820   ┆ 1     ┆ 0                 │
│ 1830   ┆ 1     ┆ 1                 │
│ 1840   ┆ 2     ┆ 0                 │
│ 1860   ┆ 2     ┆ 0                 │
│ 1900   ┆ 1     ┆ 0                 │
└────────┴───────┴───────────────────┘


alt.Chart(...)

In [13]:
# =============================================================================
# CELL 10b: Investigate 1540s Paintings
# =============================================================================
# Manual inspection of paintings dated to 1540s - seems unusually early for Baroque

print("🔍 INVESTIGATING 1540s PAINTINGS")
print("=" * 80)

paintings_1540s = con.execute("""
    SELECT 
        p.label,
        p.year,
        p.year_start,
        p.year_end,
        p.year_is_approximate,
        p.building_name,
        p.location_state,
        p.nfdi_uri
    FROM paintings p
    JOIN painting_subjects ps ON p.nfdi_uri = ps.nfdi_uri
    JOIN subjects s ON ps.subject_uri = s.subject_uri
    WHERE p.year_start >= 1540 AND p.year_start < 1550
      AND s.subject_source = 'GETTY_AAT'
      AND s.subject_label = 'ceiling paintings'
    ORDER BY p.year_start, p.label
""").pl()

print(f"Found {len(paintings_1540s)} paintings from the 1540s:\n")

for i, row in enumerate(paintings_1540s.iter_rows(named=True), 1):
    print(f"{i}. {row['label']}")
    print(f"   📅 Year string: '{row['year']}' → parsed: {row['year_start']}-{row['year_end']} (approx: {row['year_is_approximate']})")
    print(f"   🏛️ {row['building_name']}, {row['location_state']}")
    print(f"   🔗 {row['nfdi_uri']}")
    print()

🔍 INVESTIGATING 1540s PAINTINGS
Found 375 paintings from the 1540s:

1. Christi Himmelfahrt
   📅 Year string: '1540-1542, 1780' → parsed: 1540-1542 (approx: False)
   🏛️ None, None
   🔗 https://www.deckenmalerei.eu/f59c1628-8008-4e88-a63e-d31509950715

2. Diana und die Jagdhunde
   📅 Year string: '1540-1542' → parsed: 1540-1542 (approx: False)
   🏛️ Landshut, Stadtresidenz, Bayern
   🔗 https://www.deckenmalerei.eu/fe1d8777-2f6c-458f-8faf-d14f979c0cc9

3. Diana wird von Aktäon beim Bad überrascht
   📅 Year string: '1540-1542' → parsed: 1540-1542 (approx: False)
   🏛️ Landshut, Stadtresidenz, Bayern
   🔗 https://www.deckenmalerei.eu/5e2b1711-3732-4cbb-a2e0-58c9c7c50518

4. Die Auferstehung Christi (Supraporte)
   📅 Year string: '1540-1542' → parsed: 1540-1542 (approx: False)
   🏛️ Landshut, Stadtresidenz, Bayern
   🔗 https://www.deckenmalerei.eu/7bae7ccd-4086-4257-aeb4-d1abdffabfd0

5. Die Decken- und Wandmalerei im Gerichtszimmer
   📅 Year string: '1540, 1627, 1921' → parsed: 1540-1921 

In [14]:
# =============================================================================
# CELL 11: Top Painters Analysis
# =============================================================================
# Using the painting_persons junction table for accurate individual painter counts.
# The paintings.painters column contains pipe-separated values for collaborative
# works, which would incorrectly group "A | B" as a single painter.

print("🎨 TOP PAINTERS")
print("=" * 60)

painters_df = con.execute("""
    SELECT 
        pp.person_name,
        COUNT(DISTINCT pp.nfdi_uri) as painting_count,
        COUNT(DISTINCT p.building_id) as building_count,
        MIN(p.year_start) as active_from,
        MAX(p.year_end) as active_until
    FROM painting_persons pp
    JOIN paintings p ON pp.nfdi_uri = p.nfdi_uri
    WHERE pp.role = 'PAINTER'
    GROUP BY pp.person_name
    HAVING COUNT(DISTINCT pp.nfdi_uri) >= 2
    ORDER BY painting_count DESC
    LIMIT 25
""").pl()

print(painters_df)

# Visualization
if len(painters_df) > 0:
    chart = alt.Chart(painters_df.head(15).to_pandas()).mark_bar().encode(
        x=alt.X('painting_count:Q', title='Number of Paintings'),
        y=alt.Y('person_name:N', sort='-x', title='Painter'),
        color=alt.Color('building_count:Q', 
                       scale=alt.Scale(scheme='viridis'),
                       title='Buildings'),
        tooltip=['person_name', 'painting_count', 'building_count', 'active_from', 'active_until']
    ).properties(
        title='Most Prolific Baroque Ceiling Painters',
        width=500,
        height=400
    )
    ipython_display(chart)

🎨 TOP PAINTERS
shape: (25, 5)
┌──────────────────────────────┬────────────────┬────────────────┬─────────────┬──────────────┐
│ person_name                  ┆ painting_count ┆ building_count ┆ active_from ┆ active_until │
│ ---                          ┆ ---            ┆ ---            ┆ ---         ┆ ---          │
│ str                          ┆ i64            ┆ i64            ┆ i32         ┆ i32          │
╞══════════════════════════════╪════════════════╪════════════════╪═════════════╪══════════════╡
│ Harms, Johann Oswald         ┆ 99             ┆ 7              ┆ 1682        ┆ 1707         │
│ Lammers, Seivert             ┆ 80             ┆ 4              ┆ 1670        ┆ 1703         │
│ Asam, Cosmas Damian          ┆ 58             ┆ 7              ┆ 1700        ┆ 1948         │
│ Castelli, Carlo Ludovico     ┆ 56             ┆ 8              ┆ 1707        ┆ 1770         │
│ Hermann, Franz Georg         ┆ 54             ┆ 2              ┆ 1729        ┆ 1742         │
│ …       

alt.Chart(...)

In [15]:
# =============================================================================
# CELL 12: Building Function Analysis
# =============================================================================
# Join with buildings table to get the function column

print("🏛️ BUILDING FUNCTION ANALYSIS")
print("=" * 60)

building_func_df = con.execute("""
    SELECT 
        b.function as building_function,
        COUNT(*) as painting_count,
        COUNT(DISTINCT p.building_id) as building_count,
        ROUND(AVG(p.year_start), 0) as avg_year
    FROM paintings p
    JOIN buildings b ON p.building_id = b.building_id
    WHERE b.function IS NOT NULL
    GROUP BY b.function
    ORDER BY painting_count DESC
    LIMIT 20
""").pl()

print(building_func_df)

# Visualization - Pie/Donut chart
if len(building_func_df) > 0:
    chart = alt.Chart(building_func_df.head(10).to_pandas()).mark_arc(innerRadius=50).encode(
        theta=alt.Theta('painting_count:Q'),
        color=alt.Color('building_function:N', title='Building Type'),
        tooltip=['building_function', 'painting_count', 'building_count']
    ).properties(
        title='Paintings by Building Function',
        width=400,
        height=400
    )
    ipython_display(chart)

🏛️ BUILDING FUNCTION ANALYSIS
shape: (20, 4)
┌─────────────────────────────────┬────────────────┬────────────────┬──────────┐
│ building_function               ┆ painting_count ┆ building_count ┆ avg_year │
│ ---                             ┆ ---            ┆ ---            ┆ ---      │
│ str                             ┆ i64            ┆ i64            ┆ f64      │
╞═════════════════════════════════╪════════════════╪════════════════╪══════════╡
│ Hauptresidenz                   ┆ 569            ┆ 40             ┆ 1679.0   │
│ Residenz                        ┆ 555            ┆ 21             ┆ 1607.0   │
│ Schloss                         ┆ 320            ┆ 38             ┆ 1717.0   │
│ Herrenhaus / Rittersitz / Guts… ┆ 302            ┆ 67             ┆ 1716.0   │
│ Jagdschloss                     ┆ 128            ┆ 15             ┆ 1680.0   │
│ …                               ┆ …              ┆ …              ┆ …        │
│ Wohn- und Arbeits-/Geschäftsha… ┆ 30             ┆ 15         

alt.Chart(...)

In [16]:
# =============================================================================
# CELL 13: Subject/Theme Analysis (ICONCLASS only)
# =============================================================================
# Filter: Only ceiling paintings (via Getty AAT), display only ICONCLASS subjects

print("📚 SUBJECT/THEME ANALYSIS (ICONCLASS)")
print("=" * 60)

subjects_df = con.execute("""
    SELECT 
        s.subject_label,
        COUNT(DISTINCT ps.nfdi_uri) as painting_count
    FROM painting_subjects ps
    JOIN subjects s ON ps.subject_uri = s.subject_uri
    -- Only include paintings that are classified as "ceiling paintings" by Getty AAT
    WHERE ps.nfdi_uri IN (
        SELECT ps2.nfdi_uri 
        FROM painting_subjects ps2
        JOIN subjects s2 ON ps2.subject_uri = s2.subject_uri
        WHERE s2.subject_source = 'GETTY_AAT' 
          AND s2.subject_label = 'ceiling paintings'
    )
    -- Only show ICONCLASS subjects (not Getty AAT)
    AND s.subject_source = 'ICONCLASS'
    AND s.subject_label IS NOT NULL
    GROUP BY s.subject_label
    ORDER BY painting_count DESC
    LIMIT 30
""").pl()

print(subjects_df)

# Visualization
if len(subjects_df) > 0:
    chart = alt.Chart(subjects_df.head(20).to_pandas()).mark_bar().encode(
        x=alt.X('painting_count:Q', title='Number of Paintings'),
        y=alt.Y('subject_label:N', sort='-x', title='ICONCLASS Subject'),
        color=alt.Color('painting_count:Q', scale=alt.Scale(scheme='tealblues'), legend=None),
        tooltip=['subject_label', 'painting_count']
    ).properties(
        title='Most Common ICONCLASS Subjects in Baroque Ceiling Paintings',
        width=800,
        height=400
    )
    ipython_display(chart)

📚 SUBJECT/THEME ANALYSIS (ICONCLASS)
shape: (30, 2)
┌─────────────────────────────────┬────────────────┐
│ subject_label                   ┆ painting_count │
│ ---                             ┆ ---            │
│ str                             ┆ i64            │
╞═════════════════════════════════╪════════════════╡
│ cupids: 'amores', 'amoretti', … ┆ 331            │
│ clouds                          ┆ 180            │
│ garland, wreath                 ┆ 80             │
│ trees                           ┆ 71             │
│ column, pillar ~ architecture   ┆ 62             │
│ …                               ┆ …              │
│ helmet                          ┆ 36             │
│ spring, 'Ver'; 'Primavera' (Ri… ┆ 36             │
│ dog                             ┆ 36             │
│ winter, 'Hyems'; 'Inverno' (Ri… ┆ 35             │
│ (story of) Minerva (Pallas, At… ┆ 35             │
└─────────────────────────────────┴────────────────┘


alt.Chart(...)

In [17]:
# =============================================================================
# CELL 13b: ICONCLASS Category Analysis
# =============================================================================
# Categorize ICONCLASS subjects by their top-level category (first digit)
# ICONCLASS hierarchy: https://iconclass.org/

# Category mapping (first digit of ICONCLASS notation)
ICONCLASS_CATEGORIES = {
    '0': 'Abstract, Non-representational Art',
    '1': 'Religion and Magic',
    '2': 'Nature',
    '3': 'Human Being, Man in General',
    '4': 'Society, Civilization, Culture',
    '5': 'Abstract Ideas and Concepts',
    '6': 'History',
    '7': 'Bible',
    '8': 'Literature',
    '9': 'Classical Mythology and Ancient History'
}

print("📊 ICONCLASS CATEGORY ANALYSIS")
print("=" * 70)

# Query to extract ICONCLASS code and categorize
category_df = con.execute("""
    WITH iconclass_parsed AS (
        SELECT 
            s.subject_label,
            s.subject_uri,
            -- Extract ICONCLASS code from URI (after iconclass.org/)
            REGEXP_EXTRACT(s.subject_uri, 'iconclass\\.org/([^/]+)', 1) as iconclass_code,
            COUNT(DISTINCT ps.nfdi_uri) as painting_count
        FROM painting_subjects ps
        JOIN subjects s ON ps.subject_uri = s.subject_uri
        -- Only ceiling paintings
        WHERE ps.nfdi_uri IN (
            SELECT ps2.nfdi_uri 
            FROM painting_subjects ps2
            JOIN subjects s2 ON ps2.subject_uri = s2.subject_uri
            WHERE s2.subject_source = 'GETTY_AAT' 
              AND s2.subject_label = 'ceiling paintings'
        )
        AND s.subject_source = 'ICONCLASS'
        AND s.subject_label IS NOT NULL
        GROUP BY s.subject_label, s.subject_uri
    )
    SELECT 
        subject_label,
        iconclass_code,
        -- Extract first character for category
        SUBSTRING(iconclass_code, 1, 1) as category_digit,
        painting_count
    FROM iconclass_parsed
    WHERE iconclass_code IS NOT NULL
    ORDER BY painting_count DESC
""").pl()

# Add category names using Python
category_df = category_df.with_columns(
    pl.col('category_digit').replace(ICONCLASS_CATEGORIES).alias('category_name')
)

print(f"Total ICONCLASS subjects: {len(category_df)}")
print(f"\n📋 Category Distribution:")

# Summary by category
category_summary = category_df.group_by('category_digit', 'category_name').agg([
    pl.count().alias('subject_count'),
    pl.sum('painting_count').alias('total_paintings')
]).sort('total_paintings', descending=True)

print(category_summary)

# Prepare data for visualization
viz_df = category_df.head(50).to_pandas()  # Top 50 subjects

# Create a treemap-style visualization using layered bar chart
# Color by category, sorted by painting count within category

chart_bars = alt.Chart(viz_df).mark_bar().encode(
    x=alt.X('painting_count:Q', title='Number of Paintings'),
    y=alt.Y('subject_label:N', sort='-x', title='ICONCLASS Subject'),
    color=alt.Color('category_name:N', 
                   title='ICONCLASS Category',
                   scale=alt.Scale(scheme='category10')),
    tooltip=[
        alt.Tooltip('subject_label:N', title='Subject'),
        alt.Tooltip('iconclass_code:N', title='ICONCLASS Code'),
        alt.Tooltip('category_name:N', title='Category'),
        alt.Tooltip('painting_count:Q', title='Paintings')
    ]
).properties(
    title='Top 50 ICONCLASS Subjects by Category',
    width=700,
    height=800
)

ipython_display(chart_bars)

📊 ICONCLASS CATEGORY ANALYSIS
Total ICONCLASS subjects: 4080

📋 Category Distribution:
shape: (9, 4)
┌────────────────┬─────────────────────────────────┬───────────────┬─────────────────┐
│ category_digit ┆ category_name                   ┆ subject_count ┆ total_paintings │
│ ---            ┆ ---                             ┆ ---           ┆ ---             │
│ str            ┆ str                             ┆ u32           ┆ i64             │
╞════════════════╪═════════════════════════════════╪═══════════════╪═════════════════╡
│ 4              ┆ Society, Civilization, Culture  ┆ 1116          ┆ 4387            │
│ 2              ┆ Nature                          ┆ 580           ┆ 2777            │
│ 9              ┆ Classical Mythology and Ancien… ┆ 787           ┆ 2514            │
│ 1              ┆ Religion and Magic              ┆ 566           ┆ 1433            │
│ 7              ┆ Bible                           ┆ 457           ┆ 668             │
│ 3              ┆ Human Bein

C:\Users\thano\AppData\Local\Temp\ipykernel_33604\1495220835.py:68: DeprecationWarning: `pl.count()` is deprecated. Please use `pl.len()` instead.
(Deprecated in version 0.20.5)
  pl.count().alias('subject_count'),


alt.Chart(...)

In [18]:
# =============================================================================
# CELL 13c: Disease-Related ICONCLASS Analysis (31A4, 31AA4, 31E, 31F)
# =============================================================================
# ICONCLASS codes for diseases/medical topics:
#   31A4 = Specific Diseases Male
#   31AA4 = Specific Diseases Female
#   31E = Abnormalities
#   31F = Infirmities and Afflictions

print("🏥 DISEASE-RELATED ICONCLASS ANALYSIS (31A4, 31AA4, 31E, 31F)")
print("=" * 70)

disease_df = con.execute("""
    WITH disease_subjects AS (
        SELECT 
            s.subject_label,
            s.subject_uri,
            REGEXP_EXTRACT(s.subject_uri, 'iconclass\\.org/([^/]+)', 1) as iconclass_code,
            ps.nfdi_uri
        FROM painting_subjects ps
        JOIN subjects s ON ps.subject_uri = s.subject_uri
        WHERE s.subject_source = 'ICONCLASS'
          AND s.subject_label IS NOT NULL
          AND (
              REGEXP_EXTRACT(s.subject_uri, 'iconclass\\.org/([^/]+)', 1) LIKE '31A4%'
              OR REGEXP_EXTRACT(s.subject_uri, 'iconclass\\.org/([^/]+)', 1) LIKE '31AA4%'
              OR REGEXP_EXTRACT(s.subject_uri, 'iconclass\\.org/([^/]+)', 1) LIKE '31E%'
              OR REGEXP_EXTRACT(s.subject_uri, 'iconclass\\.org/([^/]+)', 1) LIKE '31F%'
          )
    )
    SELECT 
        subject_label,
        iconclass_code,
        CASE 
            WHEN iconclass_code LIKE '31A4%' THEN '31A4 - Specific Diseases Male'
            WHEN iconclass_code LIKE '31AA4%' THEN '31AA4 - Specific Diseases Female'
            WHEN iconclass_code LIKE '31E%' THEN '31E - Abnormalities'
            WHEN iconclass_code LIKE '31F%' THEN '31F - Infirmities'
            ELSE 'Other'
        END as disease_category,
        COUNT(DISTINCT nfdi_uri) as painting_count
    FROM disease_subjects
    GROUP BY subject_label, iconclass_code, disease_category
    ORDER BY painting_count DESC
""").pl()

# Summary statistics
total_disease_paintings = con.execute("""
    SELECT COUNT(DISTINCT ps.nfdi_uri) as count
    FROM painting_subjects ps
    JOIN subjects s ON ps.subject_uri = s.subject_uri
    WHERE s.subject_source = 'ICONCLASS'
      AND (
          REGEXP_EXTRACT(s.subject_uri, 'iconclass\\.org/([^/]+)', 1) LIKE '31A4%'
          OR REGEXP_EXTRACT(s.subject_uri, 'iconclass\\.org/([^/]+)', 1) LIKE '31AA4%'
          OR REGEXP_EXTRACT(s.subject_uri, 'iconclass\\.org/([^/]+)', 1) LIKE '31E%'
          OR REGEXP_EXTRACT(s.subject_uri, 'iconclass\\.org/([^/]+)', 1) LIKE '31F%'
      )
""").fetchone()[0]

print(f"\n📊 SUMMARY:")
print(f"   Total unique disease-related ICONCLASS topics: {len(disease_df)}")
print(f"   Total paintings with disease-related subjects: {total_disease_paintings}")

if len(disease_df) > 0:
    print(f"\n📋 ICONCLASS TOPIC LABELS (Disease-Related):")
    print("-" * 70)
    for row in disease_df.iter_rows(named=True):
        print(f"   [{row['iconclass_code']}] {row['subject_label']} ({row['painting_count']} paintings)")
    
    # Show category breakdown
    category_summary = disease_df.group_by('disease_category').agg([
        pl.count().alias('topic_count'),
        pl.sum('painting_count').alias('total_paintings')
    ]).sort('total_paintings', descending=True)
    
    print(f"\n📈 CATEGORY BREAKDOWN:")
    print(category_summary)
    
    # Show some example paintings
    print(f"\n🖼️ EXAMPLE PAINTINGS WITH DISEASE-RELATED SUBJECTS:")
    example_paintings = con.execute("""
        SELECT DISTINCT
            p.label as painting_label,
            p.nfdi_uri,
            s.subject_label as disease_topic,
            REGEXP_EXTRACT(s.subject_uri, 'iconclass\\.org/([^/]+)', 1) as iconclass_code
        FROM paintings p
        JOIN painting_subjects ps ON p.nfdi_uri = ps.nfdi_uri
        JOIN subjects s ON ps.subject_uri = s.subject_uri
        WHERE s.subject_source = 'ICONCLASS'
          AND (
              REGEXP_EXTRACT(s.subject_uri, 'iconclass\\.org/([^/]+)', 1) LIKE '31A4%'
              OR REGEXP_EXTRACT(s.subject_uri, 'iconclass\\.org/([^/]+)', 1) LIKE '31AA4%'
              OR REGEXP_EXTRACT(s.subject_uri, 'iconclass\\.org/([^/]+)', 1) LIKE '31E%'
              OR REGEXP_EXTRACT(s.subject_uri, 'iconclass\\.org/([^/]+)', 1) LIKE '31F%'
          )
        LIMIT 15
    """).pl()
    print(example_paintings)
    
    # Visualization
    if len(disease_df) > 0:
        viz_df = disease_df.to_pandas()
        chart = alt.Chart(viz_df).mark_bar().encode(
            x=alt.X('painting_count:Q', title='Number of Paintings'),
            y=alt.Y('subject_label:N', sort='-x', title='ICONCLASS Topic'),
            color=alt.Color('disease_category:N', 
                           title='Category',
                           scale=alt.Scale(scheme='dark2')),
            tooltip=[
                alt.Tooltip('subject_label:N', title='Topic'),
                alt.Tooltip('iconclass_code:N', title='ICONCLASS Code'),
                alt.Tooltip('disease_category:N', title='Category'),
                alt.Tooltip('painting_count:Q', title='Paintings')
            ]
        ).properties(
            title='Disease-Related ICONCLASS Subjects (31A4/31AA4/31E/31F)',
            width=700,
            height=max(200, len(viz_df) * 25)
        )
        ipython_display(chart)
else:
    print("\n⚠️ No disease-related ICONCLASS subjects found in the dataset.")

🏥 DISEASE-RELATED ICONCLASS ANALYSIS (31A4, 31AA4, 31E, 31F)

📊 SUMMARY:
   Total unique disease-related ICONCLASS topics: 40
   Total paintings with disease-related subjects: 54

📋 ICONCLASS TOPIC LABELS (Disease-Related):
----------------------------------------------------------------------
   [31A45231] sphinx (lion/woman); 'Sfinge' (Ripa) (6 paintings)
   [31E12] deathbed (5 paintings)
   [31F] symbols and personifications of Death (4 paintings)
   [31F26] Death as skeleton (3 paintings)
   [31A4561] merman (3 paintings)
   [31A456] monsters ~ fishes (3 paintings)
   [31E3] the corpse (2 paintings)
   [31F12] bones (symbol of Death) (2 paintings)
   [31F11] death's head, skull (symbol of Death) (2 paintings)
   [31E144] death of an adult person (2 paintings)
   [31A45] monsters of mixed human and animal shape; 'Mostri' (Ripa) (2 paintings)
   [31E2346] violent death by weapon (2 paintings)
   [31E23621] violent death by beheading (1 paintings)
   [31A422] dwarfs (1 paintings)
   [

C:\Users\thano\AppData\Local\Temp\ipykernel_33604\1087396341.py:73: DeprecationWarning: `pl.count()` is deprecated. Please use `pl.len()` instead.
(Deprecated in version 0.20.5)
  pl.count().alias('topic_count'),


alt.Chart(...)

In [19]:
# =============================================================================
# CELL 13d: Ancient Mythology ICONCLASS Analysis (Prefix 9)
# =============================================================================
# ICONCLASS category 9 = Classical Mythology and Ancient History
# Sub-categories include:
#   91 = Greco-Roman gods
#   92 = Greco-Roman heroes and legendary figures
#   93 = Greco-Roman legendary creatures
#   94 = Greek/Roman history
#   95 = Non-classical antiquity
#   96 = Allegories based on classical mythology

print("🏛️ ANCIENT MYTHOLOGY ICONCLASS ANALYSIS (Prefix 9)")
print("=" * 70)

mythology_df = con.execute("""
    WITH mythology_subjects AS (
        SELECT 
            s.subject_label,
            s.subject_uri,
            REGEXP_EXTRACT(s.subject_uri, 'iconclass\\.org/([^/]+)', 1) as iconclass_code,
            ps.nfdi_uri
        FROM painting_subjects ps
        JOIN subjects s ON ps.subject_uri = s.subject_uri
        WHERE s.subject_source = 'ICONCLASS'
          AND s.subject_label IS NOT NULL
          AND REGEXP_EXTRACT(s.subject_uri, 'iconclass\\.org/([^/]+)', 1) LIKE '9%'
    )
    SELECT 
        subject_label,
        iconclass_code,
        SUBSTRING(iconclass_code, 1, 2) as sub_category_code,
        COUNT(DISTINCT nfdi_uri) as painting_count
    FROM mythology_subjects
    GROUP BY subject_label, iconclass_code, sub_category_code
    ORDER BY painting_count DESC
""").pl()

# Sub-category mapping for ICONCLASS 9
MYTHOLOGY_SUBCATEGORIES = {
    '91': '91 - Greco-Roman Gods',
    '92': '92 - Heroes & Legends',
    '93': '93 - Legendary Creatures',
    '94': '94 - Greek/Roman History',
    '95': '95 - Non-Classical Antiquity',
    '96': '96 - Classical Allegories',
    '97': '97 - Metamorphoses',
    '98': '98 - Emblems',
    '9A': '9A - Other Classical'
}

# Add sub-category names
mythology_df = mythology_df.with_columns(
    pl.col('sub_category_code').replace(MYTHOLOGY_SUBCATEGORIES).alias('sub_category_name')
)

# Summary statistics
total_mythology_paintings = con.execute("""
    SELECT COUNT(DISTINCT ps.nfdi_uri) as count
    FROM painting_subjects ps
    JOIN subjects s ON ps.subject_uri = s.subject_uri
    WHERE s.subject_source = 'ICONCLASS'
      AND REGEXP_EXTRACT(s.subject_uri, 'iconclass\\.org/([^/]+)', 1) LIKE '9%'
""").fetchone()[0]

print(f"\n📊 SUMMARY:")
print(f"   Total unique mythology ICONCLASS topics: {len(mythology_df)}")
print(f"   Total paintings with mythology subjects: {total_mythology_paintings}")

if len(mythology_df) > 0:
    # Category breakdown
    category_summary = mythology_df.group_by('sub_category_code', 'sub_category_name').agg([
        pl.count().alias('topic_count'),
        pl.sum('painting_count').alias('total_paintings')
    ]).sort('total_paintings', descending=True)
    
    print(f"\n📈 SUB-CATEGORY BREAKDOWN:")
    print(category_summary)
    
    print(f"\n📋 TOP 30 ICONCLASS TOPIC LABELS (Ancient Mythology):")
    print("-" * 70)
    for i, row in enumerate(mythology_df.head(30).iter_rows(named=True)):
        print(f"   [{row['iconclass_code']}] {row['subject_label']} ({row['painting_count']} paintings)")
    
    if len(mythology_df) > 30:
        print(f"   ... and {len(mythology_df) - 30} more topics")
    
    # Show example paintings
    print(f"\n🖼️ EXAMPLE PAINTINGS WITH ANCIENT MYTHOLOGY SUBJECTS:")
    example_mythology_paintings = con.execute("""
        SELECT DISTINCT
            p.label as painting_label,
            p.nfdi_uri,
            s.subject_label as mythology_topic,
            REGEXP_EXTRACT(s.subject_uri, 'iconclass\\.org/([^/]+)', 1) as iconclass_code
        FROM paintings p
        JOIN painting_subjects ps ON p.nfdi_uri = ps.nfdi_uri
        JOIN subjects s ON ps.subject_uri = s.subject_uri
        WHERE s.subject_source = 'ICONCLASS'
          AND REGEXP_EXTRACT(s.subject_uri, 'iconclass\\.org/([^/]+)', 1) LIKE '9%'
        ORDER BY p.label
        LIMIT 20
    """).pl()
    print(example_mythology_paintings)
    
    # Visualization - Top mythology subjects by sub-category
    viz_df = mythology_df.head(50).to_pandas()
    
    chart = alt.Chart(viz_df).mark_bar().encode(
        x=alt.X('painting_count:Q', title='Number of Paintings'),
        y=alt.Y('subject_label:N', sort='-x', title='ICONCLASS Topic'),
        color=alt.Color('sub_category_name:N', 
                       title='Sub-Category',
                       scale=alt.Scale(scheme='tableau10')),
        tooltip=[
            alt.Tooltip('subject_label:N', title='Topic'),
            alt.Tooltip('iconclass_code:N', title='ICONCLASS Code'),
            alt.Tooltip('sub_category_name:N', title='Sub-Category'),
            alt.Tooltip('painting_count:Q', title='Paintings')
        ]
    ).properties(
        title='Ancient Mythology ICONCLASS Subjects (Category 9)',
        width=700,
        height=max(400, min(len(viz_df) * 20, 1000))
    )
    ipython_display(chart)
    
    # Additional: Pie chart of sub-category distribution
    pie_df = category_summary.to_pandas()
    pie_chart = alt.Chart(pie_df).mark_arc(innerRadius=50).encode(
        theta=alt.Theta('total_paintings:Q', title='Paintings'),
        color=alt.Color('sub_category_name:N', 
                       title='Sub-Category',
                       scale=alt.Scale(scheme='tableau10')),
        tooltip=[
            alt.Tooltip('sub_category_name:N', title='Category'),
            alt.Tooltip('topic_count:Q', title='Topics'),
            alt.Tooltip('total_paintings:Q', title='Paintings')
        ]
    ).properties(
        title='Distribution of Mythology Sub-Categories',
        width=400,
        height=400
    )
    ipython_display(pie_chart)
else:
    print("\n⚠️ No ancient mythology ICONCLASS subjects found in the dataset.")

🏛️ ANCIENT MYTHOLOGY ICONCLASS ANALYSIS (Prefix 9)

📊 SUMMARY:
   Total unique mythology ICONCLASS topics: 787
   Total paintings with mythology subjects: 1097

📈 SUB-CATEGORY BREAKDOWN:
shape: (9, 4)
┌───────────────────┬──────────────────────────────┬─────────────┬─────────────────┐
│ sub_category_code ┆ sub_category_name            ┆ topic_count ┆ total_paintings │
│ ---               ┆ ---                          ┆ ---         ┆ ---             │
│ str               ┆ str                          ┆ u32         ┆ i64             │
╞═══════════════════╪══════════════════════════════╪═════════════╪═════════════════╡
│ 92                ┆ 92 - Heroes & Legends        ┆ 414         ┆ 1803            │
│ 96                ┆ 96 - Classical Allegories    ┆ 72          ┆ 221             │
│ 94                ┆ 94 - Greek/Roman History     ┆ 79          ┆ 153             │
│ 98                ┆ 98 - Emblems                 ┆ 102         ┆ 115             │
│ 91                ┆ 91 - Greco-R

C:\Users\thano\AppData\Local\Temp\ipykernel_33604\164821764.py:73: DeprecationWarning: `pl.count()` is deprecated. Please use `pl.len()` instead.
(Deprecated in version 0.20.5)
  pl.count().alias('topic_count'),


alt.Chart(...)

alt.Chart(...)

In [20]:
# =============================================================================
# CELL 13d-2: Top Painters of Baroque Mythological Paintings
# =============================================================================
# Analyze which painters were most prominent in creating mythological paintings
# ICONCLASS prefixes 91-96, excluding cupids (92D1916)
#   91 = Greco-Roman gods
#   92 = Greco-Roman heroes and legendary figures (excluding 92D1916 cupids)
#   93 = Greco-Roman legendary creatures
#   94 = Greek/Roman history
#   95 = Non-classical antiquity
#   96 = Allegories based on classical mythology

print("🎨 TOP PAINTERS OF BAROQUE MYTHOLOGICAL PAINTINGS")
print("=" * 70)
print("   ICONCLASS categories 91-96 (excluding cupids 92D1916)")
print()

# Query top painters with mythology subjects
mythology_painters_df = con.execute("""
    WITH mythology_paintings AS (
        SELECT DISTINCT
            ps.nfdi_uri,
            s.subject_label,
            REGEXP_EXTRACT(s.subject_uri, 'iconclass\\.org/([^/]+)', 1) as iconclass_code
        FROM painting_subjects ps
        JOIN subjects s ON ps.subject_uri = s.subject_uri
        WHERE s.subject_source = 'ICONCLASS'
          AND s.subject_label IS NOT NULL
          AND (
              REGEXP_EXTRACT(s.subject_uri, 'iconclass\\.org/([^/]+)', 1) LIKE '91%'
              OR REGEXP_EXTRACT(s.subject_uri, 'iconclass\\.org/([^/]+)', 1) LIKE '92%'
              OR REGEXP_EXTRACT(s.subject_uri, 'iconclass\\.org/([^/]+)', 1) LIKE '93%'
              OR REGEXP_EXTRACT(s.subject_uri, 'iconclass\\.org/([^/]+)', 1) LIKE '94%'
              OR REGEXP_EXTRACT(s.subject_uri, 'iconclass\\.org/([^/]+)', 1) LIKE '95%'
              OR REGEXP_EXTRACT(s.subject_uri, 'iconclass\\.org/([^/]+)', 1) LIKE '96%'
          )
          -- Exclude cupids (92D1916)
          AND REGEXP_EXTRACT(s.subject_uri, 'iconclass\\.org/([^/]+)', 1) != '92D1916'
    ),
    painter_stats AS (
        SELECT 
            pp.person_name as painter_name,
            pp.person_id,
            COUNT(DISTINCT mp.nfdi_uri) as mythology_painting_count,
            COUNT(DISTINCT mp.iconclass_code) as unique_mythology_topics
        FROM mythology_paintings mp
        JOIN painting_persons pp ON mp.nfdi_uri = pp.nfdi_uri
        WHERE pp.role = 'PAINTER'
        GROUP BY pp.person_name, pp.person_id
    )
    SELECT 
        painter_name,
        person_id,
        mythology_painting_count,
        unique_mythology_topics
    FROM painter_stats
    ORDER BY mythology_painting_count DESC
    LIMIT 20
""").pl()

# Summary statistics
total_mythology_painters = con.execute("""
    SELECT COUNT(DISTINCT pp.person_name)
    FROM painting_subjects ps
    JOIN subjects s ON ps.subject_uri = s.subject_uri
    JOIN painting_persons pp ON ps.nfdi_uri = pp.nfdi_uri
    WHERE pp.role = 'PAINTER'
      AND s.subject_source = 'ICONCLASS'
      AND (
          REGEXP_EXTRACT(s.subject_uri, 'iconclass\\.org/([^/]+)', 1) LIKE '91%'
          OR REGEXP_EXTRACT(s.subject_uri, 'iconclass\\.org/([^/]+)', 1) LIKE '92%'
          OR REGEXP_EXTRACT(s.subject_uri, 'iconclass\\.org/([^/]+)', 1) LIKE '93%'
          OR REGEXP_EXTRACT(s.subject_uri, 'iconclass\\.org/([^/]+)', 1) LIKE '94%'
          OR REGEXP_EXTRACT(s.subject_uri, 'iconclass\\.org/([^/]+)', 1) LIKE '95%'
          OR REGEXP_EXTRACT(s.subject_uri, 'iconclass\\.org/([^/]+)', 1) LIKE '96%'
      )
      AND REGEXP_EXTRACT(s.subject_uri, 'iconclass\\.org/([^/]+)', 1) != '92D1916'
""").fetchone()[0]

print(f"📊 SUMMARY:")
print(f"   Total painters with mythology subjects: {total_mythology_painters}")
print(f"   Top 20 shown below")

if len(mythology_painters_df) > 0:
    print(f"\n🏆 TOP 20 PAINTERS OF MYTHOLOGICAL PAINTINGS:")
    print("-" * 70)
    for i, row in enumerate(mythology_painters_df.iter_rows(named=True), 1):
        print(f"   {i:2}. {row['painter_name']:<35} {row['mythology_painting_count']:>3} paintings, {row['unique_mythology_topics']:>3} topics")
    
    # Visualization
    chart = alt.Chart(mythology_painters_df.to_pandas()).mark_bar().encode(
        x=alt.X('mythology_painting_count:Q', title='Number of Mythological Paintings'),
        y=alt.Y('painter_name:N', sort='-x', title='Painter'),
        color=alt.Color('unique_mythology_topics:Q', 
                       title='Unique Topics',
                       scale=alt.Scale(scheme='viridis')),
        tooltip=[
            alt.Tooltip('painter_name:N', title='Painter'),
            alt.Tooltip('mythology_painting_count:Q', title='Paintings'),
            alt.Tooltip('unique_mythology_topics:Q', title='Unique Topics')
        ]
    ).properties(
        title='Top 20 Painters of Baroque Mythological Paintings (91-96, excl. cupids)',
        width=600,
        height=500
    )
    ipython_display(chart)

# Get example paintings for top 3 painters
print(f"\n🖼️ EXAMPLE MYTHOLOGICAL PAINTINGS BY TOP PAINTERS:")
print("=" * 70)

top_3_painters = mythology_painters_df.head(3)['painter_name'].to_list()

for painter in top_3_painters:
    print(f"\n👤 {painter}:")
    print("-" * 50)
    
    example_paintings = con.execute("""
        SELECT DISTINCT
            p.label as painting_label,
            s.subject_label as mythology_subject,
            REGEXP_EXTRACT(s.subject_uri, 'iconclass\\.org/([^/]+)', 1) as iconclass_code,
            b.name as building
        FROM paintings p
        JOIN painting_subjects ps ON p.nfdi_uri = ps.nfdi_uri
        JOIN subjects s ON ps.subject_uri = s.subject_uri
        JOIN painting_persons pp ON p.nfdi_uri = pp.nfdi_uri
        LEFT JOIN buildings b ON p.building_id = b.building_id
        WHERE pp.person_name = ?
          AND pp.role = 'PAINTER'
          AND s.subject_source = 'ICONCLASS'
          AND (
              REGEXP_EXTRACT(s.subject_uri, 'iconclass\\.org/([^/]+)', 1) LIKE '91%'
              OR REGEXP_EXTRACT(s.subject_uri, 'iconclass\\.org/([^/]+)', 1) LIKE '92%'
              OR REGEXP_EXTRACT(s.subject_uri, 'iconclass\\.org/([^/]+)', 1) LIKE '93%'
              OR REGEXP_EXTRACT(s.subject_uri, 'iconclass\\.org/([^/]+)', 1) LIKE '94%'
              OR REGEXP_EXTRACT(s.subject_uri, 'iconclass\\.org/([^/]+)', 1) LIKE '95%'
              OR REGEXP_EXTRACT(s.subject_uri, 'iconclass\\.org/([^/]+)', 1) LIKE '96%'
          )
          AND REGEXP_EXTRACT(s.subject_uri, 'iconclass\\.org/([^/]+)', 1) != '92D1916'
        ORDER BY p.label
        LIMIT 5
    """, [painter]).pl()
    
    for row in example_paintings.iter_rows(named=True):
        title = row['painting_label'][:50] + '...' if len(row['painting_label']) > 50 else row['painting_label']
        building = row['building'][:30] + '...' if row['building'] and len(row['building']) > 30 else (row['building'] or 'Unknown')
        print(f"   • {title}")
        print(f"     📍 {building}")
        print(f"     🏛️ [{row['iconclass_code']}] {row['mythology_subject']}")

# Show most popular mythology subjects among these painters
print(f"\n📚 MOST POPULAR MYTHOLOGY SUBJECTS AMONG TOP PAINTERS:")
print("-" * 70)

popular_subjects = con.execute("""
    SELECT 
        s.subject_label,
        REGEXP_EXTRACT(s.subject_uri, 'iconclass\\.org/([^/]+)', 1) as iconclass_code,
        COUNT(DISTINCT pp.person_name) as painter_count,
        COUNT(DISTINCT ps.nfdi_uri) as painting_count
    FROM painting_subjects ps
    JOIN subjects s ON ps.subject_uri = s.subject_uri
    JOIN painting_persons pp ON ps.nfdi_uri = pp.nfdi_uri
    WHERE pp.role = 'PAINTER'
      AND s.subject_source = 'ICONCLASS'
      AND (
          REGEXP_EXTRACT(s.subject_uri, 'iconclass\\.org/([^/]+)', 1) LIKE '91%'
          OR REGEXP_EXTRACT(s.subject_uri, 'iconclass\\.org/([^/]+)', 1) LIKE '92%'
          OR REGEXP_EXTRACT(s.subject_uri, 'iconclass\\.org/([^/]+)', 1) LIKE '93%'
          OR REGEXP_EXTRACT(s.subject_uri, 'iconclass\\.org/([^/]+)', 1) LIKE '94%'
          OR REGEXP_EXTRACT(s.subject_uri, 'iconclass\\.org/([^/]+)', 1) LIKE '95%'
          OR REGEXP_EXTRACT(s.subject_uri, 'iconclass\\.org/([^/]+)', 1) LIKE '96%'
      )
      AND REGEXP_EXTRACT(s.subject_uri, 'iconclass\\.org/([^/]+)', 1) != '92D1916'
    GROUP BY s.subject_label, iconclass_code
    ORDER BY painting_count DESC
    LIMIT 15
""").pl()

for row in popular_subjects.iter_rows(named=True):
    print(f"   [{row['iconclass_code']}] {row['subject_label']}")
    print(f"      → {row['painting_count']} paintings by {row['painter_count']} painters")

🎨 TOP PAINTERS OF BAROQUE MYTHOLOGICAL PAINTINGS
   ICONCLASS categories 91-96 (excluding cupids 92D1916)

📊 SUMMARY:
   Total painters with mythology subjects: 152
   Top 20 shown below

🏆 TOP 20 PAINTERS OF MYTHOLOGICAL PAINTINGS:
----------------------------------------------------------------------
    1. Castelli, Carlo Ludovico             28 paintings,  66 topics
    2. Asam, Cosmas Damian                  22 paintings,  34 topics
    3. Wentzing, Heinrich Richard           19 paintings,  23 topics
    4. Richter, Johann Wilhelm              19 paintings,  23 topics
    5. Harms, Johann Oswald                 18 paintings,  35 topics
    6. Lammers, Seivert                     18 paintings,  22 topics
    7. Asam, Hans Georg                     14 paintings,  24 topics
    8. Asam, Maria Theresia                 13 paintings,  22 topics
    9. Ritter, Johann Heinrich              12 paintings,  70 topics
   10. Wunder, Wilhelm Ernst                11 paintings,  15 topics
   11.

alt.Chart(...)


🖼️ EXAMPLE MYTHOLOGICAL PAINTINGS BY TOP PAINTERS:

👤 Castelli, Carlo Ludovico:
--------------------------------------------------
   • Aeneas vor Jupiter
     📍 Bad Arolsen, Residenzschloss
     🏛️ [96C%28AENEAS%29] (story of) Aeneas
   • Aeneas vor Jupiter
     📍 Bad Arolsen, Residenzschloss
     🏛️ [92B1] (story of) Jupiter (Zeus)
   • Aeneas vor Jupiter
     📍 Bad Arolsen, Residenzschloss
     🏛️ [96C%28AENEAS%2969] apotheosis, deification of Aeneas
   • Allegorie des Frühlings
     📍 Eckardtshausen, Schloss Wilhel...
     🏛️ [96A23] (story of) Flora
   • Allegorie des Frühlings
     📍 Eckardtshausen, Schloss Wilhel...
     🏛️ [96A239] (story of) Flora - offspring, companion(s), train etc.

👤 Asam, Cosmas Damian:
--------------------------------------------------
   • Apoll mit Pferdegespann
     📍 Alteglofsheim, Schloss
     🏛️ [92B378] triumph of Apollo
   • Apoll tötet Coronis
     📍 Schönach, Schloss
     🏛️ [92B38%28LYRE%29] attributes of Apollo: lyre
   • Apoll tötet Coronis

In [21]:
# =============================================================================
# CELL 13d-3: Top Baroque Buildings for Mythological Paintings
# =============================================================================
# Analyze which buildings contain the most mythological paintings
# ICONCLASS prefixes 91-96, excluding cupids (92D1916)

print("🏛️ TOP BAROQUE BUILDINGS FOR MYTHOLOGICAL PAINTINGS")
print("=" * 70)
print("   ICONCLASS categories 91-96 (excluding cupids 92D1916)")
print()

# Query top buildings with mythology paintings
# Note: Building names typically include city (e.g., "Rastatt, Residenzschloss")
# We extract city from building name using SPLIT_PART
mythology_buildings_df = con.execute("""
    WITH mythology_paintings AS (
        SELECT DISTINCT
            ps.nfdi_uri,
            s.subject_label,
            REGEXP_EXTRACT(s.subject_uri, 'iconclass\\.org/([^/]+)', 1) as iconclass_code
        FROM painting_subjects ps
        JOIN subjects s ON ps.subject_uri = s.subject_uri
        WHERE s.subject_source = 'ICONCLASS'
          AND s.subject_label IS NOT NULL
          AND (
              REGEXP_EXTRACT(s.subject_uri, 'iconclass\\.org/([^/]+)', 1) LIKE '91%'
              OR REGEXP_EXTRACT(s.subject_uri, 'iconclass\\.org/([^/]+)', 1) LIKE '92%'
              OR REGEXP_EXTRACT(s.subject_uri, 'iconclass\\.org/([^/]+)', 1) LIKE '93%'
              OR REGEXP_EXTRACT(s.subject_uri, 'iconclass\\.org/([^/]+)', 1) LIKE '94%'
              OR REGEXP_EXTRACT(s.subject_uri, 'iconclass\\.org/([^/]+)', 1) LIKE '95%'
              OR REGEXP_EXTRACT(s.subject_uri, 'iconclass\\.org/([^/]+)', 1) LIKE '96%'
          )
          -- Exclude cupids (92D1916)
          AND REGEXP_EXTRACT(s.subject_uri, 'iconclass\\.org/([^/]+)', 1) != '92D1916'
    ),
    building_stats AS (
        SELECT 
            b.name as building_name,
            b.building_id,
            SPLIT_PART(b.name, ',', 1) as location_city,
            b.location_state,
            b.function as building_function,
            COUNT(DISTINCT mp.nfdi_uri) as mythology_painting_count,
            COUNT(DISTINCT mp.iconclass_code) as unique_mythology_topics
        FROM mythology_paintings mp
        JOIN paintings p ON mp.nfdi_uri = p.nfdi_uri
        JOIN buildings b ON p.building_id = b.building_id
        GROUP BY b.name, b.building_id, b.location_state, b.function
    )
    SELECT 
        building_name,
        building_id,
        location_city,
        location_state,
        building_function,
        mythology_painting_count,
        unique_mythology_topics
    FROM building_stats
    ORDER BY mythology_painting_count DESC
    LIMIT 20
""").pl()

# Summary statistics
total_mythology_buildings = con.execute("""
    SELECT COUNT(DISTINCT b.building_id)
    FROM painting_subjects ps
    JOIN subjects s ON ps.subject_uri = s.subject_uri
    JOIN paintings p ON ps.nfdi_uri = p.nfdi_uri
    JOIN buildings b ON p.building_id = b.building_id
    WHERE s.subject_source = 'ICONCLASS'
      AND (
          REGEXP_EXTRACT(s.subject_uri, 'iconclass\\.org/([^/]+)', 1) LIKE '91%'
          OR REGEXP_EXTRACT(s.subject_uri, 'iconclass\\.org/([^/]+)', 1) LIKE '92%'
          OR REGEXP_EXTRACT(s.subject_uri, 'iconclass\\.org/([^/]+)', 1) LIKE '93%'
          OR REGEXP_EXTRACT(s.subject_uri, 'iconclass\\.org/([^/]+)', 1) LIKE '94%'
          OR REGEXP_EXTRACT(s.subject_uri, 'iconclass\\.org/([^/]+)', 1) LIKE '95%'
          OR REGEXP_EXTRACT(s.subject_uri, 'iconclass\\.org/([^/]+)', 1) LIKE '96%'
      )
      AND REGEXP_EXTRACT(s.subject_uri, 'iconclass\\.org/([^/]+)', 1) != '92D1916'
""").fetchone()[0]

print(f"📊 SUMMARY:")
print(f"   Total buildings with mythology paintings: {total_mythology_buildings}")
print(f"   Top 20 shown below")

if len(mythology_buildings_df) > 0:
    print(f"\n🏆 TOP 20 BUILDINGS WITH MYTHOLOGICAL PAINTINGS:")
    print("-" * 70)
    for i, row in enumerate(mythology_buildings_df.iter_rows(named=True), 1):
        building_name = row['building_name'][:40] + '...' if row['building_name'] and len(row['building_name']) > 40 else (row['building_name'] or 'Unknown')
        city = row['location_city'] or 'Unknown'
        state = row['location_state'] or ''
        func = row['building_function'] or ''
        print(f"   {i:2}. {building_name}")
        print(f"       📍 {city}, {state} | {func}")
        print(f"       🎨 {row['mythology_painting_count']} paintings, {row['unique_mythology_topics']} unique topics")
    
    # Visualization - Bar chart
    chart = alt.Chart(mythology_buildings_df.head(15).to_pandas()).mark_bar().encode(
        x=alt.X('mythology_painting_count:Q', title='Number of Mythological Paintings'),
        y=alt.Y('building_name:N', sort='-x', title='Building'),
        color=alt.Color('unique_mythology_topics:Q', 
                       title='Unique Topics',
                       scale=alt.Scale(scheme='plasma')),
        tooltip=[
            alt.Tooltip('building_name:N', title='Building'),
            alt.Tooltip('location_city:N', title='City'),
            alt.Tooltip('location_state:N', title='State'),
            alt.Tooltip('building_function:N', title='Function'),
            alt.Tooltip('mythology_painting_count:Q', title='Paintings'),
            alt.Tooltip('unique_mythology_topics:Q', title='Unique Topics')
        ]
    ).properties(
        title='Top 15 Baroque Buildings for Mythological Paintings (91-96, excl. cupids)',
        width=600,
        height=450
    )
    ipython_display(chart)

# Buildings by function type
print(f"\n📊 BUILDINGS BY FUNCTION TYPE:")
print("-" * 70)

buildings_by_function = con.execute("""
    SELECT 
        COALESCE(b.function, 'Unknown') as building_function,
        COUNT(DISTINCT b.building_id) as building_count,
        COUNT(DISTINCT ps.nfdi_uri) as total_paintings
    FROM painting_subjects ps
    JOIN subjects s ON ps.subject_uri = s.subject_uri
    JOIN paintings p ON ps.nfdi_uri = p.nfdi_uri
    JOIN buildings b ON p.building_id = b.building_id
    WHERE s.subject_source = 'ICONCLASS'
      AND (
          REGEXP_EXTRACT(s.subject_uri, 'iconclass\\.org/([^/]+)', 1) LIKE '91%'
          OR REGEXP_EXTRACT(s.subject_uri, 'iconclass\\.org/([^/]+)', 1) LIKE '92%'
          OR REGEXP_EXTRACT(s.subject_uri, 'iconclass\\.org/([^/]+)', 1) LIKE '93%'
          OR REGEXP_EXTRACT(s.subject_uri, 'iconclass\\.org/([^/]+)', 1) LIKE '94%'
          OR REGEXP_EXTRACT(s.subject_uri, 'iconclass\\.org/([^/]+)', 1) LIKE '95%'
          OR REGEXP_EXTRACT(s.subject_uri, 'iconclass\\.org/([^/]+)', 1) LIKE '96%'
      )
      AND REGEXP_EXTRACT(s.subject_uri, 'iconclass\\.org/([^/]+)', 1) != '92D1916'
    GROUP BY b.function
    ORDER BY total_paintings DESC
""").pl()

for row in buildings_by_function.iter_rows(named=True):
    print(f"   {row['building_function']:<30} {row['building_count']:>3} buildings, {row['total_paintings']:>4} paintings")

# Geographic distribution - by state
print(f"\n🗺️ GEOGRAPHIC DISTRIBUTION BY STATE:")
print("-" * 70)

buildings_by_state = con.execute("""
    SELECT 
        COALESCE(b.location_state, 'Unknown') as state,
        COUNT(DISTINCT b.building_id) as building_count,
        COUNT(DISTINCT ps.nfdi_uri) as total_paintings
    FROM painting_subjects ps
    JOIN subjects s ON ps.subject_uri = s.subject_uri
    JOIN paintings p ON ps.nfdi_uri = p.nfdi_uri
    JOIN buildings b ON p.building_id = b.building_id
    WHERE s.subject_source = 'ICONCLASS'
      AND (
          REGEXP_EXTRACT(s.subject_uri, 'iconclass\\.org/([^/]+)', 1) LIKE '91%'
          OR REGEXP_EXTRACT(s.subject_uri, 'iconclass\\.org/([^/]+)', 1) LIKE '92%'
          OR REGEXP_EXTRACT(s.subject_uri, 'iconclass\\.org/([^/]+)', 1) LIKE '93%'
          OR REGEXP_EXTRACT(s.subject_uri, 'iconclass\\.org/([^/]+)', 1) LIKE '94%'
          OR REGEXP_EXTRACT(s.subject_uri, 'iconclass\\.org/([^/]+)', 1) LIKE '95%'
          OR REGEXP_EXTRACT(s.subject_uri, 'iconclass\\.org/([^/]+)', 1) LIKE '96%'
      )
      AND REGEXP_EXTRACT(s.subject_uri, 'iconclass\\.org/([^/]+)', 1) != '92D1916'
    GROUP BY state
    ORDER BY total_paintings DESC
    LIMIT 10
""").pl()

for row in buildings_by_state.iter_rows(named=True):
    print(f"   {row['state']:<25} {row['building_count']:>3} buildings, {row['total_paintings']:>4} paintings")

# Show example paintings for top 3 buildings
print(f"\n🖼️ EXAMPLE MYTHOLOGICAL PAINTINGS IN TOP BUILDINGS:")
print("=" * 70)

top_3_buildings = mythology_buildings_df.head(3)

for row in top_3_buildings.iter_rows(named=True):
    building_name = row['building_name']
    building_id = row['building_id']
    city = row['location_city'] or 'Unknown'
    
    print(f"\n🏛️ {building_name}")
    print(f"   📍 {city}")
    print("-" * 50)
    
    example_paintings = con.execute("""
        SELECT DISTINCT
            p.label as painting_label,
            s.subject_label as mythology_subject,
            REGEXP_EXTRACT(s.subject_uri, 'iconclass\\.org/([^/]+)', 1) as iconclass_code
        FROM paintings p
        JOIN painting_subjects ps ON p.nfdi_uri = ps.nfdi_uri
        JOIN subjects s ON ps.subject_uri = s.subject_uri
        WHERE p.building_id = ?
          AND s.subject_source = 'ICONCLASS'
          AND (
              REGEXP_EXTRACT(s.subject_uri, 'iconclass\\.org/([^/]+)', 1) LIKE '91%'
              OR REGEXP_EXTRACT(s.subject_uri, 'iconclass\\.org/([^/]+)', 1) LIKE '92%'
              OR REGEXP_EXTRACT(s.subject_uri, 'iconclass\\.org/([^/]+)', 1) LIKE '93%'
              OR REGEXP_EXTRACT(s.subject_uri, 'iconclass\\.org/([^/]+)', 1) LIKE '94%'
              OR REGEXP_EXTRACT(s.subject_uri, 'iconclass\\.org/([^/]+)', 1) LIKE '95%'
              OR REGEXP_EXTRACT(s.subject_uri, 'iconclass\\.org/([^/]+)', 1) LIKE '96%'
          )
          AND REGEXP_EXTRACT(s.subject_uri, 'iconclass\\.org/([^/]+)', 1) != '92D1916'
        ORDER BY p.label
        LIMIT 5
    """, [building_id]).pl()
    
    for painting_row in example_paintings.iter_rows(named=True):
        title = painting_row['painting_label'][:50] + '...' if len(painting_row['painting_label']) > 50 else painting_row['painting_label']
        print(f"   • {title}")
        print(f"     🏛️ [{painting_row['iconclass_code']}] {painting_row['mythology_subject']}")

🏛️ TOP BAROQUE BUILDINGS FOR MYTHOLOGICAL PAINTINGS
   ICONCLASS categories 91-96 (excluding cupids 92D1916)

📊 SUMMARY:
   Total buildings with mythology paintings: 197
   Top 20 shown below

🏆 TOP 20 BUILDINGS WITH MYTHOLOGICAL PAINTINGS:
----------------------------------------------------------------------
    1. Landshut, Stadtresidenz
       📍 Landshut, Bayern | Residenz
       🎨 65 paintings, 74 unique topics
    2. Sondershausen, Residenzschloss
       📍 Sondershausen, Thüringen | Hauptresidenz
       🎨 25 paintings, 55 unique topics
    3. Alteglofsheim, Schloss
       📍 Alteglofsheim, Bayern | Herrenhaus / Rittersitz / Gutshof / Hofmark / Häuptlingssitz
       🎨 22 paintings, 18 unique topics
    4. Bad Arolsen, Residenzschloss
       📍 Bad Arolsen, Hessen | Hauptresidenz
       🎨 14 paintings, 32 unique topics
    5. Schönach, Schloss
       📍 Schönach, Bayern | Schloss
       🎨 13 paintings, 22 unique topics
    6. Landshut, Burg Trausnitz
       📍 Landshut, Bayern | Nebenr

alt.Chart(...)


📊 BUILDINGS BY FUNCTION TYPE:
----------------------------------------------------------------------
   Hauptresidenz                   30 buildings,  131 paintings
   Residenz                        14 buildings,  108 paintings
   Herrenhaus / Rittersitz / Gutshof / Hofmark / Häuptlingssitz  33 buildings,   89 paintings
   Schloss                         18 buildings,   47 paintings
   Lustschloss                     11 buildings,   39 paintings
   Unknown                         13 buildings,   29 paintings
   Jagdschloss                      9 buildings,   26 paintings
   Witwensitz                       7 buildings,   24 paintings
   Nebenresidenz                    3 buildings,   15 paintings
   Landschloss                      1 buildings,   11 paintings
   Wohnhaus                         9 buildings,    9 paintings
   Amtshaus / Amtssitz              6 buildings,    9 paintings
   Sammlungsbau                     2 buildings,    8 paintings
   Wohn- und Arbeits-/Geschäftshaus 

In [22]:
# =============================================================================
# CELL 13e: Display Images for a Specific ICONCLASS Code
# =============================================================================
# Enter an ICONCLASS code to view related paintings and their images
# Examples: 31E12 (diseases), 92C4 (Venus), 92D1916 (cupids/putti)

from IPython.display import display, HTML

# ============================================
# 🔧 CONFIGURE: Set your ICONCLASS code here
# ============================================
ICONCLASS_CODE = "92C4"  # Change this to explore different subjects
MAX_IMAGES = 10           # Maximum number of images to display

print(f"🖼️ IMAGES FOR ICONCLASS CODE: {ICONCLASS_CODE}")
print("=" * 70)

# First, get the label for this ICONCLASS code
subject_info = con.execute("""
    SELECT 
        subject_label,
        subject_uri
    FROM subjects
    WHERE subject_source = 'ICONCLASS'
      AND REGEXP_EXTRACT(subject_uri, 'iconclass\\.org/([^/]+)', 1) = ?
    LIMIT 1
""", [ICONCLASS_CODE]).fetchone()

if subject_info:
    print(f"📚 Topic: {subject_info[0]}")
    print(f"🔗 URI: {subject_info[1]}")
else:
    print(f"⚠️ No subject found with code '{ICONCLASS_CODE}'")
    print("   Try codes like: 92D1916, 92C4, 31A5331, 11G, etc.")

# Query paintings with this ICONCLASS code that have images
paintings_with_images = con.execute("""
    SELECT DISTINCT
        p.label as painting_label,
        p.nfdi_uri,
        p.imageUrl,
        b.name as building,
        p.year_start,
        p.year_end
    FROM paintings p
    JOIN painting_subjects ps ON p.nfdi_uri = ps.nfdi_uri
    JOIN subjects s ON ps.subject_uri = s.subject_uri
    LEFT JOIN buildings b ON p.building_id = b.building_id
    WHERE s.subject_source = 'ICONCLASS'
      AND REGEXP_EXTRACT(s.subject_uri, 'iconclass\\.org/([^/]+)', 1) = ?
      AND p.imageUrl IS NOT NULL
    ORDER BY p.label
""", [ICONCLASS_CODE]).pl()

# Also count total (with and without images)
total_count = con.execute("""
    SELECT COUNT(DISTINCT ps.nfdi_uri)
    FROM painting_subjects ps
    JOIN subjects s ON ps.subject_uri = s.subject_uri
    WHERE s.subject_source = 'ICONCLASS'
      AND REGEXP_EXTRACT(s.subject_uri, 'iconclass\\.org/([^/]+)', 1) = ?
""", [ICONCLASS_CODE]).fetchone()[0]

print(f"\n📊 Found {total_count} paintings with this subject")
print(f"   {len(paintings_with_images)} have images available")

if len(paintings_with_images) > 0:
    # Limit to MAX_IMAGES
    display_df = paintings_with_images.head(MAX_IMAGES)
    
    if len(paintings_with_images) > MAX_IMAGES:
        print(f"   Showing first {MAX_IMAGES} images")
    
    print("\n" + "-" * 70)
    
    # Generate HTML gallery
    html_parts = [f"""
    <style>
        .iconclass-gallery {{
            display: flex;
            flex-wrap: wrap;
            gap: 20px;
            justify-content: flex-start;
        }}
        .iconclass-item {{
            width: 300px;
            border: 1px solid #ddd;
            border-radius: 8px;
            overflow: hidden;
            background: #fff;
            box-shadow: 0 2px 5px rgba(0,0,0,0.1);
        }}
        .iconclass-item img {{
            width: 100%;
            height: 200px;
            object-fit: cover;
        }}
        .iconclass-item .info {{
            padding: 10px;
            font-size: 12px;
        }}
        .iconclass-item .title {{
            font-weight: bold;
            margin-bottom: 5px;
            font-size: 13px;
            color: #333;
        }}
        .iconclass-item .meta {{
            color: #666;
            font-size: 11px;
        }}
    </style>
    <h3>🎨 Paintings with ICONCLASS {ICONCLASS_CODE}</h3>
    <div class="iconclass-gallery">
    """]
    
    for row in display_df.iter_rows(named=True):
        year_str = ""
        if row['year_start'] and row['year_end']:
            if row['year_start'] == row['year_end']:
                year_str = str(row['year_start'])
            else:
                year_str = f"{row['year_start']}-{row['year_end']}"
        elif row['year_start']:
            year_str = f"from {row['year_start']}"
        
        building = row['building'] or 'Unknown location'
        title = row['painting_label'][:60] + '...' if len(row['painting_label']) > 60 else row['painting_label']
        
        html_parts.append(f"""
        <div class="iconclass-item">
            <a href="{row['nfdi_uri']}" target="_blank">
                <img src="{row['imageUrl']}" alt="{title}" onerror="this.src='https://via.placeholder.com/300x200?text=Image+not+available'">
            </a>
            <div class="info">
                <div class="title">{title}</div>
                <div class="meta">📍 {building}</div>
                <div class="meta">📅 {year_str}</div>
            </div>
        </div>
        """)
    
    html_parts.append("</div>")
    display(HTML("".join(html_parts)))
    
    # Also show as table
    print("\n📋 Painting Details:")
    print(display_df.select(['painting_label', 'building', 'year_start', 'year_end']))
else:
    print("\n⚠️ No images found for this ICONCLASS code.")
    print("   The paintings may exist but lack image URLs in the dataset.")

🖼️ IMAGES FOR ICONCLASS CODE: 92C4
📚 Topic: (story of) Venus (Aphrodite)
🔗 URI: https://iconclass.org/92C4

📊 Found 37 paintings with this subject
   37 have images available
   Showing first 10 images

----------------------------------------------------------------------



📋 Painting Details:
shape: (10, 4)
┌─────────────────────────────────┬─────────────────────────────────┬────────────┬──────────┐
│ painting_label                  ┆ building                        ┆ year_start ┆ year_end │
│ ---                             ┆ ---                             ┆ ---        ┆ ---      │
│ str                             ┆ str                             ┆ i32        ┆ i32      │
╞═════════════════════════════════╪═════════════════════════════════╪════════════╪══════════╡
│ "auff der Rehnbahn / im Kunsth… ┆ Kassel, ehem. Kunsthaus         ┆ 1701       ┆ 1702     │
│ Allegorie des Feuers            ┆ Bückeburg, Residenzschloss      ┆ 1605       ┆ 1605     │
│ Apotheose des Herkules          ┆ null                            ┆ 1740       ┆ 1740     │
│ Ceres, Venus und Bacchus mit G… ┆ Bad Arolsen, Residenzschloss    ┆ 1721       ┆ 1722     │
│ Das Deckengemälde               ┆ null                            ┆ 1705       ┆ 1515     │
│ Das Deckengemälde     

In [23]:
# =============================================================================
# CELL 13c: ICONCLASS Category Summary Visualization
# =============================================================================
# Donut chart showing category distribution + faceted detail view

# Category summary donut chart
category_summary_pd = category_summary.to_pandas()

donut = alt.Chart(category_summary_pd).mark_arc(innerRadius=60, outerRadius=120).encode(
    theta=alt.Theta('total_paintings:Q', stack=True),
    color=alt.Color('category_name:N', 
                   title='ICONCLASS Category',
                   scale=alt.Scale(scheme='category10'),
                   legend=alt.Legend(orient='right', titleLimit=200, labelLimit=200)),
    tooltip=[
        alt.Tooltip('category_name:N', title='Category'),
        alt.Tooltip('subject_count:Q', title='Unique Subjects'),
        alt.Tooltip('total_paintings:Q', title='Total Paintings')
    ]
).properties(
    title='ICONCLASS Categories in Baroque Ceiling Paintings',
    width=300,
    height=300
)

# Text labels in center showing totals
text_center = alt.Chart(category_summary_pd).mark_text(
    size=20, fontWeight='bold'
).encode(
    text=alt.value(f"{category_summary_pd['total_paintings'].sum():,}")
).properties(width=300, height=300)

text_label = alt.Chart(category_summary_pd).mark_text(
    size=12, dy=25
).encode(
    text=alt.value("total subject\noccurrences")
).properties(width=300, height=300)

donut_with_text = donut + text_center + text_label

# Top subjects per category (faceted view)
top_per_category = category_df.group_by('category_name').head(5).to_pandas()

faceted = alt.Chart(top_per_category).mark_bar().encode(
    x=alt.X('painting_count:Q', title='Paintings'),
    y=alt.Y('subject_label:N', sort='-x', title=None),
    color=alt.Color('category_name:N', legend=None),
    tooltip=['subject_label', 'iconclass_code', 'painting_count']
).properties(
    width=150,
    height=120
).facet(
    column=alt.Column('category_name:N', title=None, header=alt.Header(labelFontSize=10, labelAngle=0))
).properties(
    title='Top 5 Subjects per ICONCLASS Category'
).resolve_scale(
    y='independent'
)

ipython_display(donut_with_text)
ipython_display(faceted)

alt.LayerChart(...)

alt.FacetChart(...)

In [24]:
# =============================================================================
# CELL 13d: Painter Topic Profile Analysis (Full ICONCLASS Hierarchy)
# =============================================================================
# Analyze each painter's thematic preferences using FULL ICONCLASS codes
# 
# ICONCLASS Structure: e.g., "92D1521" = 
#   9 (Mythology) > 92 (specific) > 92D (gods) > 92D1 (Apollo) > etc.
# 
# Approach:
# 1. For each painter, collect all FULL ICONCLASS codes from their paintings
# 2. Compute hierarchical similarity between painters using code prefix matching
# 3. Distance metric: weighted prefix similarity (deeper matches = higher weight)

import numpy as np
from collections import Counter

print("🎨 PAINTER TOPIC PROFILE ANALYSIS (Full ICONCLASS Hierarchy)")
print("=" * 70)

# Step 1: Get all (painter, iconclass_code) pairs for ceiling paintings
painter_subjects_df = con.execute("""
    SELECT 
        pp.person_name as painter,
        REGEXP_EXTRACT(s.subject_uri, 'iconclass\\.org/([^/]+)', 1) as iconclass_code,
        s.subject_label
    FROM painting_persons pp
    JOIN paintings p ON pp.nfdi_uri = p.nfdi_uri
    JOIN painting_subjects ps ON p.nfdi_uri = ps.nfdi_uri
    JOIN subjects s ON ps.subject_uri = s.subject_uri
    WHERE pp.role = 'PAINTER'
      AND s.subject_source = 'ICONCLASS'
      AND s.subject_label IS NOT NULL
      -- Only ceiling paintings
      AND p.nfdi_uri IN (
          SELECT ps2.nfdi_uri 
          FROM painting_subjects ps2
          JOIN subjects s2 ON ps2.subject_uri = s2.subject_uri
          WHERE s2.subject_source = 'GETTY_AAT' 
            AND s2.subject_label = 'ceiling paintings'
      )
""").pl()

print(f"📊 Total painter-subject pairs: {len(painter_subjects_df):,}")

# Step 2: Filter to painters with enough data (min 20 subject occurrences)
painter_counts = painter_subjects_df.group_by('painter').len()
active_painters = painter_counts.filter(pl.col('len') >= 20)['painter'].to_list()
print(f"📊 Painters with ≥20 subject occurrences: {len(active_painters)}")

# Filter to active painters
painter_subjects_filtered = painter_subjects_df.filter(pl.col('painter').is_in(active_painters))

# Step 3: Build ICONCLASS code frequency vectors for each painter
# Store as dict: painter -> {code: count}
painter_code_vectors = {}
for painter in active_painters:
    codes = painter_subjects_filtered.filter(pl.col('painter') == painter)['iconclass_code'].to_list()
    painter_code_vectors[painter] = Counter(codes)

# Also extract all unique codes for analysis
all_unique_codes = set()
for codes in painter_code_vectors.values():
    all_unique_codes.update(codes.keys())

print(f"📊 Unique ICONCLASS codes across all painters: {len(all_unique_codes)}")

# Step 4: Define hierarchical prefix extraction function
def get_all_prefixes(code):
    """
    Extract all hierarchical prefixes from an ICONCLASS code.
    e.g., "92D1521" -> ["9", "92", "92D", "92D1", "92D15", "92D152", "92D1521"]
    """
    if not code:
        return []
    prefixes = []
    current = ""
    for char in code:
        current += char
        # Only add meaningful prefixes (skip URL-encoded parts like %28)
        if not current.endswith('%'):
            prefixes.append(current)
    return prefixes

# Build prefix frequency vectors (weighted by depth)
# Deeper prefixes get exponentially higher weights
def build_prefix_vector(code_counter):
    """Build a prefix-weighted vector from ICONCLASS code frequencies"""
    prefix_weights = Counter()
    for code, count in code_counter.items():
        prefixes = get_all_prefixes(code)
        for i, prefix in enumerate(prefixes):
            # Weight increases with depth: 2^(depth-1)
            depth_weight = 2 ** i
            prefix_weights[prefix] += count * depth_weight
    return prefix_weights

painter_prefix_vectors = {p: build_prefix_vector(codes) for p, codes in painter_code_vectors.items()}

# Collect all prefixes across all painters
all_prefixes = set()
for pv in painter_prefix_vectors.values():
    all_prefixes.update(pv.keys())

print(f"📊 Total unique prefixes (all depths): {len(all_prefixes)}")

# Step 5: Build a matrix for painters x top-level categories (for overview)
ICONCLASS_CATEGORIES = {
    '0': 'Abstract, Non-representational Art',
    '1': 'Religion and Magic',
    '2': 'Nature',
    '3': 'Human Being, Man in General',
    '4': 'Society, Civilization, Culture',
    '5': 'Abstract Ideas and Concepts',
    '6': 'History',
    '7': 'Bible',
    '8': 'Literature',
    '9': 'Classical Mythology and Ancient History'
}

# Also build the top-level category matrix for visualization
def get_top_category(code):
    if code and len(code) > 0 and code[0].isdigit():
        return code[0]
    return None

painter_subjects_with_cat = painter_subjects_filtered.with_columns(
    pl.col('iconclass_code').map_elements(get_top_category, return_dtype=pl.Utf8).alias('category')
).filter(pl.col('category').is_not_null())

category_counts = painter_subjects_with_cat.group_by(['painter', 'category']).len().rename({'len': 'count'})
painter_matrix = category_counts.pivot(
    values='count',
    index='painter', 
    on='category'
).fill_null(0)

painter_totals = painter_subjects_with_cat.group_by('painter').len().rename({'len': 'total'})
painter_matrix = painter_matrix.join(painter_totals, on='painter')

# Show sample of the most common full codes per painter
print("\n📋 Sample: Top 5 most used ICONCLASS codes by top painter:")
top_painter = active_painters[0] if active_painters else None
if top_painter:
    top_codes = painter_code_vectors[top_painter].most_common(5)
    for code, count in top_codes:
        label = painter_subjects_filtered.filter(
            (pl.col('painter') == top_painter) & (pl.col('iconclass_code') == code)
        )['subject_label'].to_list()[0]
        print(f"  {code}: {label} ({count}x)")

🎨 PAINTER TOPIC PROFILE ANALYSIS (Full ICONCLASS Hierarchy)
📊 Total painter-subject pairs: 7,899
📊 Painters with ≥20 subject occurrences: 90
📊 Unique ICONCLASS codes across all painters: 2263
📊 Total unique prefixes (all depths): 9545

📋 Sample: Top 5 most used ICONCLASS codes by top painter:
  25G21: fruits (2x)
  48A9872: ornament ~ grotesque (2x)
  73D: Passion of Christ (1x)
  11D122: cross with symbol of Christ on it, e.g. lamb (1x)
  11L14: 'ante legem', 'sub lege', 'sub gratia' (1x)


In [25]:
# =============================================================================
# CELL 13e: Painter Topic Metrics (Full Hierarchical Analysis)
# =============================================================================
# Compute diversity using full ICONCLASS codes, not just top-level categories

# Get category columns (digits 0-9) for top-level view
all_categories = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']
available_categories = [c for c in all_categories if c in painter_matrix.columns]

# Convert to numpy for calculations
painters_list = painter_matrix['painter'].to_list()
totals = painter_matrix['total'].to_numpy()

# Build percentage matrix for top-level categories (for visualization)
pct_data = {}
for cat in available_categories:
    if cat in painter_matrix.columns:
        pct_data[cat] = (painter_matrix[cat].to_numpy() / totals * 100).round(1)
    else:
        pct_data[cat] = np.zeros(len(painters_list))

# =============================================================================
# NEW: Hierarchical Diversity Score using FULL ICONCLASS codes
# =============================================================================
# Instead of just top-level category entropy, we compute:
# 1. Code-level entropy: diversity across unique full ICONCLASS codes
# 2. Depth-weighted diversity: considers how deeply specialized the painter is

def shannon_entropy(probs):
    """Compute Shannon entropy from probability distribution"""
    probs = np.array(probs)
    probs = probs[probs > 0]
    if len(probs) == 0:
        return 0
    probs = probs / probs.sum()
    return -np.sum(probs * np.log2(probs))

def compute_hierarchical_diversity(code_counter):
    """
    Compute hierarchical diversity score that considers:
    1. Number of unique codes at each hierarchy level
    2. How evenly distributed codes are at each level
    """
    if not code_counter:
        return 0, 0, 0
    
    # Group codes by prefix length (hierarchy depth)
    level_codes = {}  # depth -> set of prefixes at that depth
    for code in code_counter.keys():
        prefixes = get_all_prefixes(code)
        for depth, prefix in enumerate(prefixes):
            if depth not in level_codes:
                level_codes[depth] = Counter()
            level_codes[depth][prefix] += code_counter[code]
    
    # Compute entropy at each level
    level_entropies = []
    level_unique_counts = []
    for depth in sorted(level_codes.keys()):
        counts = list(level_codes[depth].values())
        level_entropies.append(shannon_entropy(counts))
        level_unique_counts.append(len(level_codes[depth]))
    
    # Aggregate metrics
    total_unique_codes = len(code_counter)
    avg_entropy = np.mean(level_entropies) if level_entropies else 0
    max_depth = max(level_codes.keys()) if level_codes else 0
    
    return total_unique_codes, avg_entropy, max_depth

# Compute metrics for each painter
hierarchical_metrics = []
for painter in painters_list:
    code_counter = painter_code_vectors.get(painter, Counter())
    unique_codes, avg_entropy, max_depth = compute_hierarchical_diversity(code_counter)
    
    # Also compute top-level category stats
    top_counts = [painter_matrix[cat].to_numpy()[painters_list.index(painter)] 
                  if cat in painter_matrix.columns else 0 
                  for cat in available_categories]
    
    top_level_entropy = shannon_entropy(top_counts)
    
    # Dominant category
    if sum(top_counts) > 0:
        max_idx = np.argmax(top_counts)
        dominant_cat = available_categories[max_idx]
        dominant_pct = top_counts[max_idx] / sum(top_counts) * 100
    else:
        dominant_cat = None
        dominant_pct = 0
    
    hierarchical_metrics.append({
        'painter': painter,
        'unique_codes': unique_codes,
        'avg_level_entropy': avg_entropy,
        'max_depth': max_depth,
        'top_level_entropy': top_level_entropy,
        'dominant_category': dominant_cat,
        'dominant_pct': dominant_pct
    })

metrics_df = pl.DataFrame(hierarchical_metrics)

# Normalize diversity scores
max_unique = metrics_df['unique_codes'].max()
max_top_entropy = np.log2(len(available_categories))

# Create final profiles dataframe
painter_profiles = pl.DataFrame({
    'painter': painters_list,
    'total_subjects': totals,
}).join(metrics_df, on='painter')

# Add combined diversity score (weighted combination of metrics)
painter_profiles = painter_profiles.with_columns([
    (pl.col('unique_codes') / max_unique * 50 + 
     pl.col('avg_level_entropy') / 5 * 30 +  # Avg entropy rarely exceeds 5
     pl.col('top_level_entropy') / max_top_entropy * 20
    ).alias('diversity_score'),
    pl.col('dominant_category').replace(ICONCLASS_CATEGORIES).alias('dominant_category_name')
])

# Normalize to 0-100
max_div = painter_profiles['diversity_score'].max()
painter_profiles = painter_profiles.with_columns(
    (pl.col('diversity_score') / max_div * 100).alias('diversity_normalized')
)

# Sort by total subjects
painter_profiles = painter_profiles.sort('total_subjects', descending=True)

# Also keep entropies list for compatibility with heatmap
entropies = painter_profiles['diversity_score'].to_list()

print("📊 PAINTER TOPIC PROFILES (Full Hierarchical Analysis)")
print("=" * 100)
print(painter_profiles.select([
    'painter', 'total_subjects', 'unique_codes', 'dominant_category_name', 
    'dominant_pct', 'diversity_normalized'
]).head(20))

print(f"\n📈 Hierarchical Diversity Statistics:")
print(f"   Unique ICONCLASS codes range: {metrics_df['unique_codes'].min()} - {metrics_df['unique_codes'].max()}")
print(f"   Average unique codes per painter: {metrics_df['unique_codes'].mean():.1f}")
most_diverse = painter_profiles.sort('diversity_normalized', descending=True).head(1)
most_focused = painter_profiles.sort('diversity_normalized').head(1)
print(f"   Most diverse painter: {most_diverse['painter'][0]} ({most_diverse['unique_codes'][0]} unique codes)")
print(f"   Most focused painter: {most_focused['painter'][0]} ({most_focused['unique_codes'][0]} unique codes)")

📊 PAINTER TOPIC PROFILES (Full Hierarchical Analysis)
shape: (20, 6)
┌─────────────────┬────────────────┬──────────────┬────────────────┬──────────────┬────────────────┐
│ painter         ┆ total_subjects ┆ unique_codes ┆ dominant_categ ┆ dominant_pct ┆ diversity_norm │
│ ---             ┆ ---            ┆ ---          ┆ ory_name       ┆ ---          ┆ alized         │
│ str             ┆ u32            ┆ i64          ┆ ---            ┆ f64          ┆ ---            │
│                 ┆                ┆              ┆ str            ┆              ┆ f64            │
╞═════════════════╪════════════════╪══════════════╪════════════════╪══════════════╪════════════════╡
│ Hermann, Franz  ┆ 556            ┆ 298          ┆ Society,       ┆ 48.021583    ┆ 100.0          │
│ Georg           ┆                ┆              ┆ Civilization,  ┆              ┆                │
│                 ┆                ┆              ┆ Culture        ┆              ┆                │
│ Asam, Cosmas    ┆ 32

In [26]:
# =============================================================================
# CELL 13f: Painter Topic Heatmap Visualization
# =============================================================================
# Create a heatmap showing topic distribution for top painters

# Prepare data for heatmap - top 25 painters by total subjects
top_n = 25
top_painters = painter_profiles.head(top_n)['painter'].to_list()

# Build long-form data for Altair
heatmap_data = []
for painter in top_painters:
    idx = painters_list.index(painter)
    total = totals[idx]
    diversity = entropies[idx]
    
    for cat in available_categories:
        if cat in painter_matrix.columns:
            count = painter_matrix.filter(pl.col('painter') == painter)[cat].to_numpy()[0]
            pct = count / total * 100 if total > 0 else 0
        else:
            pct = 0
        
        heatmap_data.append({
            'painter': painter,
            'category': cat,
            'category_name': ICONCLASS_CATEGORIES.get(cat, cat),
            'percentage': round(pct, 1),
            'count': int(count) if cat in painter_matrix.columns else 0,
            'diversity_score': round(diversity, 2)
        })

heatmap_df = pl.DataFrame(heatmap_data).to_pandas()

# Sort painters by total subjects (already sorted in painter_profiles)
painter_order = top_painters

# Create heatmap
heatmap = alt.Chart(heatmap_df).mark_rect().encode(
    x=alt.X('category_name:N', 
            title='ICONCLASS Category',
            sort=['Abstract, Non-representational Art', 'Religion and Magic', 'Nature',
                  'Human Being, Man in General', 'Society, Civilization, Culture',
                  'Abstract Ideas and Concepts', 'History', 'Bible', 'Literature',
                  'Classical Mythology and Ancient History'],
            axis=alt.Axis(labelAngle=-45, labelLimit=150)),
    y=alt.Y('painter:N', title='Painter', sort=painter_order),
    color=alt.Color('percentage:Q', 
                   scale=alt.Scale(scheme='viridis', domain=[0, 60]),
                   title='% of Subjects'),
    tooltip=[
        alt.Tooltip('painter:N', title='Painter'),
        alt.Tooltip('category_name:N', title='Category'),
        alt.Tooltip('percentage:Q', title='Percentage'),
        alt.Tooltip('count:Q', title='Count'),
        alt.Tooltip('diversity_score:Q', title='Diversity Score')
    ]
).properties(
    title=f'Topic Distribution Heatmap (Top {top_n} Painters by Subject Count)',
    width=600,
    height=600
)

# Add diversity score as a separate bar chart
diversity_bars = alt.Chart(
    painter_profiles.head(top_n).to_pandas()
).mark_bar(color='steelblue').encode(
    x=alt.X('diversity_normalized:Q', title='Diversity Score (0-100)', scale=alt.Scale(domain=[0, 100])),
    y=alt.Y('painter:N', title=None, sort=painter_order, axis=None),
    tooltip=[
        alt.Tooltip('painter:N', title='Painter'),
        alt.Tooltip('diversity_normalized:Q', title='Diversity %'),
        alt.Tooltip('dominant_category_name:N', title='Dominant Category')
    ]
).properties(
    width=150,
    height=600,
    title='Diversity'
)

# Combine horizontally
combined = alt.hconcat(heatmap, diversity_bars).resolve_scale(y='shared')

ipython_display(combined)

alt.HConcatChart(...)

In [27]:
# =============================================================================
# CELL 13g: Painter Similarity Analysis (Hierarchical ICONCLASS Distance)
# =============================================================================
# Compute distance using FULL ICONCLASS hierarchy with weighted prefix matching
# 
# Key insight: Two painters who both depict "92D15" (Apollo with lyre) are MORE
# similar than two who share only "9" (both mythology) 

import numpy as np

print("🔗 PAINTER SIMILARITY ANALYSIS (Full Hierarchical Distance)")
print("=" * 70)

# =============================================================================
# Hierarchical Jaccard Similarity with Depth Weighting
# =============================================================================
def hierarchical_similarity(codes1, codes2, max_weight_depth=8):
    """
    Compute similarity between two sets of ICONCLASS codes using hierarchical prefix matching.
    
    For each pair of codes, similarity is based on the longest common prefix.
    Deeper shared prefixes indicate higher similarity.
    
    Returns a value between 0 (no overlap) and 1 (identical code sets).
    """
    if not codes1 or not codes2:
        return 0.0
    
    # Get all prefixes from both code sets (with counts)
    prefixes1 = Counter()
    prefixes2 = Counter()
    
    for code, count in codes1.items():
        for i, prefix in enumerate(get_all_prefixes(code)[:max_weight_depth]):
            depth_weight = (i + 1) ** 2  # Quadratic depth weighting
            prefixes1[prefix] += count * depth_weight
    
    for code, count in codes2.items():
        for i, prefix in enumerate(get_all_prefixes(code)[:max_weight_depth]):
            depth_weight = (i + 1) ** 2
            prefixes2[prefix] += count * depth_weight
    
    # Compute weighted Jaccard-like similarity
    all_prefixes = set(prefixes1.keys()) | set(prefixes2.keys())
    
    intersection_sum = 0
    union_sum = 0
    
    for prefix in all_prefixes:
        v1 = prefixes1.get(prefix, 0)
        v2 = prefixes2.get(prefix, 0)
        intersection_sum += min(v1, v2)
        union_sum += max(v1, v2)
    
    return intersection_sum / union_sum if union_sum > 0 else 0.0

# Compute pairwise hierarchical similarity for top painters
top_n_sim = 20
top_painters_sim = painter_profiles.head(top_n_sim)['painter'].to_list()

print(f"Computing hierarchical similarity for top {top_n_sim} painters...")
print(f"Painters: {top_painters_sim}")

# Build similarity matrix
hierarchical_sim_matrix = np.zeros((top_n_sim, top_n_sim))

for i, p1 in enumerate(top_painters_sim):
    for j, p2 in enumerate(top_painters_sim):
        if i == j:
            hierarchical_sim_matrix[i, j] = 1.0
        elif j > i:
            sim = hierarchical_similarity(
                painter_code_vectors.get(p1, {}),
                painter_code_vectors.get(p2, {})
            )
            hierarchical_sim_matrix[i, j] = sim
            hierarchical_sim_matrix[j, i] = sim

print("✅ Similarity matrix computed!")

# Prepare data for heatmap
sim_data = []
for i, p1 in enumerate(top_painters_sim):
    for j, p2 in enumerate(top_painters_sim):
        sim_data.append({
            'painter_1': p1,
            'painter_2': p2,
            'similarity': float(hierarchical_sim_matrix[i, j])  # Ensure it's a Python float
        })

sim_df = pl.DataFrame(sim_data).to_pandas()

# Debug: Check data types
print(f"\nDataFrame dtypes:\n{sim_df.dtypes}")
print(f"\nSample data:\n{sim_df.head()}")

# Create similarity heatmap - using simplest possible encoding
similarity_heatmap = alt.Chart(sim_df).mark_rect().encode(
    x=alt.X('painter_1:N', title='Painter', sort=top_painters_sim,
            axis=alt.Axis(labelAngle=-45, labelLimit=150)),
    y=alt.Y('painter_2:N', title='Painter', sort=top_painters_sim,
            axis=alt.Axis(labelLimit=150)),
    color=alt.Color('similarity:Q', 
                   scale=alt.Scale(scheme='redyellowgreen', domain=[0, 1]),
                   title='Similarity')
).properties(
    title='Painter Topic Similarity (Weighted Hierarchical ICONCLASS Distance)',
    width=550,
    height=550
)

ipython_display(similarity_heatmap)

# Find most similar and most different pairs
sim_pairs = []
for i in range(len(top_painters_sim)):
    for j in range(i+1, len(top_painters_sim)):
        sim_pairs.append({
            'painter_1': top_painters_sim[i],
            'painter_2': top_painters_sim[j],
            'similarity': hierarchical_sim_matrix[i, j]
        })

sim_pairs_df = pl.DataFrame(sim_pairs).sort('similarity', descending=True)

print("\n🔗 Most Similar Painter Pairs (by full ICONCLASS hierarchy):")
print("="*70)
for row in sim_pairs_df.head(5).iter_rows(named=True):
    # Find shared codes
    codes1 = set(painter_code_vectors.get(row['painter_1'], {}).keys())
    codes2 = set(painter_code_vectors.get(row['painter_2'], {}).keys())
    shared = len(codes1 & codes2)
    print(f"  {row['painter_1'][:25]:<28} ↔ {row['painter_2'][:25]:<28}")
    print(f"      Similarity: {row['similarity']:.3f} | Shared codes: {shared}")

print("\n🔀 Most Different Painter Pairs:")
print("="*70)
for row in sim_pairs_df.tail(5).iter_rows(named=True):
    codes1 = set(painter_code_vectors.get(row['painter_1'], {}).keys())
    codes2 = set(painter_code_vectors.get(row['painter_2'], {}).keys())
    shared = len(codes1 & codes2)
    print(f"  {row['painter_1'][:25]:<28} ↔ {row['painter_2'][:25]:<28}")
    print(f"      Similarity: {row['similarity']:.3f} | Shared codes: {shared}")

🔗 PAINTER SIMILARITY ANALYSIS (Full Hierarchical Distance)
Computing hierarchical similarity for top 20 painters...
Painters: ['Hermann, Franz Georg', 'Asam, Cosmas Damian', 'Harms, Johann Oswald', 'Castelli, Carlo Ludovico', 'Wolcker, Johann Georg', 'Höder, Johann Wilhelm', 'Kilian, Magnus', 'Ritter, Johann Heinrich', 'Lammers, Seivert', 'Kuen, Franz Martin', 'Wohlhaupter, Emanuel', 'Huber, Johann Joseph Anton', 'Spiegler, Franz Josef', 'Asam, Hans Georg', 'Asam, Maria Theresia', 'Speer, Martin', 'Paur, Ignaz', 'Rode, Bernhard', 'Paduano, Alessandro', 'Hackert, Jakob Philipp']
✅ Similarity matrix computed!

DataFrame dtypes:
painter_1         str
painter_2         str
similarity    float64
dtype: object

Sample data:
              painter_1                 painter_2  similarity
0  Hermann, Franz Georg      Hermann, Franz Georg    1.000000
1  Hermann, Franz Georg       Asam, Cosmas Damian    0.161927
2  Hermann, Franz Georg      Harms, Johann Oswald    0.118208
3  Hermann, Franz Georg 

alt.Chart(...)


🔗 Most Similar Painter Pairs (by full ICONCLASS hierarchy):
  Asam, Hans Georg             ↔ Asam, Maria Theresia        
      Similarity: 0.950 | Shared codes: 107
  Asam, Cosmas Damian          ↔ Asam, Maria Theresia        
      Similarity: 0.386 | Shared codes: 101
  Asam, Cosmas Damian          ↔ Asam, Hans Georg            
      Similarity: 0.380 | Shared codes: 101
  Wolcker, Johann Georg        ↔ Spiegler, Franz Josef       
      Similarity: 0.175 | Shared codes: 28
  Hermann, Franz Georg         ↔ Wolcker, Johann Georg       
      Similarity: 0.172 | Shared codes: 41

🔀 Most Different Painter Pairs:
  Paduano, Alessandro          ↔ Hackert, Jakob Philipp      
      Similarity: 0.015 | Shared codes: 0
  Ritter, Johann Heinrich      ↔ Paur, Ignaz                 
      Similarity: 0.014 | Shared codes: 0
  Höder, Johann Wilhelm        ↔ Ritter, Johann Heinrich     
      Similarity: 0.013 | Shared codes: 1
  Höder, Johann Wilhelm        ↔ Speer, Martin               
    

In [28]:
# =============================================================================
# CELL 13h: Individual Painter Topic Profiles vs Average (with Full Hierarchy Details)
# =============================================================================
# Compare individual painters against the overall "average" + show specific ICONCLASS codes

# For top-level category comparison, use normalized matrix
category_columns = [col for col in painter_matrix.columns if col in ICONCLASS_CATEGORIES]
matrix_np = painter_matrix.select(category_columns).to_numpy().astype(float)
row_sums = matrix_np.sum(axis=1, keepdims=True)
row_sums[row_sums == 0] = 1
matrix_normalized = matrix_np / row_sums

avg_profile = matrix_normalized.mean(axis=0)
avg_profile_pct = avg_profile * 100

# Select featured painters (most prolific + most unique based on our analysis)
featured_painters = [
    "Hermann, Franz Georg",        # Most subjects
    "Asam, Cosmas Damian",         # Famous Baroque painter
    "Asam, Hans Georg",            # High similarity cluster
    "Pictorius, Johann Martin",    # Highly diverse
    "Meil, Johann Ludwig",         # Most focused
]

# Only include painters that exist in our data
featured_painters = [p for p in featured_painters if p in painters_list]

# Build comparison data for top-level categories
comparison_data = []

# Add average profile
for idx, cat in enumerate(category_columns):
    comparison_data.append({
        'painter': '📊 AVERAGE (all painters)',
        'category': cat,
        'category_name': ICONCLASS_CATEGORIES.get(cat, cat),
        'percentage': round(avg_profile_pct[idx], 2),
        'is_average': True
    })

# Add individual painter profiles
for painter in featured_painters:
    p_idx = painters_list.index(painter)
    profile_pct = matrix_normalized[p_idx] * 100
    
    for idx, cat in enumerate(category_columns):
        comparison_data.append({
            'painter': painter,
            'category': cat,
            'category_name': ICONCLASS_CATEGORIES.get(cat, cat),
            'percentage': round(profile_pct[idx], 2),
            'is_average': False
        })

comparison_df = pl.DataFrame(comparison_data).to_pandas()

# Create faceted bar chart
category_order = ['Abstract, Non-representational Art', 'Religion and Magic', 'Nature',
                  'Human Being, Man in General', 'Society, Civilization, Culture',
                  'Abstract Ideas and Concepts', 'History', 'Bible', 'Literature',
                  'Classical Mythology and Ancient History']
category_order = [c for c in category_order if c in comparison_df['category_name'].unique()]

profile_chart = alt.Chart(comparison_df).mark_bar().encode(
    x=alt.X('category_name:N', title='Topic Category', sort=category_order,
            axis=alt.Axis(labelAngle=-45, labelLimit=150)),
    y=alt.Y('percentage:Q', title='Percentage of Subjects', scale=alt.Scale(domain=[0, 70])),
    color=alt.Color('is_average:N', 
                   scale=alt.Scale(domain=[False, True], range=['steelblue', 'orange']),
                   legend=alt.Legend(title='Type', labelExpr="datum.value ? 'Average' : 'Individual'")),
    tooltip=[
        alt.Tooltip('painter:N', title='Painter'),
        alt.Tooltip('category_name:N', title='Category'),
        alt.Tooltip('percentage:Q', title='Percentage', format='.1f')
    ]
).properties(
    width=200,
    height=200
).facet(
    facet=alt.Facet('painter:N', title=None, 
                    sort=['📊 AVERAGE (all painters)'] + featured_painters,
                    header=alt.Header(labelFontSize=11, labelLimit=200)),
    columns=3
).properties(
    title='Painter Topic Profiles vs Overall Average (Top-Level Categories)'
)

ipython_display(profile_chart)

# =============================================================================
# NEW: Show FULL ICONCLASS code signatures for each painter
# =============================================================================
print("\n" + "=" * 80)
print("🎨 PAINTER ICONCLASS SIGNATURES (Full Hierarchical Codes)")
print("=" * 80)
print("Each painter's top 8 most frequently used ICONCLASS codes:\n")

for painter in featured_painters:
    codes = painter_code_vectors.get(painter, Counter())
    total_codes = sum(codes.values())
    unique_codes = len(codes)
    
    # Get profile info
    profile_row = painter_profiles.filter(pl.col('painter') == painter)
    diversity = profile_row['diversity_normalized'][0] if len(profile_row) > 0 else 0
    
    print(f"📌 {painter}")
    print(f"   Unique codes: {unique_codes} | Diversity: {diversity:.1f}%")
    print(f"   Top codes:")
    
    top_codes = codes.most_common(8)
    for code, count in top_codes:
        # Get label for this code
        label_rows = painter_subjects_filtered.filter(
            (pl.col('painter') == painter) & (pl.col('iconclass_code') == code)
        )['subject_label'].to_list()
        label = label_rows[0] if label_rows else "Unknown"
        pct = count / total_codes * 100
        print(f"      {code:<15} ({pct:5.1f}%) {label[:45]}")
    print()

# Compute hierarchical distance from "average painter"
print("=" * 80)
print("📏 HIERARCHICAL DISTANCE FROM AVERAGE")
print("=" * 80)

# Build average code vector (aggregate of all painters)
avg_code_vector = Counter()
for painter in active_painters:
    for code, count in painter_code_vectors.get(painter, {}).items():
        avg_code_vector[code] += count

# Normalize by number of painters
for code in avg_code_vector:
    avg_code_vector[code] = avg_code_vector[code] / len(active_painters)

print(f"\nDistance from average code distribution (0 = identical, lower = more typical):\n")
for painter in featured_painters:
    sim = hierarchical_similarity(
        painter_code_vectors.get(painter, {}),
        avg_code_vector
    )
    distance = 1 - sim
    
    # Get unique codes count
    unique = len(painter_code_vectors.get(painter, {}))
    
    bar = "█" * int(distance * 40)
    print(f"  {painter:<35} {distance:.3f} {bar}")

alt.FacetChart(...)


🎨 PAINTER ICONCLASS SIGNATURES (Full Hierarchical Codes)
Each painter's top 8 most frequently used ICONCLASS codes:

📌 Hermann, Franz Georg
   Unique codes: 298 | Diversity: 100.0%
   Top codes:
      26A             (  2.9%) clouds
      92D1916         (  2.0%) cupids: 'amores', 'amoretti', 'putti'
      49MM32          (  1.8%) book - MM - book open
      11G192          (  1.6%) angel(s) represented as youth(s) or adult(s)
      41D263          (  1.4%) walking-stick, staff, cane
      48C161          (  1.4%) column, pillar ~ architecture
      11G191          (  1.4%) angel(s) represented as child(ren)
      41B31           (  1.3%) candle

📌 Asam, Cosmas Damian
   Unique codes: 255 | Diversity: 88.7%
   Top codes:
      44B192          (  1.2%) sceptre, staff (symbol of sovereignty)
      44B191          (  1.2%) crown (symbol of sovereignty)
      11H%28John%20Nepomuk%294 (  1.2%) [11H%28John%20Nepomuk%294]
      61B2            (  1.2%) historical persons
      44B112        

In [29]:
# =============================================================================
# CELL 14: Painter Collaboration Network
# =============================================================================

print("🤝 PAINTER COLLABORATIONS")
print("=" * 60)
print("(Painters who worked on the same painting)")

collabs_df = con.execute("""
    SELECT 
        pp1.person_name AS painter1,
        pp2.person_name AS painter2,
        COUNT(DISTINCT pp1.nfdi_uri) AS collaborations
    FROM painting_persons pp1
    JOIN painting_persons pp2 
        ON pp1.nfdi_uri = pp2.nfdi_uri 
        AND pp1.person_id < pp2.person_id
    WHERE pp1.role = 'PAINTER' AND pp2.role = 'PAINTER'
    GROUP BY pp1.person_name, pp2.person_name
    HAVING COUNT(DISTINCT pp1.nfdi_uri) >= 1
    ORDER BY collaborations DESC
    LIMIT 20
""").pl()

print(collabs_df)

# Visualization - Collaboration pairs
if len(collabs_df) > 0 and collabs_df['collaborations'].max() > 0:
    df_viz = collabs_df.head(15).to_pandas()
    df_viz['pair'] = df_viz['painter1'].str[:20] + ' + ' + df_viz['painter2'].str[:20]
    
    chart = alt.Chart(df_viz).mark_bar().encode(
        x=alt.X('collaborations:Q', title='Number of Collaborations'),
        y=alt.Y('pair:N', sort='-x', title='Painter Pair'),
        color=alt.Color('collaborations:Q', scale=alt.Scale(scheme='purples'), legend=None),
        tooltip=['painter1', 'painter2', 'collaborations']
    ).properties(
        title='Painter Collaborations',
        width=500,
        height=400
    )
    ipython_display(chart)

🤝 PAINTER COLLABORATIONS
(Painters who worked on the same painting)
shape: (20, 3)
┌────────────────────────────┬────────────────────────────┬────────────────┐
│ painter1                   ┆ painter2                   ┆ collaborations │
│ ---                        ┆ ---                        ┆ ---            │
│ str                        ┆ str                        ┆ i64            │
╞════════════════════════════╪════════════════════════════╪════════════════╡
│ Richter, Johann Wilhelm    ┆ Wentzing, Heinrich Richard ┆ 30             │
│ Wentzing, Heinrich Richard ┆ Lammers, Seivert           ┆ 28             │
│ Richter, Johann Wilhelm    ┆ Lammers, Seivert           ┆ 28             │
│ Asam, Hans Georg           ┆ Asam, Maria Theresia       ┆ 24             │
│ Asam, Cosmas Damian        ┆ Asam, Maria Theresia       ┆ 23             │
│ …                          ┆ …                          ┆ …              │
│ Zimmermann, Franz Michael  ┆ Zimmermann, Johann Baptist ┆ 6         

alt.Chart(...)

---
# Bildindex Cross-Reference Analysis

The following cells demonstrate how to join CbDD ceiling paintings with Bildindex historical photographs using shared GND identifiers.

In [30]:
# =============================================================================
# CELL: Bildindex Overview Statistics
# =============================================================================

print("📗 BILDINDEX DATA OVERVIEW")
print("=" * 60)

try:
    # Check if Bildindex tables exist
    bi_tables = con.execute("""
        SELECT table_name FROM information_schema.tables 
        WHERE table_name LIKE 'bi_%'
    """).fetchall()
    
    if not bi_tables:
        print("⚠️ Bildindex tables not loaded. Run the database creation cells first.")
    else:
        # Basic counts
        bi_items_count = con.execute("SELECT COUNT(*) FROM bi_items").fetchone()[0]
        bi_buildings_count = con.execute("SELECT COUNT(*) FROM bi_buildings").fetchone()[0]
        bi_painters_count = con.execute("SELECT COUNT(*) FROM bi_painters").fetchone()[0]
        bi_subjects_count = con.execute("SELECT COUNT(*) FROM bi_subjects").fetchone()[0]
        
        print(f"   📷 Bildindex items:           {bi_items_count:>6,}")
        print(f"   🏛️  Building connections:     {bi_buildings_count:>6,}")
        print(f"   🎨 Painter connections:       {bi_painters_count:>6,}")
        print(f"   📜 Subject connections:       {bi_subjects_count:>6,}")
        
        # GND overlap summary
        print("\n📊 GND Overlap Summary:")
        gnd_summary = con.execute("""
            SELECT 
                gnd_type,
                COUNT(*) as shared_gnds,
                SUM(cbdd_count) as cbdd_items,
                SUM(bildindex_count) as bildindex_items
            FROM bi_gnd_overlaps
            GROUP BY gnd_type
        """).pl()
        print(gnd_summary)
        
        # Collection method breakdown
        print("\n📈 Collection Method:")
        method_df = con.execute("""
            SELECT 
                collection_method,
                tier,
                COUNT(*) as item_count
            FROM bi_items
            GROUP BY collection_method, tier
            ORDER BY tier, item_count DESC
        """).pl()
        print(method_df)
        
except Exception as e:
    print(f"⚠️ Error querying Bildindex tables: {e}")

📗 BILDINDEX DATA OVERVIEW
   📷 Bildindex items:              948
   🏛️  Building connections:        211
   🎨 Painter connections:          737
   📜 Subject connections:          946

📊 GND Overlap Summary:
shape: (2, 4)
┌──────────┬─────────────┬───────────────┬─────────────────┐
│ gnd_type ┆ shared_gnds ┆ cbdd_items    ┆ bildindex_items │
│ ---      ┆ ---         ┆ ---           ┆ ---             │
│ str      ┆ i64         ┆ decimal[38,0] ┆ decimal[38,0]   │
╞══════════╪═════════════╪═══════════════╪═════════════════╡
│ painter  ┆ 364         ┆ 2294          ┆ 737             │
│ building ┆ 69          ┆ 784           ┆ 211             │
└──────────┴─────────────┴───────────────┴─────────────────┘

📈 Collection Method:
shape: (2, 3)
┌───────────────────┬──────┬────────────┐
│ collection_method ┆ tier ┆ item_count │
│ ---               ┆ ---  ┆ ---        │
│ str               ┆ i64  ┆ i64        │
╞═══════════════════╪══════╪════════════╡
│ building_gnd      ┆ 1    ┆ 211        │
│ p

In [31]:
# =============================================================================
# CELL: Cross-Reference: CbDD Paintings with Bildindex Images
# =============================================================================
# Find CbDD paintings that have related Bildindex images via shared painter GNDs

print("🔗 CROSS-REFERENCE: CbDD Paintings ↔ Bildindex Images")
print("=" * 60)

try:
    # Find paintings with Bildindex connections via painter GND
    cross_ref_df = con.execute("""
        WITH cbdd_painter_gnds AS (
            -- Extract individual GNDs from the piped string in paintings
            SELECT DISTINCT
                p.nfdi_uri,
                p.label as painting_label,
                p.painters,
                p.building_name,
                TRIM(gnd.value) as painter_gnd
            FROM paintings p,
                 UNNEST(STRING_SPLIT(p.creatorGnds, '|')) as gnd(value)
            WHERE p.creatorGnds IS NOT NULL AND p.creatorGnds != ''
        )
        SELECT 
            cpg.painting_label,
            cpg.painters as cbdd_painter,
            cpg.building_name,
            bi.title as bildindex_title,
            bi.bildindex_url,
            bi.tier as collection_tier
        FROM cbdd_painter_gnds cpg
        JOIN bi_painters bp ON cpg.painter_gnd = bp.painter_gnd
        JOIN bi_items bi ON bp.bildindex_uri = bi.bildindex_uri
        ORDER BY cpg.painters, cpg.painting_label
        LIMIT 50
    """).pl()
    
    if len(cross_ref_df) > 0:
        print(f"✅ Found {len(cross_ref_df)} cross-references (showing max 50)")
        print("\n" + "-" * 60)
        ipython_display(cross_ref_df)
    else:
        print("⚠️ No cross-references found. This may mean:")
        print("   - The creatorGnds column is empty in paintings table")
        print("   - No shared painter GNDs between CbDD and Bildindex")
        
except Exception as e:
    print(f"⚠️ Error in cross-reference query: {e}")

🔗 CROSS-REFERENCE: CbDD Paintings ↔ Bildindex Images
✅ Found 50 cross-references (showing max 50)

------------------------------------------------------------


painting_label,cbdd_painter,building_name,bildindex_title,bildindex_url,collection_tier
str,str,str,str,str,i64
"""Die drei Jünglinge im Feuerofe…","""Aloisi, Andrea""","""Bad Iburg, Schloss Iburg""","""Die Heilige Helena""","""http://www.bildindex.de/docume…",2
"""Die drei Jünglinge im Feuerofe…","""Aloisi, Andrea""","""Bad Iburg, Schloss Iburg""","""Innendekoration des Rittersaal…","""http://www.bildindex.de/docume…",2
"""Die drei Jünglinge im Feuerofe…","""Aloisi, Andrea""","""Bad Iburg, Schloss Iburg""","""Heilige Helena""","""http://www.bildindex.de/docume…",2
"""Die drei Jünglinge im Feuerofe…","""Aloisi, Andrea""","""Bad Iburg, Schloss Iburg""","""Innendekoration des Rittersaal…","""http://www.bildindex.de/docume…",2
"""Die drei Jünglinge im Feuerofe…","""Aloisi, Andrea""","""Bad Iburg, Schloss Iburg""","""Innendekoration des Rittersaal…","""http://www.bildindex.de/docume…",2
…,…,…,…,…,…
"""Gemälde der Emporenbrüstung""","""Appiani, Joseph Ignaz""","""Meersburg, Priesterseminar""","""Neues Schloss""","""http://www.bildindex.de/docume…",2
"""Gemälde der linken Seitenkapel…","""Appiani, Joseph Ignaz""","""Altshausen, Kirche St. Michael""","""Ehemalige Schloßkirche""","""http://www.bildindex.de/docume…",2
"""Gemälde der linken Seitenkapel…","""Appiani, Joseph Ignaz""","""Altshausen, Kirche St. Michael""","""Torbau""","""http://www.bildindex.de/docume…",2


In [32]:
# =============================================================================
# CELL: Cross-Reference: Buildings with Most Bildindex Coverage
# =============================================================================

print("🏛️ BUILDINGS WITH MOST BILDINDEX COVERAGE")
print("=" * 60)

try:
    building_coverage_df = con.execute("""
        WITH cbdd_building_gnds AS (
            SELECT DISTINCT
                p.building_id,
                p.building_name,
                TRIM(gnd.value) as location_gnd,
                COUNT(DISTINCT p.nfdi_uri) as cbdd_paintings
            FROM paintings p,
                 UNNEST(STRING_SPLIT(p.locationGnds, '|')) as gnd(value)
            WHERE p.locationGnds IS NOT NULL AND p.locationGnds != ''
            GROUP BY p.building_id, p.building_name, TRIM(gnd.value)
        )
        SELECT 
            cbg.building_name,
            cbg.cbdd_paintings,
            COUNT(DISTINCT bi.bildindex_uri) as bildindex_images,
            cbg.location_gnd
        FROM cbdd_building_gnds cbg
        JOIN bi_buildings bb ON cbg.location_gnd = bb.building_gnd
        JOIN bi_items bi ON bb.bildindex_uri = bi.bildindex_uri
        GROUP BY cbg.building_name, cbg.cbdd_paintings, cbg.location_gnd
        ORDER BY bildindex_images DESC
        LIMIT 20
    """).pl()
    
    if len(building_coverage_df) > 0:
        print(f"✅ Found {len(building_coverage_df)} buildings with Bildindex coverage:")
        ipython_display(building_coverage_df)
        
        # Visualization
        if len(building_coverage_df) > 0:
            df_viz = building_coverage_df.head(15).to_pandas()
            chart = alt.Chart(df_viz).mark_bar().encode(
                x=alt.X('bildindex_images:Q', title='Bildindex Images'),
                y=alt.Y('building_name:N', sort='-x', title='Building'),
                color=alt.Color('cbdd_paintings:Q', 
                               scale=alt.Scale(scheme='greens'),
                               title='CbDD Paintings'),
                tooltip=['building_name', 'cbdd_paintings', 'bildindex_images']
            ).properties(
                title='Buildings with Bildindex Coverage',
                width=500,
                height=400
            )
            ipython_display(chart)
    else:
        print("⚠️ No building cross-references found.")
        
except Exception as e:
    print(f"⚠️ Error in building coverage query: {e}")

🏛️ BUILDINGS WITH MOST BILDINDEX COVERAGE
✅ Found 20 buildings with Bildindex coverage:


building_name,cbdd_paintings,bildindex_images,location_gnd
str,i64,i64,str
null,4,9,"""https://d-nb.info/gnd/4132604-…"
"""Potsdam, Schloss ""Sanssouci""""",27,9,"""https://d-nb.info/gnd/4132604-…"
"""Ellingen, Landkomturei des Deu…",16,8,"""https://d-nb.info/gnd/4223060-…"
null,2,8,"""https://d-nb.info/gnd/4223060-…"
"""Ponitz, sog. Renaissanceschlos…",1,7,"""https://d-nb.info/gnd/4837686-…"
…,…,…,…
"""Greiz, Oberes Schloss""",6,5,"""https://d-nb.info/gnd/4803528-…"
"""Kannawurf, Gut""",1,5,"""https://d-nb.info/gnd/7759837-…"
"""Bayreuth, Altes Schloss der Er…",7,5,"""https://d-nb.info/gnd/4359350-…"


alt.Chart(...)

In [33]:
# =============================================================================
# CELL: Shared ICONCLASS Subjects Between CbDD and Bildindex
# =============================================================================

print("📜 SHARED ICONCLASS SUBJECTS")
print("=" * 60)

try:
    # Find ICONCLASS codes that appear in both datasets
    shared_subjects_df = con.execute("""
        SELECT 
            s.subject_uri,
            s.subject_label,
            COUNT(DISTINCT ps.nfdi_uri) as cbdd_paintings,
            COUNT(DISTINCT bis.bildindex_uri) as bildindex_items
        FROM subjects s
        JOIN painting_subjects ps ON s.subject_uri = ps.subject_uri
        JOIN bi_subjects bis ON s.subject_uri = bis.iconclass_code
        WHERE s.subject_source = 'ICONCLASS'
        GROUP BY s.subject_uri, s.subject_label
        HAVING cbdd_paintings > 0 AND bildindex_items > 0
        ORDER BY (cbdd_paintings + bildindex_items) DESC
        LIMIT 30
    """).pl()
    
    if len(shared_subjects_df) > 0:
        print(f"✅ Found {len(shared_subjects_df)} shared ICONCLASS subjects:")
        ipython_display(shared_subjects_df)
        
        # Scatter plot: CbDD vs Bildindex coverage per subject
        df_viz = shared_subjects_df.to_pandas()
        chart = alt.Chart(df_viz).mark_circle(size=100).encode(
            x=alt.X('cbdd_paintings:Q', title='CbDD Paintings', scale=alt.Scale(type='log')),
            y=alt.Y('bildindex_items:Q', title='Bildindex Items', scale=alt.Scale(type='log')),
            color=alt.value('#8856a7'),
            tooltip=['subject_label', 'cbdd_paintings', 'bildindex_items']
        ).properties(
            title='Shared ICONCLASS Subjects: CbDD vs Bildindex',
            width=500,
            height=400
        )
        ipython_display(chart)
    else:
        print("⚠️ No shared ICONCLASS subjects found.")
        
except Exception as e:
    print(f"⚠️ Error in shared subjects query: {e}")

📜 SHARED ICONCLASS SUBJECTS
✅ Found 30 shared ICONCLASS subjects:


subject_uri,subject_label,cbdd_paintings,bildindex_items
str,str,i64,i64
"""https://iconclass.org/92D1916""","""cupids: 'amores', 'amoretti', …",331,11
"""https://iconclass.org/26A""","""clouds""",180,2
"""https://iconclass.org/25G3""","""trees""",71,2
"""https://iconclass.org/25G41""","""flowers""",56,3
"""https://iconclass.org/48A9872""","""ornament ~ grotesque""",57,1
…,…,…,…
"""https://iconclass.org/25G21""","""fruits""",20,5
"""https://iconclass.org/11G21""","""angels singing, making music""",22,1
"""https://iconclass.org/41A7751""","""container made of plant materi…",22,1


alt.Chart(...)

# Top 3 Painters with GND Overlap - Comprehensive Biographies

This section identifies the painters with the highest cross-reference counts between CbDD and Bildindex datasets, then builds comprehensive biographical profiles showing:
- **Personal information** (birth, death, GND identifier)
- **All paintings** (from CbDD)
- **All Bildindex images** (historical photographs)
- **Buildings worked in** (from both datasets)
- **Collaborators** (persons they worked with)
- **Timeline visualization**

Data source indicators:
- 🟦 **CbDD** (Ceiling Paintings Database Germany)
- 🟧 **Bildindex** (Historical Image Archive)

In [34]:
# =============================================================================
# CELL: Identify Top 3 Painters with Highest GND Overlap
# =============================================================================
# Find painters that appear in BOTH CbDD and Bildindex datasets via GND matching

print("🎨 TOP 3 PAINTERS WITH HIGHEST CBDD ↔ BILDINDEX GND OVERLAP")
print("=" * 70)

# First, let's examine the relevant table schemas
print("\n📋 Examining table schemas...")
print("\n   painting_persons columns:")
pp_schema = con.execute("DESCRIBE painting_persons").pl()
for row in pp_schema.iter_rows(named=True):
    print(f"      - {row['column_name']}: {row['column_type']}")

print("\n   persons columns:")
p_schema = con.execute("DESCRIBE persons").pl()
for row in p_schema.iter_rows(named=True):
    print(f"      - {row['column_name']}: {row['column_type']}")

print("\n   bi_painters columns:")
bp_schema = con.execute("DESCRIBE bi_painters").pl()
for row in bp_schema.iter_rows(named=True):
    print(f"      - {row['column_name']}: {row['column_type']}")

print("\n" + "=" * 70)

try:
    # The paintings table has creatorGnds column with pipe-separated GND URIs
    # We need to match these with bi_painters.painter_gnd
    # But first, let's verify there's data in creatorGnds
    
    gnd_check = con.execute("""
        SELECT 
            COUNT(*) as total,
            COUNT(CASE WHEN creatorGnds IS NOT NULL AND creatorGnds != '' THEN 1 END) as with_gnd
        FROM paintings
    """).fetchone()
    
    print(f"\n📊 Paintings GND coverage:")
    print(f"   Total paintings: {gnd_check[0]}")
    print(f"   With creatorGnds: {gnd_check[1]}")
    
    if gnd_check[1] == 0:
        print("\n⚠️ No creatorGnds found in paintings table!")
        print("   Falling back to matching via person names...")
        
        # Alternative approach: match by person name
        # First get all Bildindex painter GNDs 
        # Then find matching persons in CbDD by looking at the persons table
        
        top_painters_overlap = con.execute("""
            WITH bildindex_painter_gnds AS (
                -- Get unique painter GNDs from Bildindex
                SELECT DISTINCT painter_gnd
                FROM bi_painters
            ),
            cbdd_persons_with_gnd AS (
                -- Find CbDD persons that might match (we need to extract GND from their NFDI URI or name)
                SELECT 
                    ps.person_id,
                    ps.person_name,
                    pp.nfdi_uri,
                    pp.role
                FROM persons ps
                JOIN painting_persons pp ON ps.person_id = pp.person_id
                WHERE pp.role = 'PAINTER'
            ),
            -- Count how many paintings and bildindex items per painter
            painter_counts AS (
                SELECT 
                    pp.person_id,
                    pp.person_name,
                    COUNT(DISTINCT pp.nfdi_uri) as cbdd_painting_count
                FROM painting_persons pp
                WHERE pp.role = 'PAINTER'
                GROUP BY pp.person_id, pp.person_name
                ORDER BY cbdd_painting_count DESC
                LIMIT 10
            )
            SELECT * FROM painter_counts
        """).pl()
        
        print("\n📋 Top painters by CbDD painting count (no GND matching available):")
        print(top_painters_overlap)
        
        TOP_3_PAINTERS = []
        for row in top_painters_overlap.head(3).iter_rows(named=True):
            TOP_3_PAINTERS.append({
                'person_id': row['person_id'],
                'person_name': row['person_name'],
                'cbdd_painting_count': row['cbdd_painting_count'],
                'bildindex_item_count': 0,  # No GND matching available
                'painter_gnd': None
            })
        
    else:
        # GND-based matching
        top_painters_overlap = con.execute("""
            WITH cbdd_painter_gnds AS (
                -- Get all painter GNDs from CbDD paintings (from creatorGnds column)
                SELECT DISTINCT
                    TRIM(gnd.value) as painter_gnd,
                    p.nfdi_uri,
                    p.label as painting_label,
                    pp.person_name,
                    pp.person_id
                FROM paintings p,
                     UNNEST(STRING_SPLIT(p.creatorGnds, '|')) as gnd(value)
                JOIN painting_persons pp ON p.nfdi_uri = pp.nfdi_uri AND pp.role = 'PAINTER'
                WHERE p.creatorGnds IS NOT NULL AND p.creatorGnds != ''
            ),
            gnd_overlap_counts AS (
                -- Count how many Bildindex items each painter GND has
                SELECT 
                    cpg.painter_gnd,
                    cpg.person_name,
                    cpg.person_id,
                    COUNT(DISTINCT cpg.nfdi_uri) as cbdd_painting_count,
                    COUNT(DISTINCT bp.bildindex_uri) as bildindex_item_count
                FROM cbdd_painter_gnds cpg
                JOIN bi_painters bp ON cpg.painter_gnd = bp.painter_gnd
                GROUP BY cpg.painter_gnd, cpg.person_name, cpg.person_id
            )
            SELECT 
                painter_gnd,
                person_name,
                person_id,
                cbdd_painting_count,
                bildindex_item_count,
                (cbdd_painting_count + bildindex_item_count) as total_items
            FROM gnd_overlap_counts
            ORDER BY total_items DESC
            LIMIT 3
        """).pl()
        
        if len(top_painters_overlap) > 0:
            print(f"\n✅ Found {len(top_painters_overlap)} painters with cross-dataset presence:\n")
            for i, row in enumerate(top_painters_overlap.iter_rows(named=True), 1):
                print(f"  {i}. {row['person_name']}")
                print(f"     GND: {row['painter_gnd']}")
                print(f"     🟦 CbDD Paintings: {row['cbdd_painting_count']}")
                print(f"     🟧 Bildindex Items: {row['bildindex_item_count']}")
                print(f"     Total cross-references: {row['total_items']}")
                print()
            
            # Store for use in subsequent cells
            TOP_3_PAINTERS = top_painters_overlap.to_dicts()
            print("📌 Stored painter data in TOP_3_PAINTERS for biography generation")
            
            # Display as table
            ipython_display(top_painters_overlap)
        else:
            print("⚠️ No painters with GND overlap found between datasets.")
            TOP_3_PAINTERS = []
        
except Exception as e:
    print(f"⚠️ Error finding top painters: {e}")
    import traceback
    traceback.print_exc()
    TOP_3_PAINTERS = []

🎨 TOP 3 PAINTERS WITH HIGHEST CBDD ↔ BILDINDEX GND OVERLAP

📋 Examining table schemas...

   painting_persons columns:
      - nfdi_uri: VARCHAR
      - cbdd_painting_id: VARCHAR
      - person_id: VARCHAR
      - person_name: VARCHAR
      - role: VARCHAR

   persons columns:
      - person_id: VARCHAR
      - name: VARCHAR
      - person_type: VARCHAR
      - val: DOUBLE

   bi_painters columns:
      - bildindex_uri: VARCHAR
      - painter_gnd: VARCHAR


📊 Paintings GND coverage:
   Total paintings: 4594
   With creatorGnds: 2703

✅ Found 3 painters with cross-dataset presence:

  1. Hermann, Franz Georg
     GND: https://d-nb.info/gnd/123546850
     🟦 CbDD Paintings: 51
     🟧 Bildindex Items: 2
     Total cross-references: 53

  2. Lammers, Seivert
     GND: https://d-nb.info/gnd/102109117
     🟦 CbDD Paintings: 40
     🟧 Bildindex Items: 5
     Total cross-references: 45

  3. Harms, Johann Oswald
     GND: https://d-nb.info/gnd/119052520
     🟦 CbDD Paintings: 33
     🟧 Bildind

painter_gnd,person_name,person_id,cbdd_painting_count,bildindex_item_count,total_items
str,str,str,i64,i64,i64
"""https://d-nb.info/gnd/12354685…","""Hermann, Franz Georg""","""20f9622e-6580-44a3-b8fd-1eb3d6…",51,2,53
"""https://d-nb.info/gnd/10210911…","""Lammers, Seivert""","""c4edb090-f559-4970-b9fb-aba18d…",40,5,45
"""https://d-nb.info/gnd/11905252…","""Harms, Johann Oswald""","""5b74e880-6e0d-11ea-a2e8-4df471…",33,1,34


In [35]:
# =============================================================================
# CELL: Comprehensive Painter Biographies with Timeline Visualization
# =============================================================================
# For each top painter, gather ALL available data from both datasets and visualize

import pandas as pd
from datetime import datetime

print("📜 COMPREHENSIVE PAINTER BIOGRAPHIES")
print("=" * 70)

def gather_painter_biography(painter_gnd, painter_name, person_id):
    """
    Gather all available data for a painter from both CbDD and Bildindex.
    Returns a dictionary with all biographical information.
    """
    bio = {
        'name': painter_name,
        'gnd': painter_gnd,
        'person_id': person_id,
        'cbdd': {},
        'bildindex': {},
        'timeline_events': []
    }
    
    # ===========================================
    # 1. CbDD DATA - Personal Information from persons table
    # ===========================================
    try:
        person_info = con.execute("""
            SELECT 
                person_id,
                name,
                person_type,
                val
            FROM persons
            WHERE person_id = ?
        """, [person_id]).pl()
        
        if len(person_info) > 0:
            info = person_info.row(0, named=True)
            bio['cbdd']['person_type'] = info.get('person_type')
            bio['cbdd']['full_name'] = info.get('name')
    except Exception as e:
        print(f"  ⚠️ Error fetching person info: {e}")
    
    # ===========================================
    # 2. CbDD DATA - All Paintings (using actual column names)
    # ===========================================
    try:
        paintings = con.execute("""
            SELECT DISTINCT
                p.nfdi_uri,
                p.label as painting_label,
                p.year,
                p.year_start,
                p.year_end,
                p.year_is_approximate,
                p.building_name,
                p.building_id,
                p.room_name,
                p.location_state,
                p.painters,
                p.imageUrl as image_url
            FROM paintings p
            JOIN painting_persons pp ON p.nfdi_uri = pp.nfdi_uri
            WHERE pp.person_id = ? AND pp.role = 'PAINTER'
            ORDER BY p.year_start
        """, [person_id]).pl()
        
        bio['cbdd']['paintings'] = paintings.to_dicts()
        bio['cbdd']['painting_count'] = len(paintings)
        
        # Calculate active period from painting years
        years = [p.get('year_start') for p in bio['cbdd']['paintings'] if p.get('year_start')]
        if years:
            bio['cbdd']['active_from'] = min(years)
            end_years = [p.get('year_end') or p.get('year_start') for p in bio['cbdd']['paintings'] if p.get('year_start')]
            bio['cbdd']['active_until'] = max(end_years) if end_years else max(years)
        
        # Add painting events to timeline
        for row in paintings.iter_rows(named=True):
            if row.get('year_start'):
                approx = "~" if row.get('year_is_approximate') else ""
                bio['timeline_events'].append({
                    'year': row['year_start'],
                    'event': f"{approx}Painted: {row['painting_label'][:50]}{'...' if len(row['painting_label']) > 50 else ''}",
                    'type': 'painting',
                    'source': 'CbDD',
                    'location': row.get('building_name', ''),
                    'full_label': row['painting_label']
                })
    except Exception as e:
        print(f"  ⚠️ Error fetching paintings: {e}")
        bio['cbdd']['paintings'] = []
        bio['cbdd']['painting_count'] = 0
    
    # ===========================================
    # 3. CbDD DATA - All Buildings (using actual columns)
    # ===========================================
    try:
        buildings = con.execute("""
            SELECT DISTINCT
                b.building_id,
                b.name as building_name,
                b.function as building_function,
                b.location_state
            FROM buildings b
            JOIN paintings p ON b.building_id = p.building_id
            JOIN painting_persons pp ON p.nfdi_uri = pp.nfdi_uri
            WHERE pp.person_id = ? AND pp.role = 'PAINTER'
        """, [person_id]).pl()
        
        bio['cbdd']['buildings'] = buildings.to_dicts()
        bio['cbdd']['building_count'] = len(buildings)
    except Exception as e:
        print(f"  ⚠️ Error fetching buildings: {e}")
        bio['cbdd']['buildings'] = []
        bio['cbdd']['building_count'] = 0
    
    # ===========================================
    # 4. CbDD DATA - Collaborators (other persons worked with)
    # ===========================================
    try:
        collaborators = con.execute("""
            WITH painter_works AS (
                SELECT DISTINCT p.nfdi_uri
                FROM paintings p
                JOIN painting_persons pp ON p.nfdi_uri = pp.nfdi_uri
                WHERE pp.person_id = ? AND pp.role = 'PAINTER'
            )
            SELECT DISTINCT
                pp2.person_name,
                pp2.person_id,
                pp2.role,
                COUNT(DISTINCT pp2.nfdi_uri) as shared_works
            FROM painter_works pw
            JOIN painting_persons pp2 ON pw.nfdi_uri = pp2.nfdi_uri
            WHERE pp2.person_id != ?
            GROUP BY pp2.person_name, pp2.person_id, pp2.role
            ORDER BY shared_works DESC, pp2.role
        """, [person_id, person_id]).pl()
        
        bio['cbdd']['collaborators'] = collaborators.to_dicts()
        bio['cbdd']['collaborator_count'] = len(collaborators)
    except Exception as e:
        print(f"  ⚠️ Error fetching collaborators: {e}")
        bio['cbdd']['collaborators'] = []
        bio['cbdd']['collaborator_count'] = 0
    
    # ===========================================
    # 5. CbDD DATA - ICONCLASS Subjects
    # ===========================================
    try:
        subjects = con.execute("""
            SELECT DISTINCT
                s.subject_uri,
                s.subject_label,
                s.subject_source,
                COUNT(DISTINCT ps.nfdi_uri) as usage_count
            FROM subjects s
            JOIN painting_subjects ps ON s.subject_uri = ps.subject_uri
            JOIN painting_persons pp ON ps.nfdi_uri = pp.nfdi_uri
            WHERE pp.person_id = ? AND pp.role = 'PAINTER'
              AND s.subject_source = 'ICONCLASS'
            GROUP BY s.subject_uri, s.subject_label, s.subject_source
            ORDER BY usage_count DESC
            LIMIT 20
        """, [person_id]).pl()
        
        bio['cbdd']['subjects'] = subjects.to_dicts()
    except Exception as e:
        print(f"  ⚠️ Error fetching subjects: {e}")
        bio['cbdd']['subjects'] = []
    
    # ===========================================
    # 6. BILDINDEX DATA - All Images
    # ===========================================
    try:
        if painter_gnd:
            bildindex_items = con.execute("""
                SELECT DISTINCT
                    bi.bildindex_uri,
                    bi.bildindex_url,
                    bi.title,
                    bi.tier,
                    bi.collection_method
                FROM bi_items bi
                JOIN bi_painters bp ON bi.bildindex_uri = bp.bildindex_uri
                WHERE bp.painter_gnd = ?
                ORDER BY bi.title
            """, [painter_gnd]).pl()
            
            bio['bildindex']['items'] = bildindex_items.to_dicts()
            bio['bildindex']['item_count'] = len(bildindex_items)
            
            # Add Bildindex items to timeline (without year)
            for row in bildindex_items.iter_rows(named=True):
                bio['timeline_events'].append({
                    'year': None,
                    'event': f"Bildindex: {row['title'][:50] if row.get('title') else 'Untitled'}",
                    'type': 'bildindex_photo',
                    'source': 'Bildindex',
                    'url': row.get('bildindex_url')
                })
        else:
            bio['bildindex']['items'] = []
            bio['bildindex']['item_count'] = 0
    except Exception as e:
        print(f"  ⚠️ Error fetching Bildindex items: {e}")
        bio['bildindex']['items'] = []
        bio['bildindex']['item_count'] = 0
    
    # ===========================================
    # 7. BILDINDEX DATA - Buildings mentioned
    # ===========================================
    try:
        if painter_gnd:
            bi_buildings = con.execute("""
                SELECT DISTINCT
                    bb.building_gnd,
                    COUNT(DISTINCT bb.bildindex_uri) as image_count
                FROM bi_buildings bb
                JOIN bi_painters bp ON bb.bildindex_uri = bp.bildindex_uri
                WHERE bp.painter_gnd = ?
                GROUP BY bb.building_gnd
            """, [painter_gnd]).pl()
            
            bio['bildindex']['buildings'] = bi_buildings.to_dicts()
            bio['bildindex']['building_count'] = len(bi_buildings)
        else:
            bio['bildindex']['buildings'] = []
            bio['bildindex']['building_count'] = 0
    except Exception as e:
        print(f"  ⚠️ Error fetching Bildindex buildings: {e}")
        bio['bildindex']['buildings'] = []
        bio['bildindex']['building_count'] = 0
    
    # ===========================================
    # 8. BILDINDEX DATA - Subjects
    # ===========================================
    try:
        if painter_gnd:
            bi_subjects = con.execute("""
                SELECT DISTINCT
                    bis.iconclass_code,
                    COUNT(DISTINCT bis.bildindex_uri) as usage_count
                FROM bi_subjects bis
                JOIN bi_painters bp ON bis.bildindex_uri = bp.bildindex_uri
                WHERE bp.painter_gnd = ?
                GROUP BY bis.iconclass_code
                ORDER BY usage_count DESC
                LIMIT 20
            """, [painter_gnd]).pl()
            
            bio['bildindex']['subjects'] = bi_subjects.to_dicts()
        else:
            bio['bildindex']['subjects'] = []
    except Exception as e:
        print(f"  ⚠️ Error fetching Bildindex subjects: {e}")
        bio['bildindex']['subjects'] = []
    
    return bio

# Gather biographies for all top painters
PAINTER_BIOGRAPHIES = []

if 'TOP_3_PAINTERS' in dir() and TOP_3_PAINTERS:
    for i, painter in enumerate(TOP_3_PAINTERS, 1):
        print(f"\n{'='*70}")
        print(f"📖 Gathering biography for Painter {i}: {painter['person_name']}")
        print(f"{'='*70}")
        
        bio = gather_painter_biography(
            painter.get('painter_gnd'),
            painter['person_name'],
            painter['person_id']
        )
        PAINTER_BIOGRAPHIES.append(bio)
        
        # Print summary
        print(f"\n  📊 Data Summary for {bio['name']}:")
        print(f"     GND: {bio['gnd']}")
        print(f"     🟦 CbDD Data:")
        print(f"        - Active period: {bio['cbdd'].get('active_from', '?')} - {bio['cbdd'].get('active_until', '?')}")
        print(f"        - Paintings: {bio['cbdd'].get('painting_count', 0)}")
        print(f"        - Buildings: {bio['cbdd'].get('building_count', 0)}")
        print(f"        - Collaborators: {bio['cbdd'].get('collaborator_count', 0)}")
        print(f"        - ICONCLASS subjects: {len(bio['cbdd'].get('subjects', []))}")
        print(f"     🟧 Bildindex Data:")
        print(f"        - Historical images: {bio['bildindex'].get('item_count', 0)}")
        print(f"        - Buildings referenced: {bio['bildindex'].get('building_count', 0)}")
        print(f"        - ICONCLASS subjects: {len(bio['bildindex'].get('subjects', []))}")
        print(f"     📅 Timeline events: {len(bio['timeline_events'])}")
else:
    print("⚠️ TOP_3_PAINTERS not available. Run the previous cell first.")

print(f"\n✅ Collected biographies for {len(PAINTER_BIOGRAPHIES)} painters")

📜 COMPREHENSIVE PAINTER BIOGRAPHIES

📖 Gathering biography for Painter 1: Hermann, Franz Georg

  📊 Data Summary for Hermann, Franz Georg:
     GND: https://d-nb.info/gnd/123546850
     🟦 CbDD Data:
        - Active period: 1729 - 1742
        - Paintings: 54
        - Buildings: 2
        - Collaborators: 3
        - ICONCLASS subjects: 20
     🟧 Bildindex Data:
        - Historical images: 2
        - Buildings referenced: 0
        - ICONCLASS subjects: 2
     📅 Timeline events: 56

📖 Gathering biography for Painter 2: Lammers, Seivert

  📊 Data Summary for Lammers, Seivert:
     GND: https://d-nb.info/gnd/102109117
     🟦 CbDD Data:
        - Active period: 1670 - 1703
        - Paintings: 80
        - Buildings: 4
        - Collaborators: 14
        - ICONCLASS subjects: 20
     🟧 Bildindex Data:
        - Historical images: 5
        - Buildings referenced: 0
        - ICONCLASS subjects: 1
     📅 Timeline events: 84

📖 Gathering biography for Painter 3: Harms, Johann Oswald

  📊

In [36]:
# =============================================================================
# CELL: Visualize Painter Biographies - Timeline & Summary Cards
# =============================================================================
# Create comprehensive visual biographies for each painter

from IPython.display import HTML, display

def create_biography_html(bio, index):
    """Create an HTML card showing painter biography with data source indicators."""
    
    name = bio['name']
    gnd = bio['gnd'] or 'Unknown'
    person_id = bio.get('person_id', 'Unknown')
    
    # Active period (derived from paintings)
    active_from = bio['cbdd'].get('active_from', '?')
    active_until = bio['cbdd'].get('active_until', '?')
    active_period = f"{active_from} - {active_until}" if active_from != '?' else "Unknown"
    
    # Build HTML
    html = f"""
    <div style="border: 2px solid #333; border-radius: 10px; padding: 20px; margin: 20px 0; 
                background: linear-gradient(135deg, #f5f7fa 0%, #e4e8eb 100%); color: black;">
        <h2 style="margin-top: 0; color: #2c3e50; border-bottom: 2px solid #3498db; padding-bottom: 10px;">
            🎨 Painter {index}: {name}
        </h2>
        
        <div style="display: grid; grid-template-columns: 1fr 1fr; gap: 20px;">
            <!-- Left Column: Personal Info -->
            <div>
                <h3 style="color: #2980b9;">📋 Personal Information</h3>
                <table style="width: 100%; border-collapse: collapse;">
                    <tr><td style="padding: 5px; font-weight: bold;">Active Period:</td><td>{active_period}</td></tr>
                    <tr><td style="padding: 5px; font-weight: bold;">GND:</td>
                        <td><a href="{gnd}" target="_blank">{gnd.split('/')[-1] if gnd != 'Unknown' else 'N/A'}</a></td></tr>
                    <tr><td style="padding: 5px; font-weight: bold;">Person ID:</td>
                        <td style="font-size: 0.8em; word-break: break-all;">{person_id[:20]}...</td></tr>
                </table>
                
                <h3 style="color: #2980b9; margin-top: 15px;">📊 Data Summary</h3>
                <table style="width: 100%; border-collapse: collapse;">
                    <tr style="background: #3498db22;">
                        <td style="padding: 8px; font-weight: bold;">Source</td>
                        <td style="padding: 8px; font-weight: bold;">Category</td>
                        <td style="padding: 8px; font-weight: bold; text-align: right;">Count</td>
                    </tr>
                    <tr><td style="padding: 5px;">🟦 CbDD</td><td>Paintings</td>
                        <td style="text-align: right; font-weight: bold;">{bio['cbdd'].get('painting_count', 0)}</td></tr>
                    <tr><td style="padding: 5px;">🟦 CbDD</td><td>Buildings</td>
                        <td style="text-align: right;">{bio['cbdd'].get('building_count', 0)}</td></tr>
                    <tr><td style="padding: 5px;">🟦 CbDD</td><td>Collaborators</td>
                        <td style="text-align: right;">{bio['cbdd'].get('collaborator_count', 0)}</td></tr>
                    <tr style="background: #e67e2222;">
                        <td style="padding: 5px;">🟧 Bildindex</td><td>Historical Images</td>
                        <td style="text-align: right; font-weight: bold;">{bio['bildindex'].get('item_count', 0)}</td></tr>
                    <tr style="background: #e67e2222;">
                        <td style="padding: 5px;">🟧 Bildindex</td><td>Buildings</td>
                        <td style="text-align: right;">{bio['bildindex'].get('building_count', 0)}</td></tr>
                </table>
            </div>
            
            <!-- Right Column: Top Subjects -->
            <div>
                <h3 style="color: #27ae60;">📚 Top ICONCLASS Subjects (CbDD)</h3>
                <ul style="margin: 0; padding-left: 20px; max-height: 150px; overflow-y: auto;">
    """
    
    for subj in bio['cbdd'].get('subjects', [])[:8]:
        html += f"<li><small>{subj['subject_label']} ({subj['usage_count']}x)</small></li>"
    
    if not bio['cbdd'].get('subjects'):
        html += "<li><em>No ICONCLASS subjects found</em></li>"
    
    html += """
                </ul>
                
                <h3 style="color: #e67e22; margin-top: 15px;">📚 Bildindex ICONCLASS</h3>
                <ul style="margin: 0; padding-left: 20px; max-height: 100px; overflow-y: auto;">
    """
    
    for subj in bio['bildindex'].get('subjects', [])[:5]:
        html += f"<li><small>{subj['iconclass_code']} ({subj['usage_count']}x)</small></li>"
    
    if not bio['bildindex'].get('subjects'):
        html += "<li><em>No ICONCLASS subjects found</em></li>"
    
    html += """
                </ul>
            </div>
        </div>
    </div>
    """
    
    return html

def create_paintings_table(bio):
    """Create HTML table of all paintings."""
    paintings = bio['cbdd'].get('paintings', [])
    if not paintings:
        return "<p><em>No paintings found in CbDD</em></p>"
    
    html = """
    <div style="margin: 10px 0;">
        <h4 style="color: #3498db;">🟦 CbDD Paintings ({} total)</h4>
        <div style="max-height: 300px; overflow-y: auto; border: 1px solid #ddd; border-radius: 5px;">
            <table style="width: 100%; border-collapse: collapse; font-size: 0.85em;">
                <tr style="background: #3498db; color: white; position: sticky; top: 0;">
                    <th style="padding: 8px; text-align: left;">Year</th>
                    <th style="padding: 8px; text-align: left;">Painting</th>
                    <th style="padding: 8px; text-align: left;">Building</th>
                    <th style="padding: 8px; text-align: left;">Location</th>
                </tr>
    """.format(len(paintings))
    
    for i, p in enumerate(paintings):
        bg = "#f8f9fa" if i % 2 == 0 else "#ffffff"
        year = p.get('year', '?')
        approx = "~" if p.get('year_is_approximate') else ""
        html += f"""
            <tr style="background: {bg}; color: black;">
                <td style="padding: 6px;">{approx}{year}</td>
                <td style="padding: 6px;">{p.get('painting_label', 'Untitled')[:60]}{'...' if len(p.get('painting_label', '')) > 60 else ''}</td>
                <td style="padding: 6px;">{p.get('building_name', '')[:30]}</td>
                <td style="padding: 6px;">{p.get('location_city', '')}, {p.get('location_state', '')}</td>
            </tr>
        """
    
    html += "</table></div></div>"
    return html

def create_bildindex_table(bio):
    """Create HTML table of all Bildindex items."""
    items = bio['bildindex'].get('items', [])
    if not items:
        return "<p><em>No items found in Bildindex</em></p>"
    
    html = """
    <div style="margin: 10px 0;">
        <h4 style="color: #e67e22;">🟧 Bildindex Historical Images ({} total)</h4>
        <div style="max-height: 250px; overflow-y: auto; border: 1px solid #ddd; border-radius: 5px;">
            <table style="width: 100%; border-collapse: collapse; font-size: 0.85em;">
                <tr style="background: #e67e22; color: white; position: sticky; top: 0;">
                    <th style="padding: 8px; text-align: left;">Title</th>
                    <th style="padding: 8px; text-align: left;">Tier</th>
                    <th style="padding: 8px; text-align: left;">Link</th>
                </tr>
    """.format(len(items))
    
    for i, item in enumerate(items):
        bg = "#fef5e7" if i % 2 == 0 else "#ffffff"
        title = item.get('title', 'Untitled') or 'Untitled'
        url = item.get('bildindex_url', '')
        html += f"""
            <tr style="background: {bg}; color: black;">
                <td style="padding: 6px;">{title[:70]}{'...' if len(title) > 70 else ''}</td>
                <td style="padding: 6px;">{item.get('tier', '?')}</td>
                <td style="padding: 6px;"><a href="{url}" target="_blank">🔗 View</a></td>
            </tr>
        """
    
    html += "</table></div></div>"
    return html

def create_collaborators_table(bio):
    """Create HTML table of collaborators."""
    collabs = bio['cbdd'].get('collaborators', [])
    if not collabs:
        return "<p><em>No collaborators found</em></p>"
    
    html = """
    <div style="margin: 10px 0;">
        <h4 style="color: #9b59b6;">👥 Collaborators ({} persons)</h4>
        <div style="max-height: 200px; overflow-y: auto; border: 1px solid #ddd; border-radius: 5px;">
            <table style="width: 100%; border-collapse: collapse; font-size: 0.85em;">
                <tr style="background: #9b59b6; color: white;">
                    <th style="padding: 8px; text-align: left;">Person</th>
                    <th style="padding: 8px; text-align: left;">Role</th>
                    <th style="padding: 8px; text-align: right;">Shared Works</th>
                </tr>
    """.format(len(collabs))
    
    for i, c in enumerate(collabs[:15]):  # Show top 15
        bg = "#f5f0f7" if i % 2 == 0 else "#ffffff"
        html += f"""
            <tr style="background: {bg}; color: black;">
                <td style="padding: 6px;">{c.get('person_name', 'Unknown')}</td>
                <td style="padding: 6px;">{c.get('role', '?')}</td>
                <td style="padding: 6px; text-align: right;">{c.get('shared_works', 0)}</td>
            </tr>
        """
    
    if len(collabs) > 15:
        html += f"<tr><td colspan='3' style='padding: 6px; text-align: center;'><em>... and {len(collabs) - 15} more</em></td></tr>"
    
    html += "</table></div></div>"
    return html

def create_buildings_table(bio):
    """Create HTML table of buildings from both sources."""
    cbdd_buildings = bio['cbdd'].get('buildings', [])
    bi_buildings = bio['bildindex'].get('buildings', [])
    
    html = """
    <div style="margin: 10px 0;">
        <h4 style="color: #16a085;">🏛️ Buildings</h4>
        <div style="display: grid; grid-template-columns: 1fr 1fr; gap: 10px;">
    """
    
    # CbDD Buildings
    html += """
        <div style="border: 1px solid #3498db; border-radius: 5px; padding: 10px; background: #3498db; color: white;">
            <h5 style="color: white; margin-top: 0;">🟦 CbDD Buildings ({})</h5>
            <ul style="margin: 0; padding-left: 20px; max-height: 150px; overflow-y: auto; font-size: 0.85em; color: white;">
    """.format(len(cbdd_buildings))
    
    for b in cbdd_buildings:
        html += f"<li>{b.get('building_name', 'Unknown')} ({b.get('location_city', '')}) - <em>{b.get('building_function', '')}</em></li>"
    
    if not cbdd_buildings:
        html += "<li><em>No buildings found</em></li>"
    
    html += "</ul></div>"
    
    # Bildindex Buildings (just GNDs)
    html += """
        <div style="border: 1px solid #e67e22; border-radius: 5px; padding: 10px; background: #e67e22; color: white;">
            <h5 style="color: white; margin-top: 0;">🟧 Bildindex Building GNDs ({})</h5>
            <ul style="margin: 0; padding-left: 20px; max-height: 150px; overflow-y: auto; font-size: 0.85em; color: white;">
    """.format(len(bi_buildings))
    
    for b in bi_buildings:
        gnd = b.get('building_gnd', '')
        html += f"<li>{gnd} ({b.get('image_count', 0)} images)</li>"
    
    if not bi_buildings:
        html += "<li><em>No building references found</em></li>"
    
    html += "</ul></div></div></div>"
    return html

# ===========================================
# Display Biographies
# ===========================================
print("📜 DISPLAYING COMPREHENSIVE BIOGRAPHIES")
print("=" * 70)

if 'PAINTER_BIOGRAPHIES' in dir() and PAINTER_BIOGRAPHIES:
    for i, bio in enumerate(PAINTER_BIOGRAPHIES, 1):
        # Summary Card
        display(HTML(create_biography_html(bio, i)))
        
        # Detailed Tables
        detail_html = f"""
        <div style="border: 1px solid #ccc; border-radius: 5px; padding: 15px; margin: 10px 0 30px 0; color: black;">
            <h3 style="margin-top: 0; color: black;">📋 Detailed Data for {bio['name']}</h3>
            <div style="display: grid; grid-template-columns: 1fr 1fr; gap: 15px;">
                <div>{create_paintings_table(bio)}</div>
                <div>{create_bildindex_table(bio)}</div>
            </div>
            <div style="display: grid; grid-template-columns: 1fr 1fr; gap: 15px; margin-top: 15px;">
                <div>{create_collaborators_table(bio)}</div>
                <div>{create_buildings_table(bio)}</div>
            </div>
        </div>
        """
        display(HTML(detail_html))
else:
    print("⚠️ No biographies available. Run the previous cells first.")

📜 DISPLAYING COMPREHENSIVE BIOGRAPHIES


Year,Painting,Building,Location
1729,Deckengemälde: Verbannung der Laster durch die Kardinaltugen...,"Wolfegg, Schloss",", Baden-Württemberg"
1729,Möglicher Ort des Tondos Christus in Emmaus (Hermann 1729),"Wolfegg, Schloss",", Baden-Württemberg"
1729,Minerva überreicht Ganymed den vom Haus Waldburg-Wolfegg beh...,"Wolfegg, Schloss",", Baden-Württemberg"
~um 1732–1742,Jakobs Kampf mit dem Engel,"Kempten, fürstbischöfliche Res",", Bayern"
~um 1732–1742,Der Tod Joachims,"Kempten, fürstbischöfliche Res",", Bayern"
~um 1732–1742,Jakobs Begegnung mit dem Engel,"Kempten, fürstbischöfliche Res",", Bayern"
~um 1732–1742,Eine Mondlandschaft,"Kempten, fürstbischöfliche Res",", Bayern"
~um 1732–1742,Ein sitzender Putto,"Kempten, fürstbischöfliche Res",", Bayern"
~um 1732–1742,Der Tod des Hl. Benedikts,"Kempten, fürstbischöfliche Res",", Bayern"
~um 1732–1742,Die Verherrlichung des Fürststifts,"Kempten, fürstbischöfliche Res",", Bayern"


Year,Painting,Building,Location
1670-1680,Joseph wird vom Pharao erhoben,"Rudolstadt, Residenzschloss He",", Thüringen"
1670-1680,Joseph und das Weib des Potiphar,"Rudolstadt, Residenzschloss He",", Thüringen"
1670-1680,Joseph deutet den Traum des Mundschenks und den Traum des Bä...,"Rudolstadt, Residenzschloss He",", Thüringen"
1688,Die Wandmalerei des Fürstenzimmers,"Leutenberg, ehem. Schloss ""Fri",", Thüringen"
1690-1703,Die Bildern in den Kassetten - Meleager und Atalante auf der...,"Sondershausen, Residenzschloss",", Thüringen"
1690-1703,Die Bilder in den Kassetten - Cephalus und Procris,"Sondershausen, Residenzschloss",", Thüringen"
1690-1703,Tiberius,"Sondershausen, Residenzschloss",", Thüringen"
1690-1703,Titus,"Sondershausen, Residenzschloss",", Thüringen"
1690-1703,Die Bilder in den Kassetten - Aesacus und Hesperie,"Sondershausen, Residenzschloss",", Thüringen"
1690-1703,Die Malerei in der Voute - Ermordung Caesars,"Sondershausen, Residenzschloss",", Thüringen"


Year,Painting,Building,Location
1682,„REMISSA.COMMISSA“,"Weißenfels, Schloss Neu-August",", Sachsen-Anhalt"
1682,„LUSTRANDO.LUSTRAT.OMNI“,"Weißenfels, Schloss Neu-August",", Sachsen-Anhalt"
1682,Engel mit Fanfare,"Weißenfels, Schloss Neu-August",", Sachsen-Anhalt"
1682,Engel mit Notenblatt,"Weißenfels, Schloss Neu-August",", Sachsen-Anhalt"
1682,Das erste Hauptbild - Die Entführung der Europa,"Eisenberg, Residenzschloss Chr",", Thüringen"
1682,Engel mit Harfe,"Weißenfels, Schloss Neu-August",", Sachsen-Anhalt"
1682,Beschneidung Isaaks,"Weißenfels, Schloss Neu-August",", Sachsen-Anhalt"
1682,David überschreitet den Bach Kidron,"Weißenfels, Schloss Neu-August",", Sachsen-Anhalt"
1682,Engel mit Flöte,"Weißenfels, Schloss Neu-August",", Sachsen-Anhalt"
1682,Engel mit Kesselpauken,"Weißenfels, Schloss Neu-August",", Sachsen-Anhalt"


In [37]:
# =============================================================================
# CELL: Timeline Visualization for Each Painter
# =============================================================================
# Create visual timelines showing when each painter created works

import pandas as pd

print("📅 PAINTER CAREER TIMELINES")
print("=" * 70)

def create_timeline_chart(bio):
    """Create an Altair timeline visualization for a painter."""
    
    name = bio['name']
    events = bio['timeline_events']
    
    # Filter to events with years (exclude Bildindex items without dates)
    dated_events = [e for e in events if e.get('year') is not None]
    
    if not dated_events:
        return None, "No dated events available"
    
    # Create DataFrame
    timeline_df = pd.DataFrame(dated_events)
    
    # Define colors by type and source
    color_map = {
        'life': '#2c3e50',      # Dark blue-gray for birth/death
        'painting': '#3498db',   # Blue for CbDD paintings
        'bildindex_photo': '#e67e22'  # Orange for Bildindex
    }
    
    # Create a source column for coloring
    timeline_df['color_key'] = timeline_df['type']
    
    # Add jitter for overlapping years
    year_counts = timeline_df.groupby('year').cumcount()
    timeline_df['y_offset'] = year_counts * 0.3
    
    # Get life span for chart bounds
    birth_year = bio['cbdd'].get('birth_year')
    death_year = bio['cbdd'].get('death_year')
    
    min_year = timeline_df['year'].min() - 5
    max_year = timeline_df['year'].max() + 5
    
    if birth_year and birth_year < min_year:
        min_year = birth_year - 5
    if death_year and death_year > max_year:
        max_year = death_year + 5
    
    # Main scatter plot
    points = alt.Chart(timeline_df).mark_circle(size=100).encode(
        x=alt.X('year:Q', 
                scale=alt.Scale(domain=[min_year, max_year]),
                title='Year'),
        y=alt.Y('y_offset:Q', 
                axis=None,
                scale=alt.Scale(domain=[-0.5, max(5, timeline_df['y_offset'].max() + 1)])),
        color=alt.Color('type:N',
                       scale=alt.Scale(
                           domain=['life', 'painting'],
                           range=['#2c3e50', '#3498db']
                       ),
                       title='Event Type'),
        tooltip=[
            alt.Tooltip('year:Q', title='Year'),
            alt.Tooltip('event:N', title='Event'),
            alt.Tooltip('source:N', title='Source'),
            alt.Tooltip('location:N', title='Location')
        ]
    )
    
    # Add life span bar if we have birth/death
    if birth_year and death_year:
        life_df = pd.DataFrame([{
            'start': birth_year,
            'end': death_year,
            'y': -0.3
        }])
        
        life_bar = alt.Chart(life_df).mark_rule(
            color='#95a5a6',
            strokeWidth=8,
            opacity=0.5
        ).encode(
            x='start:Q',
            x2='end:Q',
            y='y:Q'
        )
        
        chart = alt.layer(life_bar, points)
    else:
        chart = points
    
    # Final chart config
    chart = chart.properties(
        title=f"Career Timeline: {name}",
        width=700,
        height=200
    ).configure_view(
        strokeWidth=0
    )
    
    return chart, None

def create_combined_timeline(biographies):
    """Create a combined timeline showing all three painters."""
    
    all_events = []
    for bio in biographies:
        dated_events = [e for e in bio['timeline_events'] if e.get('year') is not None]
        for e in dated_events:
            e['painter'] = bio['name']
        all_events.extend(dated_events)
    
    if not all_events:
        return None
    
    df = pd.DataFrame(all_events)
    
    chart = alt.Chart(df).mark_circle(size=60, opacity=0.7).encode(
        x=alt.X('year:Q', title='Year'),
        y=alt.Y('painter:N', title='Painter'),
        color=alt.Color('type:N',
                       scale=alt.Scale(
                           domain=['life', 'painting'],
                           range=['#e74c3c', '#3498db']
                       ),
                       title='Event Type'),
        tooltip=[
            alt.Tooltip('painter:N', title='Painter'),
            alt.Tooltip('year:Q', title='Year'),
            alt.Tooltip('event:N', title='Event'),
            alt.Tooltip('source:N', title='Source')
        ]
    ).properties(
        title='Combined Career Timelines: Top 3 Painters with GND Overlap',
        width=800,
        height=200
    )
    
    return chart

# ===========================================
# Generate Visualizations
# ===========================================

if 'PAINTER_BIOGRAPHIES' in dir() and PAINTER_BIOGRAPHIES:
    # Combined timeline
    print("\n📊 Combined Timeline (all 3 painters):")
    combined_chart = create_combined_timeline(PAINTER_BIOGRAPHIES)
    if combined_chart:
        ipython_display(combined_chart)
    else:
        print("  ⚠️ Not enough data for combined timeline")
    
    # Individual timelines
    print("\n" + "=" * 70)
    print("📊 Individual Timelines:")
    
    for i, bio in enumerate(PAINTER_BIOGRAPHIES, 1):
        print(f"\n  {i}. {bio['name']}")
        chart, error = create_timeline_chart(bio)
        if chart:
            ipython_display(chart)
        else:
            print(f"     ⚠️ {error}")
else:
    print("⚠️ No biographies available. Run the previous cells first.")

📅 PAINTER CAREER TIMELINES

📊 Combined Timeline (all 3 painters):


alt.Chart(...)


📊 Individual Timelines:

  1. Hermann, Franz Georg


alt.Chart(...)


  2. Lammers, Seivert


alt.Chart(...)


  3. Harms, Johann Oswald


alt.Chart(...)

In [38]:
# =============================================================================
# CELL: Data Coverage Comparison - CbDD vs Bildindex
# =============================================================================
# Visual comparison of what data each source provides for the top painters

import pandas as pd

print("📊 DATA COVERAGE COMPARISON: CbDD vs Bildindex")
print("=" * 70)

if 'PAINTER_BIOGRAPHIES' in dir() and PAINTER_BIOGRAPHIES:
    # Build comparison data
    comparison_data = []
    
    for bio in PAINTER_BIOGRAPHIES:
        name = bio['name']
        
        # CbDD metrics
        comparison_data.append({
            'painter': name,
            'source': '🟦 CbDD',
            'metric': 'Artworks',
            'count': bio['cbdd'].get('painting_count', 0)
        })
        comparison_data.append({
            'painter': name,
            'source': '🟦 CbDD',
            'metric': 'Buildings',
            'count': bio['cbdd'].get('building_count', 0)
        })
        comparison_data.append({
            'painter': name,
            'source': '🟦 CbDD',
            'metric': 'Collaborators',
            'count': bio['cbdd'].get('collaborator_count', 0)
        })
        comparison_data.append({
            'painter': name,
            'source': '🟦 CbDD',
            'metric': 'Subjects',
            'count': len(bio['cbdd'].get('subjects', []))
        })
        
        # Bildindex metrics
        comparison_data.append({
            'painter': name,
            'source': '🟧 Bildindex',
            'metric': 'Artworks',
            'count': bio['bildindex'].get('item_count', 0)
        })
        comparison_data.append({
            'painter': name,
            'source': '🟧 Bildindex',
            'metric': 'Buildings',
            'count': bio['bildindex'].get('building_count', 0)
        })
        comparison_data.append({
            'painter': name,
            'source': '🟧 Bildindex',
            'metric': 'Collaborators',
            'count': 0  # Not available in Bildindex
        })
        comparison_data.append({
            'painter': name,
            'source': '🟧 Bildindex',
            'metric': 'Subjects',
            'count': len(bio['bildindex'].get('subjects', []))
        })
    
    df = pd.DataFrame(comparison_data)
    
    # Create grouped bar chart
    chart = alt.Chart(df).mark_bar().encode(
        x=alt.X('metric:N', title='Data Category', axis=alt.Axis(labelAngle=0)),
        y=alt.Y('count:Q', title='Count'),
        color=alt.Color('source:N', 
                       scale=alt.Scale(
                           domain=['🟦 CbDD', '🟧 Bildindex'],
                           range=['#3498db', '#e67e22']
                       ),
                       title='Data Source'),
        xOffset='source:N',
        tooltip=['painter', 'source', 'metric', 'count']
    ).properties(
        width=150,
        height=200
    ).facet(
        column=alt.Column('painter:N', title='Painter', header=alt.Header(labelAngle=0))
    ).properties(
        title='Data Coverage Comparison: CbDD vs Bildindex by Painter'
    )
    
    ipython_display(chart)
    
    # Summary statistics
    print("\n📋 Summary Statistics:")
    print("-" * 50)
    
    total_cbdd = sum(bio['cbdd'].get('painting_count', 0) for bio in PAINTER_BIOGRAPHIES)
    total_bi = sum(bio['bildindex'].get('item_count', 0) for bio in PAINTER_BIOGRAPHIES)
    
    print(f"   Total CbDD paintings across 3 painters:    {total_cbdd}")
    print(f"   Total Bildindex items across 3 painters:   {total_bi}")
    print(f"   Combined cross-referenced items:           {total_cbdd + total_bi}")
    
    # Create a pie chart showing overall distribution
    pie_data = pd.DataFrame([
        {'source': 'CbDD Paintings', 'count': total_cbdd, 'color': '#3498db'},
        {'source': 'Bildindex Images', 'count': total_bi, 'color': '#e67e22'}
    ])
    
    pie_chart = alt.Chart(pie_data).mark_arc(innerRadius=50).encode(
        theta=alt.Theta('count:Q'),
        color=alt.Color('source:N',
                       scale=alt.Scale(
                           domain=['CbDD Paintings', 'Bildindex Images'],
                           range=['#3498db', '#e67e22']
                       ),
                       title='Source'),
        tooltip=['source', 'count']
    ).properties(
        title='Total Items by Data Source (Top 3 Painters)',
        width=300,
        height=300
    )
    
    ipython_display(pie_chart)
    
else:
    print("⚠️ No biographies available. Run the previous cells first.")

📊 DATA COVERAGE COMPARISON: CbDD vs Bildindex


alt.FacetChart(...)


📋 Summary Statistics:
--------------------------------------------------
   Total CbDD paintings across 3 painters:    233
   Total Bildindex items across 3 painters:   8
   Combined cross-referenced items:           241


alt.Chart(...)

In [39]:
# =============================================================================
# CELL 15: Close Connection (run when done)
# =============================================================================

# Uncomment to close the connection
# con.close()
# print("✅ DuckDB connection closed")

print("💡 Tip: The database file 'baroque.duckdb' can be shared with collaborators.")
print("   They can open it with:")
print("   >>> con = duckdb.connect('baroque.duckdb')")
print("   >>> con.execute('SELECT * FROM paintings LIMIT 5').pl()")

💡 Tip: The database file 'baroque.duckdb' can be shared with collaborators.
   They can open it with:
   >>> con = duckdb.connect('baroque.duckdb')
   >>> con.execute('SELECT * FROM paintings LIMIT 5').pl()


In [40]:
# =============================================================================
# Entity-Relationship / Ontology Diagram for Baroque Ceiling Paintings Database
# =============================================================================
# Generates a presentation-ready graph showing all entities (tables) and their
# relationships, derived directly from the DuckDB schema and foreign keys.

import duckdb
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from matplotlib.patches import FancyBboxPatch
import networkx as nx
import numpy as np
from pathlib import Path

# ── Connect ──────────────────────────────────────────────────────────────────
DUCKDB_PATH = Path(r"c:/Users/thano/Documents/_Studium/KIT/DataStories/DataStories/baroque.duckdb")
con = duckdb.connect(str(DUCKDB_PATH), read_only=True)

# ── Gather row counts ───────────────────────────────────────────────────────
table_counts = {}
for row in con.execute("SHOW TABLES").fetchall():
    tname = row[0]
    cnt = con.execute(f"SELECT COUNT(*) FROM {tname}").fetchone()[0]
    table_counts[tname] = cnt

# ── Define entities (nodes) with metadata ────────────────────────────────────
# Two data sources: CbDD (primary) and Bildindex (secondary)
cbdd_entities = {
    "Painting":    {"table": "paintings",         "rows": table_counts["paintings"],         "key_attrs": ["label", "year", "lat/lon", "imageUrl", "method"]},
    "Person":      {"table": "persons",           "rows": table_counts["persons"],           "key_attrs": ["name", "person_type"]},
    "Building":    {"table": "buildings",         "rows": table_counts["buildings"],         "key_attrs": ["name", "function", "location_state"]},
    "Room":        {"table": "rooms",             "rows": table_counts["rooms"],             "key_attrs": ["name", "function"]},
    "Ensemble":    {"table": "ensembles",         "rows": table_counts["ensembles"],         "key_attrs": ["name"]},
    "Subject":     {"table": "subjects",          "rows": table_counts["subjects"],          "key_attrs": ["label", "source (ICONCLASS/AAT)"]},
}

bi_entities = {
    "BI_Item":     {"table": "bi_items",          "rows": table_counts["bi_items"],          "key_attrs": ["title", "url", "tier"]},
    "BI_GND\nOverlap": {"table": "bi_gnd_overlaps", "rows": table_counts["bi_gnd_overlaps"], "key_attrs": ["gnd_uri", "gnd_type"]},
}

# ── Define relationships (edges) ────────────────────────────────────────────
# Format: (source, target, label, cardinality, style)
relationships = [
    # CbDD core relationships
    ("Painting",  "Building",  "located in",           "N : 1",  "solid"),
    ("Painting",  "Room",      "displayed in",         "N : 1",  "solid"),
    ("Room",      "Building",  "part of",              "N : 1",  "solid"),
    ("Building",  "Ensemble",  "belongs to",           "N : 1",  "dashed"),

    # Person relationships via junction tables
    ("Person",    "Painting",  "painted\n(PAINTER)",             "N : M",  "solid"),
    ("Person",    "Painting",  "commissioned\n(COMMISSIONER)",   "N : M",  "solid"),
    ("Person",    "Painting",  "other roles*",         "N : M",  "dotted"),
    ("Person",    "Building",  "built / commissioned", "N : M",  "solid"),
    ("Person",    "Room",      "designed / built",     "N : M",  "solid"),

    # Subject relationship
    ("Painting",  "Subject",   "depicts\n(ICONCLASS / AAT)",   "N : M",  "solid"),

    # Cross-reference via GND
    ("Painting",  "BI_Item",   "linked via\nGND (painter)",    "N : M",  "dashed"),
    ("Building",  "BI_Item",   "linked via\nGND (building)",   "N : M",  "dashed"),
    ("BI_Item",   "BI_GND\nOverlap", "summarised in",         "N : 1",  "dashed"),
]

# ── Build the graph ─────────────────────────────────────────────────────────
G = nx.MultiDiGraph()

all_entities = {**cbdd_entities, **bi_entities}
for name, info in all_entities.items():
    G.add_node(name, **info)

for src, tgt, label, card, style in relationships:
    G.add_edge(src, tgt, label=label, cardinality=card, style=style)

# ── Layout — manual positions for a clean class-diagram look ─────────────────
pos = {
    # Top level: spatial hierarchy
    "Ensemble":    (-3.0,  3.0),
    "Building":    (-1.0,  3.0),
    "Room":        ( 1.5,  3.0),

    # Center: the main entity
    "Painting":    ( 0.0,  0.0),

    # Left: people
    "Person":      (-4.0,  0.0),

    # Right: subjects
    "Subject":     ( 4.0,  0.0),

    # Bottom: Bildindex cross-references
    "BI_Item":     ( 0.0, -3.0),
    "BI_GND\nOverlap": ( 3.5, -3.0),
}

# ── Colour palette ───────────────────────────────────────────────────────────
CBDD_COLOR    = "#4A90D9"   # blue
CBDD_FILL     = "#D6E6F5"
BI_COLOR      = "#27AE60"   # green
BI_FILL       = "#D5F5E3"
JUNCTION_CLR  = "#E67E22"   # orange for junction info
EDGE_COLORS   = {"solid": "#333333", "dashed": "#888888", "dotted": "#AAAAAA"}

# ── Draw ─────────────────────────────────────────────────────────────────────
fig, ax = plt.subplots(1, 1, figsize=(20, 14))
ax.set_xlim(-6.5, 6.5)
ax.set_ylim(-5.5, 5.5)
ax.set_aspect("equal")
ax.axis("off")

# Title
ax.text(0, 5.0, "Baroque Ceiling Paintings — Entity-Relationship Diagram",
        fontsize=20, fontweight="bold", ha="center", va="center",
        fontfamily="sans-serif", color="#222")
ax.text(0, 4.5, "Data sources: CbDD / NFDI4Culture (blue) · Bildindex (green) — linked via GND identifiers",
        fontsize=11, ha="center", va="center", color="#555", fontstyle="italic")

# Draw entity boxes
BOX_W, BOX_H = 2.6, 1.8
for name, (x, y) in pos.items():
    info = all_entities[name]
    is_bi = name.startswith("BI_")
    fill = BI_FILL if is_bi else CBDD_FILL
    edge_c = BI_COLOR if is_bi else CBDD_COLOR

    # Box
    rect = FancyBboxPatch(
        (x - BOX_W/2, y - BOX_H/2), BOX_W, BOX_H,
        boxstyle="round,pad=0.1",
        facecolor=fill, edgecolor=edge_c, linewidth=2.5,
        zorder=3
    )
    ax.add_patch(rect)

    # Entity name (header)
    display_name = name.replace("\n", " ")
    ax.text(x, y + BOX_H/2 - 0.30, display_name,
            fontsize=13, fontweight="bold", ha="center", va="center",
            color=edge_c, zorder=4, fontfamily="sans-serif")

    # Divider line
    ax.plot([x - BOX_W/2 + 0.15, x + BOX_W/2 - 0.15],
            [y + BOX_H/2 - 0.50, y + BOX_H/2 - 0.50],
            color=edge_c, linewidth=1, alpha=0.5, zorder=4)

    # Table name + row count
    ax.text(x, y + BOX_H/2 - 0.68, f"{info['table']}  ({info['rows']:,})",
            fontsize=8.5, ha="center", va="center", color="#555",
            zorder=4, fontstyle="italic", fontfamily="monospace")

    # Key attributes
    attrs = info.get("key_attrs", [])
    for i, attr in enumerate(attrs[:4]):
        ax.text(x, y + BOX_H/2 - 0.95 - i * 0.22, f"• {attr}",
                fontsize=8.5, ha="center", va="center", color="#333",
                zorder=4, fontfamily="sans-serif")

# Draw edges with labels
drawn_pairs = {}  # track (src,tgt) to offset parallel edges
for src, tgt, label, card, style in relationships:
    x1, y1 = pos[src]
    x2, y2 = pos[tgt]

    pair_key = tuple(sorted([src, tgt]))
    idx = drawn_pairs.get(pair_key, 0)
    drawn_pairs[pair_key] = idx + 1

    # Perpendicular offset for parallel edges
    dx, dy = x2 - x1, y2 - y1
    length = max(np.sqrt(dx**2 + dy**2), 0.01)
    perp_x, perp_y = -dy / length, dx / length
    offset = (idx - 0.5) * 0.25 if drawn_pairs[pair_key] > 1 else idx * 0.25

    # Shorten lines so they stop at box edges
    shrink = 1.35
    ratio = shrink / length
    sx = x1 + dx * ratio + perp_x * offset
    sy = y1 + dy * ratio + perp_y * offset
    ex = x2 - dx * ratio + perp_x * offset
    ey = y2 - dy * ratio + perp_y * offset

    linestyle = {"solid": "-", "dashed": "--", "dotted": ":"}[style]
    color = EDGE_COLORS[style]

    ax.annotate("",
                xy=(ex, ey), xytext=(sx, sy),
                arrowprops=dict(arrowstyle="-|>", color=color, lw=1.5,
                                linestyle=linestyle, shrinkA=0, shrinkB=0),
                zorder=2)

    # Edge label
    mx = (sx + ex) / 2 + perp_x * 0.15
    my = (sy + ey) / 2 + perp_y * 0.15
    ax.text(mx, my, f"{label}\n[{card}]",
            fontsize=7.5, ha="center", va="center",
            color=color, fontfamily="sans-serif",
            bbox=dict(boxstyle="round,pad=0.2", facecolor="white",
                      edgecolor="none", alpha=0.85),
            zorder=5)

# Junction table annotations (small badges near edges)
junction_info = [
    ("painting_persons", f"{table_counts['painting_persons']:,} links", -2.3, 0.65),
    ("painting_subjects", f"{table_counts['painting_subjects']:,} links", 2.3, 0.65),
    ("building_persons", f"{table_counts['building_persons']:,} links", -3.2, 1.8),
    ("room_persons", f"{table_counts['room_persons']:,} links", -1.5, 1.8),
    ("bi_painters", f"{table_counts['bi_painters']:,} links", -1.5, -1.5),
    ("bi_buildings", f"{table_counts['bi_buildings']:,} links", 1.0, -1.5),
    ("bi_subjects", f"{table_counts['bi_subjects']:,} links", 3.5, -1.5),
]

for tbl, info_text, jx, jy in junction_info:
    ax.text(jx, jy, f"⚡ {tbl}\n{info_text}",
            fontsize=7, ha="center", va="center", color=JUNCTION_CLR,
            fontweight="bold", fontfamily="sans-serif",
            bbox=dict(boxstyle="round,pad=0.25", facecolor="#FFF3E0",
                      edgecolor=JUNCTION_CLR, linewidth=1, alpha=0.9),
            zorder=6)

# Legend
legend_elements = [
    mpatches.Patch(facecolor=CBDD_FILL, edgecolor=CBDD_COLOR, linewidth=2, label="CbDD / NFDI4Culture Entity"),
    mpatches.Patch(facecolor=BI_FILL, edgecolor=BI_COLOR, linewidth=2, label="Bildindex Entity"),
    mpatches.Patch(facecolor="#FFF3E0", edgecolor=JUNCTION_CLR, linewidth=1.5, label="Junction / Link Table"),
    plt.Line2D([0], [0], color="#333", linewidth=1.5, linestyle="-", label="Direct relationship"),
    plt.Line2D([0], [0], color="#888", linewidth=1.5, linestyle="--", label="Cross-reference (via GND)"),
]
ax.legend(handles=legend_elements, loc="lower left", fontsize=9,
          frameon=True, fancybox=True, shadow=True, ncol=1,
          bbox_to_anchor=(-0.02, -0.02))

# Person roles summary box
roles_text = (
    "Person Roles in Dataset:\n"
    "─────────────────────────\n"
    "Painting: Painter, Commissioner,\n"
    "  Template Provider, Plasterer,\n"
    "  Designer, Architect, …\n"
    "Building: Commissioner, Architect,\n"
    "  Plasterer, Builder, Owner, …\n"
    "Room: Commissioner, Architect,\n"
    "  Painter, Plasterer, Builder, …\n"
    "─────────────────────────\n"
    f"Person types: Individual ({2772})\n"
    f"              Society/Org ({59})"
)
ax.text(5.8, 3.5, roles_text,
        fontsize=7.5, ha="left", va="top", fontfamily="monospace",
        color="#444",
        bbox=dict(boxstyle="round,pad=0.4", facecolor="#F8F8F8",
                  edgecolor="#CCC", linewidth=1),
        zorder=6)

plt.tight_layout()
plt.savefig("baroque_entity_relationship_diagram.png", dpi=200, bbox_inches="tight",
            facecolor="white", edgecolor="none")
plt.savefig("baroque_entity_relationship_diagram.pdf", bbox_inches="tight",
            facecolor="white", edgecolor="none")
plt.show()
print("✅ Saved: baroque_entity_relationship_diagram.png + .pdf")


ConnectionException: Connection Error: Can't open a connection to same database file with a different configuration than existing connections